## Start

### Import libs

In [ ]:
#Import libs 
import pandas as pd
import numpy as np
import statistics as st
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
from scipy.stats import gmean
from scipy import stats
import math
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as mlines
import plotly.graph_objects as go
import plotly.offline as go_offline
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
import seaborn as sns
from statistics import mean
import geopandas as gpd
from shapely.geometry import Point
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.python import keras
from xgboost import XGBRegressor
from datetime import datetime
import random
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, Conv1DTranspose, Conv2DTranspose
from tensorflow.keras.initializers import RandomUniform
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### Upload main data

In [ ]:
# # Loading the ACG_wells_JOINT_BEST_v10.csv file
# path = 'C:\\jupyter\\SPP\\input\\'
# data_init = pd.read_csv(path + 'ACG_wells_JOINT_BEST_v10.csv', sep=',')
# df = data_init.copy()
# df = df[1:]

In [ ]:
# #Select only neccessary data
# df_cln = df[[   'wellName', 'MD', 'BADPORLOG', 'Casings', 'FORMATION', 'DEVI', 'HAZI',
#                 'FLANK', 'FLANK1', 'FLANK2', 'NET', 'NET_VSH','FLUIDS',
#                 'LPERM', 'PHIT', 
#                 'GR_N', 'GRMATRIX', 'GRSHALE','VSH', 'NPSS', 'RHOB', 'RHOF', 'RHOMA', 
#                 'RDEEP',  'SON', 'SONSH', 
#                 'TVD_SCS','TST', 'X', 'Y']]
# #Fill up nan and -9999 values with 0
# df_cln = df_cln.fillna(0)
# df_cln = df_cln.replace(-9999, 0)
# df_cln = df_cln.replace('-9999', '0')
# #Assing proper datatypes for df
# dicttypes = {   'wellName':'string', 'MD':'float', 'BADPORLOG':'int', 'Casings':'float', 'FORMATION':'string','DEVI':'float', 'HAZI':'float',
#                 'FLANK':'int', 'FLANK1':'int', 'FLANK2':'int', 'NET':'int', 'NET_VSH':'int', 'FLUIDS':'int',
#                 'LPERM':'float','PHIT':'float',
#                 'GR_N':'float', 'GRMATRIX':'float', 'GRSHALE':'float', 'VSH':'float', 'NPSS':'float','RHOB':'float','RHOF':'float', 'RHOMA':'float',
#                 'RDEEP':'float', 'SON':'float', 'SONSH':'float',
#                 'TVD_SCS':'float', 'TST':'float', 'X':'float', 'Y':'float'}
# df_cln = df_cln.astype(dicttypes, errors='ignore')
# df_cln.loc[df_cln.FORMATION=='0', 'FORMATION']='None'
# # Save data to parquet
# df_cln.to_parquet('ACG_wells_JOINT_BEST_v10.parquet.gzip', compression='gzip')

In [2]:
#Loading metadata, distribution wells per Platforms and all the that.
def metadata_parquet_loading():
    path = 'C:\\jupyter\\SPP\\input\\'
    metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
    metadata = metadata_init.copy()
    metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
    metadata.Status = metadata.Status.str.strip()
    metadata.Status = metadata.Status.str.lower()
    metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
    metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
    metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
    metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
    metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
    metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
    metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
    metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
    metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

    #Read data from parquet
    path = 'C:\\jupyter\\SPP\\input\\'
    df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
    df_prq.rename(columns={'wellName':'well'}, inplace=True)
    df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
    # print('wells in df totally:', len(df_prq.well.unique()))
    # Filter data with bad_well_list 
    bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP', 'A01W_2']
    df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
    #Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
    df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
    df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
    df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
    # Unite of FU for each formation

    df_bal = df_prq[df_prq.FORMATION.str.contains('Balakhany')]
    df_bal.loc[df_bal.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
    df_bal.loc[df_bal.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
    df_bal = df_bal[df_bal.FORMATION_up.notna()]
    #Getting XY mean coords of Balakhany formation
    xy_coord_mean = df_bal[['well', 'FORMATION_up', 'X', 'Y']]
    xy_coord_mean = xy_coord_mean.groupby(['well', 'FORMATION_up']).agg({'X': 'mean', 'Y':'mean'}).reset_index()
    xy_coord_mean = xy_coord_mean.rename(columns={'X':'X_mean', 'Y':'Y_mean'})
    xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
    df_bal.rename(columns={'X':'X_traj', 'Y':'Y_traj'}, inplace=True)
    df_bal = df_bal.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
    return df_bal
df_bal = metadata_parquet_loading()

## List of functions

In [3]:
# Function to display in TST well plots with logging curves
def well_display_khtst( dataset, wellname, fmname, net_var, comments, 
                        ref_depth, fm_flag, depth_step, kh_include, print):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    fm_flag = 1 if you need a FORMATION_up, 0 if just a simple FORMATION
    depth_step = step for ticks on the diagramm
    kh_include = 1 if we have KHtst in dataset, 0 if there is not KHtst
    print = 1 if we want to print the plot
    """
    if fm_flag == 0:
        data = dataset[(dataset.well==wellname) & (dataset.FORMATION == fmname)]
    if fm_flag == 1:
        data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT'] 
    net = data[net_var]
    perm = data['LPERM']
    if kh_include == 1:
        kh = data['KHtst']
    else:
        data['KHtst'] = 0
        kh = data['KHtst']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = df_bal[(df_bal.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    ax[0].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.5, zorder=5)
    twin0.set_xlim(0, 1.5)
    ax[1].plot(rhob, depth, color='red') 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].set_xlim(1.65, 2.65)
    ax[1].grid(axis='y'), ax[1].grid(axis='x')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue')
    twin1.set_xlim(0.6, 0)
    # ax[2].plot(rdeep, depth, color='black'), ax[2].set_xscale('log'), ax[2].set_xlim(0.1, 50), ax[2].invert_yaxis(), ax[2].grid(axis='x', which='both')
    ax[2].plot(phit, depth, color='green', linestyle='dashed'), ax[2].set_xlim(0.3, 0), ax[2].grid(axis='x') 
    ax[2].invert_yaxis()
    ax[2].grid(axis='y')
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    ax[3].plot(perm, depth, color='purple', alpha=0.66), ax[3].set_xscale('log'), ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    ax[3].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.66)
    twin4 = ax[3].twiny()
    twin4.plot(kh, depth, color='black', alpha=1)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print == 1:
        path = 'C:\\jupyter\\SPP\\inputoutput\\wellplots\\'
        fig.savefig(path + fmname.replace(' ','') + '_' + wellname + '.png')
    else:
        pass
# Draw a map
def map_value_2plots(metadata, dataset, formation, value, color, multi_chr = 0.001, multi_azr = 0.001):
    """
    metadata, 
    dataset = dataset with X & Y, 
    formation = 'Balakhany VIII',  
    value = for example 'KHtst' or 'tst_interv'
    multi_chr = 0.001, multi_azr = 0.001
    """
    fig = make_subplots(rows=2, cols=1, subplot_titles=('crg: ' + str(multi_chr), 'azr: ' + str(multi_azr)), 
                        vertical_spacing = 0.025)
    azr_lst = ['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI']
    chg_lst = ['CHIRAG', 'DWG', 'DDGG', 'WEST CHIRAG']
    field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
    field_avg_coord_chg = field_avg_coord[field_avg_coord.field.isin(chg_lst)]
    field_avg_coord_azr = field_avg_coord[field_avg_coord.field.isin(azr_lst)] 
    df_chg = dataset[(dataset.FORMATION_up == formation) & (dataset.field.isin(chg_lst))]
    df_azr = dataset[(dataset.FORMATION_up == formation) & (dataset.field.isin(azr_lst))]
    fig.add_trace(go.Scatter(x=df_chg.X, y=df_chg.Y, customdata = df_chg[['well', value, color]],
                            marker=dict(color=df_chg[color], size=df_chg[value]*multi_chr, colorscale='Viridis_r',  showscale=True,
                            line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            mode='markers', hovertemplate="".join(["well:%{customdata[0]}, value:%{customdata[1]}, color:%{customdata[2]}<extra></extra>"])),
                            row=1, col=1)
    fig.add_trace(go.Scatter(x=field_avg_coord_chg.X_wellhead, y=field_avg_coord_chg.Y_wellhead, customdata = field_avg_coord_chg[['field']],
                            text=field_avg_coord_chg['field'], textposition="middle right",
                            marker=dict(color='rgb(0, 0,0)', size=12),
                            mode='markers+text', 
                            marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])),
                            row=1, col=1)
    fig.add_trace(go.Scatter(x=df_azr.X, y=df_azr.Y, customdata = df_azr[['well', value, color]],
                            marker=dict(color=df_azr[color], size=df_azr[value]*multi_azr, colorscale='Viridis_r',  showscale=False,
                            line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            mode='markers', hovertemplate="".join(["well:%{customdata[0]}, value:%{customdata[1]}, color:%{customdata[2]}<extra></extra>"])),
                            row=2, col=1)
    fig.add_trace(go.Scatter(x=field_avg_coord_azr.X_wellhead, y=field_avg_coord_azr.Y_wellhead, customdata = field_avg_coord_azr[['field']],
                            text=field_avg_coord_azr['field'], textposition="middle right",
                            marker=dict(color='rgb(0, 0,0)', size=12),
                            mode='markers+text', 
                            marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])),
                            row=2, col=1)
    fig.update_layout(  title_text= ('formation: ' + str(formation) + ' value: ' + str(value) + ' color: ' + str(color)),
                        autosize=True, width=1300, height=1400, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Calculation NTD
def ntd_calculation_big(dataset, desired_fm, net_var='NET'):
    df_lst = []
    for well_in_loop in tqdm(dataset.well.unique()[:]):
        well_lst = []
        data = dataset[(dataset.well==well_in_loop)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        for k in range(len(tst_top)):
            if (round(tst_top[k],1) == round(tst_bot[k],1)):
                h_tst = 0 
            elif (round(tst_bot[k],1) == round(tst_top[k]+0.1,1)):
                h_tst = 0
            else:
                h_tst = (round((tst_bot[k] - tst_top[k]),1))
                md_perm = []
                md_phit = []
                md_vsh = []
                for i in range(len(data)):
                    if round(data.iloc[i]['TST'],1) >= round(tst_top[k],1) and round(data.iloc[i]['TST'],1) <= round(tst_bot[k],1):
                        md_perm.append(data.iloc[i]['LPERM'])
                        md_phit.append(data.iloc[i]['PHIT'])
                        md_vsh.append(data.iloc[i]['VSH'])
                if len(md_perm) == 0:
                    md_perm.append(0)
                if len(md_phit) == 0:
                    md_phit.append(0)
                if len(md_vsh) == 0:
                    md_vsh.append(0)
                well_lst.append([data.iloc[0]['well'], h_tst, tst_top[k], tst_bot[k], round(mean(md_perm),0), round(mean(md_phit),2), round(mean(md_vsh),2)])
            df_tst = pd.DataFrame(well_lst, columns = ['well', 'h_tst', 'top_tst', 'bot_tst', 'md_perm_avg', 'md_phit_avg', 'md_vsh_avg'])
        df_lst.append(df_tst)
    ntd_bal = pd.concat(df_lst)
    ntd_bal['FORMATION_up'] = desired_fm
    return ntd_bal
def ntd_calculation_brief(dataset,well,desired_fm, net_var='NET'):
    data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
    data.iloc[0, 3] = 0
    data.iloc[-1, 3] = 0
    tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
    tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
    tops = zip(tst_top, tst_bot)
    df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
    df_htst['FORMATION_up'] = desired_fm
    df_htst['well'] = well
    df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
    df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
    return df_htst
# Calculation NTD zero
def ntd_calculation_zero(dataset,well,formation, net_var='NET'):
    data = dataset[(dataset.well==well) & (dataset.FORMATION_up==formation)]
    data.iloc[0, 3] = 1
    data.iloc[-1, 3] = 1
    tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)
                if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
    tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1) 
                if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
    tops_zero = zip(tst_zero_top, tst_zero_bot)
    df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
    df_zero_htst['FORMATION_up'] = formation
    df_zero_htst['well'] = well
    df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
    df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
    return df_zero_htst
# Print numerical table with layers
def ntd_numerical(dataset, wellname, fmname):
    """
    dataset = ntd_final
    """
    df = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname) ][['well','h_tst','top_tst', 'bot_tst','FORMATION_up']]
    q50 = df['h_tst'].quantile(q=0.5, interpolation='nearest')
    df['q50'] = q50
    return df
#Cleaning NET variable and making up NET_clp with clipped data
def ntd_htst_cleaning(dataset, cutoff):
    """
    dataset - any updated dataset like df_bal...
    cutoff - value in TST to remove layers with thickness below cutoff
    """
    df_list_ntd = []
    for well in tqdm(dataset.well.unique()):
        ntd_well = dataset[(dataset.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = df_bal[['well', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', 'FORMATION']]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    return net_clp
# Cleaning NET_clp variable from zero values with zero_samples <=cutoff
def ntd_htst_zero_cleaning(dataset_zero, dataset, cutoff, net_var1, net_var2):
    df_list_ntd_zero = []
    for well in tqdm(dataset_zero.well.unique()):
        ntd_well_zero = dataset_zero[(dataset_zero.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        df_list_ntd_zero.append(well_zero_sel)
    result = pd.concat(df_list_ntd_zero)
    return result
# View desired TST-interval
def net_view1(dataset, well, top, bot):
    dataset = dataset[dataset.well==well][['well','TST','GR_N', 'RHOB', 'NET','NET_clp']]
    return dataset[(dataset.TST >= top) & (dataset.TST <= bot)].head(50)
#TST sampling & TST KH curve calculation per formation/well
def proph_calculation(dataset, net_var):
    df_smpl_lst = []
    print('TST sampling calculation')
    for well_smpl in tqdm(dataset.well.unique()[:]):
        tst_sampl = dataset[dataset.well==well_smpl]['TST'].diff()
        df_new = dataset[dataset.well==well_smpl].join(tst_sampl, rsuffix='_smpl')    
        df_smpl_lst.append(df_new)
    df_bal_tst_smpl = pd.concat(df_smpl_lst)
    df_kh_lst_fm = []
    print('KHtst calculation')
    for fm_kh in ['Balakhany VIII', 'Balakhany X']:
        df_kh_lst = []
        for well_kh in tqdm(dataset.well.unique()[:]):
            well_tst_perm = df_bal_tst_smpl[(df_bal_tst_smpl.well==well_kh) & 
                                            (df_bal_tst_smpl.FORMATION_up==fm_kh)].sort_values(by='MD', ascending=False)
            well_tst_perm.loc[well_tst_perm[net_var] == 0, 'LPERM'] = 0
            well_tst_perm.loc[well_tst_perm[net_var] == 0, 'PHIT'] = 0
            well_tst_perm.loc[well_tst_perm[net_var] == 0, 'VSH'] = 0
            well_tst_perm['khtst'] = well_tst_perm.LPERM*well_tst_perm.TST_smpl
            well_tst_perm['phithtst'] = well_tst_perm.PHIT*well_tst_perm.TST_smpl
            well_tst_perm['vshhtst'] = well_tst_perm.VSH*well_tst_perm.TST_smpl
            well_tst_perm['KHtst'] = well_tst_perm.khtst.cumsum()
            well_tst_perm['PHITHtst'] = well_tst_perm.phithtst.cumsum()
            well_tst_perm['VSHHtst'] = well_tst_perm.vshhtst.cumsum()
            well_tst_perm = well_tst_perm.sort_values(by='MD')
            df_kh_lst.append(well_tst_perm)
        df_khlst = pd.concat(df_kh_lst)
        df_kh_lst_fm.append(df_khlst)
    df_khlst_fm = pd.concat(df_kh_lst_fm)
    # df_khlst_fm = df_khlst_fm.dropna()
    return df_khlst_fm[['well', 'FORMATION_up', 'MD', 'TST', 'TST_smpl','KHtst','PHITHtst','VSHHtst']]
# Comparison NET_clp and NET_clp2
def well_display_net(dataset, well, formation, net1='NET_clp', net2_flag=0, net2='NET_clp_v2'):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    if net2_flag == 0:
        fig, ax = plt.subplots(1,3, figsize=(4.5,8), sharey=True)
        ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), 5))
        ax[0].plot(grn, depth, color='green'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150), ax[0].grid(axis='y')
        well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
        for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
            ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
            ax[0].text(10, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
        ax[1].plot(net, depth, color='orange'), ax[1].set_xlim(0, 1), ax[1].grid(axis='y')
        ax[1].fill_betweenx(depth,net, color='orange', alpha=0.33)
        ax[2].plot(net_clp, depth, color='orange'), ax[2].set_xlim(0, 1), ax[2].grid(axis='y')
        ax[2].fill_betweenx(depth,net_clp, color='orange', alpha=0.33)
        fig.suptitle(well_sel.well.unique()[0], fontsize=14)
        fig.tight_layout()
    if net2_flag == 1:
        net_clp2 = well_sel[net2]
        fig, ax = plt.subplots(1,4, figsize=(6,8), sharey=True)
        ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), 5))
        ax[0].plot(grn, depth, color='green'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150), ax[0].grid(axis='y')
        well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
        for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
            ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
            ax[0].text(10, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
        ax[1].plot(net, depth, color='orange', lw=0.25), ax[1].set_xlim(0, 1), ax[1].grid(axis='y')
        ax[1].fill_betweenx(depth,net, color='orange', alpha=0.33)
        ax[2].plot(net_clp, depth, color='orange', lw=0.25), ax[2].set_xlim(0, 1), ax[2].grid(axis='y')
        ax[2].fill_betweenx(depth,net_clp, color='orange', alpha=0.33)
        ax[3].plot(net_clp2, depth, color='orange', lw=0.25), ax[3].set_xlim(0, 1), ax[3].grid(axis='y')
        ax[3].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
        fig.suptitle(well_sel.well.unique()[0], fontsize=14)
        fig.tight_layout()
    return fig.show()
# Run RFR model with train/test split
def rfr_train_test_split(train_dataset, gs_set, scorer, target='KHtst', rng=0.25, margin=0.005):
    """
    'train_ds', 
    'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 
    'grid_search', 
    'result_df', 
    'train_df', 
    'test_df'
    --------
    scorer = make_scorer(mse, greater_is_better=False) <- format scorer like this
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well','FORMATION_up',target]])
    x = np.array(train_dataset.drop(target, axis=1))
    num = random.randint(0,100)
    # num=42
    train_dataset_list.append(train_dataset.drop(['FORMATION_up', target], axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    # x_train_wells = x_train_init[:,2]
    # x_test_wells = x_test_init[:,2]
    y_train_wells = y_train_init[:,0:2]
    y_test_wells = y_test_init[:,0:2]
    x_train = x_train_init[:,2:]
    x_test = x_test_init[:,2:]
    y_train = y_train_init[:,2]
    y_test = y_test_init[:,2]
    # GridSearch for ML-model
    grid_rfr = RandomForestRegressor(n_jobs=-1, random_state=42)
    grid_calc_rfr = GridSearchCV(estimator = grid_rfr, param_grid = gs_set, scoring=scorer, cv = 5)
    grid_calc_rfr.fit(x_train, y_train)
    gd_sr_setting = grid_calc_rfr.best_params_
    grids_setting_list.append(gd_sr_setting)
    print('Grid_search: ', grid_rfr)
    # Applying Pipeline for ML-model
    rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(**gd_sr_setting, n_jobs=-1, random_state=42))])
    rfr.fit(x_train, y_train)
    y_pred_train = rfr.predict(x_train)
    y_pred_test = rfr.predict(x_test)
    # Reporting
    print('Pipeline: ', rfr.steps[1][1])
    up_range = rng + 1
    dwn_range = 1 - rng
    well_fm_train = pd.DataFrame(y_train_wells, columns=['well', 'FORMATION_up'])
    rfr_train = pd.DataFrame(zip(y_train, y_pred_train), columns=['actual','predict'])
    df_rfr_train = well_fm_train.join(rfr_train)
    df_rfr_train['l_limit'] = df_rfr_train.actual*dwn_range - margin
    df_rfr_train['h_limit'] = df_rfr_train.actual*up_range + margin
    df_rfr_train['qc'] = 'out'
    df_rfr_train['dataset'] = 'train'
    df_rfr_train.loc[(df_rfr_train.predict >= df_rfr_train.l_limit) & (df_rfr_train.predict <= df_rfr_train.h_limit), 'qc'] = 'in'
    well_fm_test = pd.DataFrame(y_test_wells, columns=['well', 'FORMATION_up'])
    rfr_test = pd.DataFrame(zip(y_test, y_pred_test), columns=['actual','predict'])
    df_rfr_test = well_fm_test.join(rfr_test)
    df_rfr_test['l_limit'] = df_rfr_test.actual*dwn_range - margin
    df_rfr_test['h_limit'] = df_rfr_test.actual*up_range + margin
    df_rfr_test['qc'] = 'out'
    df_rfr_test['dataset'] = 'test'
    df_rfr_test.loc[(df_rfr_test.predict >= df_rfr_test.l_limit) & (df_rfr_test.predict <= df_rfr_test.h_limit), 'qc'] = 'in'
    df_rfr_result = pd.concat([df_rfr_train,df_rfr_test])
    df_rfr_result['diff'] = (df_rfr_result.actual - df_rfr_result.predict).round(3)
    metrics_dict = {    'r2_train':     r2(y_train, y_pred_train).round(2), 
                        'r2_test':      r2(y_test, y_pred_test).round(2),
                        'mae_train':    mae(y_train, y_pred_train).round(2), 
                        'mae_test':     mae(y_test, y_pred_test).round(2),
                        'train_in':     df_rfr_train['qc'].value_counts(normalize=True)['in'].round(2),
                        'test_in':      df_rfr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    feature_imp = pd.Series(rfr.steps[1][1].feature_importances_, index=train_dataset_list[0]).sort_values(ascending=True)
    return {'train_ds':train_dataset_list[0], 
            'metrics':metrics_dict, 
            'grid_search' : grids_setting_list, 
            'result_df' : df_rfr_result,
            'train_df' : df_rfr_train,
            'test_df' : df_rfr_test,
            'feature_imp' : feature_imp}
# Run RFR model with loop
def rfr_loop(dataset, fmname, target, hyperdict, rng, margin):
    """
    'train_ds', 'train_ftrs', 'result_df', 'grid_search', 'metrics'
    """
    y_test_lst = []
    y_pred_test_lst = []
    well_exclude_lst = []
    fm_exclude_lst = []
    gs_settings_lst = []
    metrics_r2_lst = []
    metrics_mae_lst = []
    ftr_imp_lst = []
    for i in tqdm(range(len(dataset))[:]):
        #Making up the feature and target datasets
        df_wo_well = dataset.drop([i])
        well_exclude = dataset.iloc[i]['well']
        well_exclude_lst.append(well_exclude)
        fm_exclude = dataset.iloc[i][fmname]
        fm_exclude_lst.append(fm_exclude)
        y_train = np.array(df_wo_well[target])
        x_train = np.array(df_wo_well.drop(['well',fmname, target], axis=1))
        well_train = np.array(df_wo_well['well'])
        y_test = np.array(dataset.iloc[i][target])
        y_test_lst.append(y_test)
        x_test = np.array(dataset.drop(['well', fmname, target], axis=1).iloc[i])
        # Statement of ML-model
        rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(**hyperdict, n_jobs=-1, random_state=42))])                                                                                  
        # Fitting the ML-model
        rfr.fit(x_train, y_train)
        y_pred_train = rfr.predict(x_train)
        y_pred_test = rfr.predict([x_test])
        y_pred_test_lst.append(y_pred_test[0])
        # Metrics computation for the ML-model
        r2_train = r2(y_train, y_pred_train).round(5)
        mae_train = mae(y_train, y_pred_train)
        metrics_r2_lst.append(r2_train)
        metrics_mae_lst.append(mae_train.round(5))
        feature_imp = pd.Series(rfr.steps[1][1].feature_importances_, index=df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist()).sort_values(ascending=True)
        ftr_imp_lst.append(feature_imp)
    # Building up of dataframe
    print(rfr.steps[1][1])
    res_rfr_sha = pd.DataFrame( zip(y_test_lst, y_pred_test_lst, well_exclude_lst, fm_exclude_lst, metrics_r2_lst, metrics_mae_lst, ftr_imp_lst), 
                            columns = ['actual','predict','well', 'FORMATION_up','metrics_r2', 'metrics_mae','features_imp'])
    res_rfr_sha['l_range'] = res_rfr_sha.actual*(1-rng) - margin 
    res_rfr_sha['h_range'] = res_rfr_sha.actual*(1+rng) + margin
    res_rfr_sha['qc'] = 'out'
    res_rfr_sha.loc[(res_rfr_sha.predict >= res_rfr_sha.l_range) & (res_rfr_sha.predict <= res_rfr_sha.h_range), 'qc'] = 'in'
    wells_tot = res_rfr_sha.shape[0]
    wells_unpred = res_rfr_sha['qc'].value_counts()['out']
    wells_unpred_vv = (res_rfr_sha['qc'].value_counts()['out']/res_rfr_sha.shape[0]).round(3)
    try:
        wells_pred = res_rfr_sha['qc'].value_counts()['in']
        wells_pred_vv =  (res_rfr_sha['qc'].value_counts()['in']/res_rfr_sha.shape[0]).round(3)
    except:
        wells_pred = 0
        wells_pred_vv = 0
    res_rfr_sha['diff'] = res_rfr_sha.actual - res_rfr_sha.predict
    res_rfr_sha = res_rfr_sha[['well','FORMATION_up','actual','predict', 'diff', 'l_range', 'h_range', 'qc', 'metrics_r2', 'metrics_mae', 'features_imp']]
    types_dict = {'actual': 'float64', 'predict': 'float64', 'diff': 'float64', 'l_range': 'float64', 'h_range': 'float64'}
    res_rfr_sha = res_rfr_sha.astype(types_dict)
    res_rfr_sha = res_rfr_sha.round({'actual': 3, 'predict': 3, 'diff': 3})
    metrics_dict = {    'wells_total':          wells_tot, 
                        'wells_unpred':         wells_unpred,
                        'wells_unpred_v/v':     wells_unpred_vv,
                        'wells_pred':           wells_pred,
                        'wells_pred_v/v':       wells_pred_vv
                    }
    return {    'train_ds': dataset.columns.tolist(),
                'train_ftrs': df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist(),
                'result_df': res_rfr_sha,
                'grid_search' : hyperdict,
                'metrics':metrics_dict,
                'feature_imp' : feature_imp
            }
# Run XGBR model with loop 
def xgbr_loop(dataset, fmname, target, hyperdict, rng, margin):
    """
    'train_ds', 'train_ftrs', 'result_df', 'grid_search', 'metrics'
    """
    y_test_lst = []
    y_pred_test_lst = []
    well_exclude_lst = []
    fm_exclude_lst = []
    gs_settings_lst = []
    metrics_r2_lst = []
    metrics_mae_lst = []
    ftr_imp_lst = []
    for i in tqdm(range(len(dataset))[:]):
        #Making up the feature and target datasets
        df_wo_well = dataset.drop([i])
        well_exclude = dataset.iloc[i]['well']
        well_exclude_lst.append(well_exclude)
        fm_exclude = dataset.iloc[i][fmname]
        fm_exclude_lst.append(fm_exclude)
        y_train = np.array(df_wo_well[target])
        x_train = np.array(df_wo_well.drop(['well',fmname, target], axis=1))
        well_train = np.array(df_wo_well['well'])
        y_test = np.array(dataset.iloc[i][target])
        y_test_lst.append(y_test)
        x_test = np.array(dataset.drop(['well', fmname, target], axis=1).iloc[i])
        xgbr = Pipeline([("scaler",StandardScaler()),("xgbr",XGBRegressor(**hyperdict, n_jobs=-1, random_state=42))])
        # Fitting the ML-model
        xgbr.fit(x_train, y_train)
        y_pred_train = xgbr.predict(x_train)
        y_pred_test = xgbr.predict([x_test])
        y_pred_test_lst.append(y_pred_test[0])
        # Metrics computation for the ML-model
        r2_train = r2(y_train, y_pred_train).round(5)
        mae_train = mae(y_train, y_pred_train)
        metrics_r2_lst.append(r2_train)
        metrics_mae_lst.append(mae_train.round(5))
        feature_imp = pd.Series(xgbr.steps[1][1].feature_importances_, index=df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist()).sort_values(ascending=True)
        ftr_imp_lst.append(feature_imp)

    # Building up of dataframe
    print(xgbr.steps[1][1])
    res_rfr_sha = pd.DataFrame( zip(y_test_lst, y_pred_test_lst, well_exclude_lst, fm_exclude_lst, metrics_r2_lst, metrics_mae_lst, ftr_imp_lst), 
                            columns = ['actual','predict','well', 'FORMATION_up','metrics_r2', 'metrics_mae','features_imp'])
    res_rfr_sha['l_range'] = res_rfr_sha.actual*(1-rng) - margin 
    res_rfr_sha['h_range'] = res_rfr_sha.actual*(1+rng) + margin 
    res_rfr_sha['qc'] = 'out'
    res_rfr_sha.loc[(res_rfr_sha.predict >= res_rfr_sha.l_range) & (res_rfr_sha.predict <= res_rfr_sha.h_range), 'qc'] = 'in'
    wells_tot = res_rfr_sha.shape[0]
    wells_unpred = res_rfr_sha['qc'].value_counts()['out']
    wells_unpred_vv = (res_rfr_sha['qc'].value_counts()['out']/res_rfr_sha.shape[0]).round(3)
    try:
        wells_pred = res_rfr_sha['qc'].value_counts()['in']
        wells_pred_vv =  (res_rfr_sha['qc'].value_counts()['in']/res_rfr_sha.shape[0]).round(3)
    except:
        wells_pred = 0
        wells_pred_vv = 0
    res_rfr_sha['diff'] = res_rfr_sha.actual - res_rfr_sha.predict
    res_rfr_sha = res_rfr_sha[['well','FORMATION_up','actual','predict', 'diff','l_range', 'h_range', 'qc', 'metrics_r2', 'metrics_mae', 'features_imp']]
    types_dict = {'actual': 'float64', 'predict': 'float64', 'diff': 'float64', 'l_range': 'float64', 'h_range': 'float64'}
    res_rfr_sha = res_rfr_sha.astype(types_dict)
    res_rfr_sha = res_rfr_sha.round({'actual': 0, 'predict': 0, 'diff': 0})
    metrics_dict = {    'wells_total':          wells_tot, 
                        'wells_unpred':         wells_unpred,
                        'wells_unpred_v/v':     wells_unpred_vv,
                        'wells_pred':           wells_pred,
                        'wells_pred_v/v':       wells_pred_vv
                    }
    return {    'train_ds': dataset.columns.tolist(),
                'train_ftrs': df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist(),
                'result_df': res_rfr_sha,
                'grid_search' : hyperdict,
                'metrics':metrics_dict,
                'feature_imp' : feature_imp
            }
# Display results of ML-modeling
def xplot_qc(dataset, dataframe, max_val, rng=0.25):
    fig1_ml = px.scatter(dataset[dataframe], x='actual', y='predict', 
                        color='qc', 
                        hover_data=['well'], 
                        width=400, height=400,
                        #  color_discrete_sequence=["red", "green"]
                        )
    up_range = rng+1
    dwn_range = 1- rng
    fig1_ml.update_traces(marker=dict(size=10,opacity=0.75,line=dict(color='rgb(47, 57, 61)', width=1)))
    fig2_ml=px.line(x=[0,max_val], y=[0,max_val])
    fig2_1_ml=px.line(x=[0,max_val], y=[0,max_val*up_range])
    fig2_2_ml=px.line(x=[0,max_val], y=[0,max_val*dwn_range])
    fig2_ml.update_traces(line=dict(color = 'blue'))
    fig2_1_ml.update_traces(line=dict(color = 'blue', dash='dash'))
    fig2_2_ml.update_traces(line=dict(color = 'blue', dash='dash'))
    fig3_ml = go.Figure(data = fig1_ml.data + fig2_ml.data + fig2_1_ml.data + fig2_2_ml.data)
    fig3_ml.update_layout(  title = 'Comparison Actual vs Pred' + 
                                    ' QC_train: ' + str(dataset['metrics']['train_in']) +
                                    ' QC_test: ' + str(dataset['metrics']['test_in']),
                            width=600,height=400, xaxis_title='actual', yaxis_title='predict',
                            margin=dict(l=10,r=10,b=10,t=40))
    return fig3_ml.show()
# Calculate weighted avg properties 
def avg_prop_calculation(dataset_ntd, dataset, formation):
    well_data = []
    well_formation = formation
    for well in tqdm(dataset_ntd.well.unique()):
        # print(well)
        ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
        well_avgprop_sel = dataset[(dataset.well==well)]
        well_phit = []
        well_phit10 = []
        well_phit50 = []
        well_phit90 = []
        well_vsh = []
        well_vsh10 = []
        well_vsh50 = []
        well_vsh90 = []
        well_gperm = []
        well_h = []
        for layers in range(len(ntd_well_avgprop.well)):
            ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
            ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
            ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
            phit_lst = []
            vsh_lst = []
            perm_lst = []
            for depth in range(len(well_avgprop_sel.TST)):
                well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                    phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                    vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                    perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
            well_phit.append(mean(phit_lst)*ntd_h)
            well_phit10.append(np.quantile(phit_lst, 0.1)*ntd_h)
            well_phit50.append(np.quantile(phit_lst, 0.5)*ntd_h)
            well_phit90.append(np.quantile(phit_lst, 0.9)*ntd_h)
            well_vsh.append(mean(vsh_lst)*ntd_h)
            well_vsh10.append(np.quantile(vsh_lst, 0.1)*ntd_h)
            well_vsh50.append(np.quantile(vsh_lst, 0.5)*ntd_h)
            well_vsh90.append(np.quantile(vsh_lst, 0.9)*ntd_h)
            well_gperm.append(gmean(perm_lst)*ntd_h)
            well_h.append(ntd_h)
        well_phit_wavg = sum(well_phit)/sum(well_h)
        well_phit10_wavg = sum(well_phit10)/sum(well_h)
        well_phit50_wavg = sum(well_phit50)/sum(well_h)
        well_phit90_wavg = sum(well_phit90)/sum(well_h)
        well_vsh_wavg = sum(well_vsh)/sum(well_h)
        well_vsh10_wavg = sum(well_vsh10)/sum(well_h)
        well_vsh50_wavg = sum(well_vsh50)/sum(well_h)
        well_vsh90_wavg = sum(well_vsh90)/sum(well_h)
        well_perm_wavg = sum(well_gperm)/sum(well_h)
        well_hmax = max(well_h)
        well_h_p50 = np.quantile(well_h, 0.5)
        well_layers_count =len(well_h)
        well_hsum = sum(well_h)
        well_data.append([  well, well_formation, 
                            well_hmax, well_h_p50, well_layers_count, well_hsum,
                            well_phit_wavg, well_phit10_wavg, well_phit50_wavg, well_phit90_wavg,
                            well_vsh_wavg, well_vsh10_wavg, well_vsh50_wavg, well_vsh90_wavg,
                            well_perm_wavg])
    result = pd.DataFrame(well_data, columns=[  'well','FORMATION_up',
                                                'htst_max', 'htst_p50','htst_count', 'htst_sum',            
                                                'phit_wavg', 'phit10_wavg','phit50_wavg','phit90_wavg',
                                                'vsh_wavg', 'vsh10_wavg', 'vsh50_wavg', 'vsh90_wavg',
                                                'perm_wavg'])
    return result
# Euclidian dist calculation with prop
def dist_prop_calc(dataset, dist_formation, dist_cutoff, value):
    """
    dataset have to contain 'X_mean', 'Y_mean', 'TVD_SCS' and 'KHtst', if you assing value as KHtst
    """
    data = dataset[(dataset.FORMATION_up == dist_formation)]
    row_name = data.well.reset_index().drop(['index'], axis=1)
    distance_fm = pd.DataFrame(euclidean_distances(data[['X_mean', 'Y_mean', 'TVD_SCS']]), columns=list(data.well))
    distance_fm_well = distance_fm.join(row_name).set_index('well')
    distance_fm_well.reset_index(inplace=True)
    def well_kh_accum(wells, dataset, kh_formation):
        well_kh_accum = []
        well_x_accum = []
        well_y_accum = []
        for i in wells:
            well_kh_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)][value].reset_index())    
            well_x_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['X_mean'].reset_index())
            well_y_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['Y_mean'].reset_index())
        well_kh3 = pd.concat(well_kh_accum).T[1:]
        well_kh3.columns = [value + '_1',value + '_2', value + '_3']
        well_x3 = pd.concat(well_x_accum).T[1:]
        well_x3.columns = ['x1','x2','x3']
        well_y3 = pd.concat(well_y_accum).T[1:]
        well_y3.columns = ['y1','y2','y3']
        final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                            well_x3.reset_index().drop('index',axis=1), 
                            well_y3.reset_index().drop('index',axis=1)], axis=1)
        return final
    df_collect = []
    for num, well_name in enumerate(distance_fm_well.well[:]):
        well_dist3 = distance_fm_well[distance_fm_well.well == well_name].T[1:].sort_values(by=num)
        well_dist3_s2 = well_dist3[well_dist3[num] > dist_cutoff][:3].reset_index()
        well_dist3_tuple = tuple(well_dist3_s2['index'])
        well_dist3_res = well_dist3_s2.T[1:].reset_index().drop('index', axis=1)   
        well_name3_res = well_dist3_s2.T[:1].reset_index().drop('index', axis=1)
        well_kh3_res = well_kh_accum(well_dist3_tuple,dataset, dist_formation)
        well_dist3_res.columns =['dist1', 'dist2', 'dist3']
        well_name3_res.columns =['well1', 'well2', 'well3']
        concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
        result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
        df_collect.append(result)     
    df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
    df_well_kh_dist['FORMATION_up'] = dist_formation
    return df_well_kh_dist
# Feature importance bar chart for 1-to-all algorithm
def feature_imp_loop(dataset, wellname, fmname, xsize, ysize):
    # dataset = test['result_df']
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    ftr_imp = data['features_imp'].values[0]
    f, ax = plt.subplots(figsize=(xsize, ysize))
    ftr_imp.plot.barh()
    ax.set_title('RFR feature imp  ' + wellname + ' ' + fmname)
    ax.tick_params(axis='y', labelsize=8, rotation=0)
    return f.show()
# Save datafram to csv
def save_tocsv(dataframe, filename, flag):
    if flag == 1:
        # Saving avg_prop dataframe to .csv
        path = 'C:\\jupyter\\SPP\\inputoutput\\'
        dataframe.to_csv(path + filename)
    else:
        pass
# Feature importance bar chart for split dataframe
def feature_imp_split(dataset, xsize, ysize):
    fig, ax = plt.subplots(figsize=(xsize, ysize))
    ax = dataset.plot.barh()
    ax.set_title("RFR Feature Importances")
    ax.tick_params(axis='y', labelsize=9, rotation=0)
    ax.figure.tight_layout()
    return fig.show()
# Logging results of ml
def write_res_file(finename, comments, target, trainds, metrics, gridsearch):
    with open(finename, 'a') as file:
        # Get the current date and time
        current_datetime = datetime.now()
        # Write the result to the file
        file.write(f'\n{current_datetime} \n {comments} target: {target}')
        file.write(f'\n training_ds_{trainds} \n metrics_{[metrics]} \n grid_search_{gridsearch}')
    file.close()
# Remover categorical values from datasets
def cat_finder(dataset):
    """
    cat_list: categorical columns to drop out
    get_dum_list: categorical columns to run via pd.get_dummies
    """
    cat_list = []
    gm_list = []
    for col in dataset.columns:
        # print(i)
        if dataset[col].dtype == 'string':
            cat_list.append(col)
            if col != 'well':
                gm_list.append(col)
    # return {'cat_list':cat_list,
    #         'get_dum_list': gm_list}
    return cat_list, gm_list
# Display results of ML-modeling ver2
def xplot_qc2(data, max_val, rng, margin, round):
    data = data.round({'actual': round, 'predict': round, 'diff': round})
    ds_train = data[data.dataset == 'train']
    ds_test = data[data.dataset == 'test']
    up_range = rng + 1
    dwn_range = 1 - rng
    colors = {'in': 'green', 'out': 'red'}
    qc_colors_tr = [colors[qc] for qc in ds_train.qc]
    qc_colors_ts = [colors[qc] for qc in ds_test.qc]
    scatter_train = go.Scatter( x=ds_train.actual, y=ds_train.predict,
                                mode='markers',
                                marker=dict(color=qc_colors_tr, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                customdata = ds_train[['well','actual','predict','diff', 'FORMATION_up']],
                                hovertemplate="".join(
                                ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]},d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                                )
    scatter_test = go.Scatter(  x=ds_test.actual, y=ds_test.predict, 
                                mode='markers',
                                marker=dict(color=qc_colors_ts, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                customdata = ds_test[['well','actual','predict','diff', 'FORMATION_up']],
                                hovertemplate="".join(
                                ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]},d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                                )
    line_trace_up = go.Scatter(x=[0, max_val], y=[0 + margin, max_val*up_range + margin], mode='lines+markers', line=dict(color='blue'))
    line_trace_dw = go.Scatter(x=[0, max_val], y=[0 - margin, max_val*dwn_range - margin], mode='lines+markers', marker=dict(color='blue'))
    fig = make_subplots(rows=1, cols=2, subplot_titles=('train ds', 'test ds'))
    fig.add_trace(scatter_train,  row=1, col=1)
    fig.add_trace(line_trace_up,  row=1, col=1)
    fig.add_trace(line_trace_dw,  row=1, col=1)
    fig.update_xaxes(title_text='actual', row=1, col=1)
    fig.update_yaxes(title_text='predict', row=1, col=1)
    fig.add_trace(scatter_test,  row=1, col=2)
    fig.add_trace(line_trace_up,  row=1, col=2)
    fig.add_trace(line_trace_dw,  row=1, col=2)
    fig.update_xaxes(title_text='actual', row=1, col=2)
    fig.update_yaxes(title_text='predict', row=1, col=2)
    fig.update_layout(  title_text= ('rfr_train_test_split'), width=900, height=450, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Display results of ML-modeling ver2 via loop    
def xplot_qc2_loop(data, max_val, rng, margin=0.005):
    data = data.round({'actual': 3, 'predict': 3, 'diff ': 3})
    up_range = rng + 1
    dwn_range = 1 - rng
    colors = {'in': 'green', 'out': 'red'}
    qc_colors = [colors[qc] for qc in data.qc]
    scatter = go.Scatter( x=data.actual, y=data.predict,
                            mode='markers',
                            marker=dict(color=qc_colors, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            customdata = data[['well','actual','predict', 'diff', 'FORMATION_up']],
                            hovertemplate="".join(
                            ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]}, d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                            )
    fig = go.Figure()
    line_up = go.Scatter(x=[0, max_val], y=[0 + margin, max_val*up_range + margin], mode='lines+markers', line=dict(color='blue'))
    line_dw = go.Scatter(x=[0, max_val], y=[0 - margin, max_val*dwn_range - margin], mode='lines+markers', marker=dict(color='blue'))
    fig.add_trace(scatter)
    fig.add_trace(line_up)
    fig.add_trace(line_dw)
    fig.update_xaxes(title_text='actual')
    fig.update_yaxes(title_text='predict')
    fig.update_layout(  title_text= ('rfr_loop'), width=450, height=450, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Display results of ML-modeling on map
def map_qc(metadata, data, fmname, scale):
    data['diff'] = abs(data['diff'])
    data = data[data.FORMATION_up == fmname]
    data_in = data[data.qc=='in']
    data_out = data[data.qc=='out']
    field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
    platform  = go.Scatter(         x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                                    text=field_avg_coord['field'], textposition="middle right",
                                    marker=dict(color='rgb(0, 0,0)', size=12),
                                    mode='markers+text', 
                                    marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])
                                    )
    scatter_data_in = go.Scatter(   x=data_in.X, y=data_in.Y,
                                    mode='markers',
                                    marker=dict(symbol='circle', color='green', size=data_in['actual']*scale,
                                    opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)
                                    ),
                                    customdata = data_in[['well', 'diff']],
                                    hovertemplate="".join(["well:%{customdata[0]}, diff:%{customdata[1]}<extra></extra>"])
                                    )
    scatter_data_out = go.Scatter(  x=data_out.X, y=data_out.Y, 
                                    mode='markers',
                                    marker=dict(symbol='diamond', color='red', size=data_out['diff']*scale,
                                    opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                    customdata = data_out[['well', 'diff']],
                                    hovertemplate="".join(["well:%{customdata[0]}, diff:%{customdata[1]}<extra></extra>"])
                                    )
    fig = go.Figure()
    fig.add_trace(platform)
    fig.add_trace(scatter_data_in)
    fig.add_trace(scatter_data_out)
    fig.update_layout(title_text= ('rfr_train_test_split'),autosize=True, width=1000, height=600, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Pairplot new version
def pairplot_special(dataset, xsize, ysize, flag=1):
    if flag == 1:
        def corrfunc(x, y, **kws):
            r, _ = stats.pearsonr(x, y)
            ax = plt.gca()
            ax.annotate("r = {:.2f}".format(r),
                        xy=(.1, .9), xycoords=ax.transAxes)
        sns.set_context(rc={'axes.labelsize':10, 'lines.linewidth': 0.75})
        g = sns.PairGrid(dataset)
        g.fig.set_size_inches(xsize,ysize)
        g.set(xticklabels=[], yticklabels=[]) 
        g.map_upper(plt.scatter, s=10, alpha=0.5)
        g.map_diag(sns.distplot, kde=False)
        g.map_lower(sns.kdeplot, cmap="Blues_d")
        g.map_lower(corrfunc)
    else:
        pass
# Columns reorder for better display of variables
def columns_reorder(dataset, selected_column):
    new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
    dataset = dataset[new_order]
    return dataset
# Just simple x-plot for 1 dataframe
def log_map_plot(dataframe, x_var, y_var, min_val, max_val):
    fig = go.Figure()
    scatter = go.Scatter(   x=dataframe[x_var], y=dataframe[y_var], 
                            mode='markers',
                            marker=dict(color='orange', size=10, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            customdata = dataframe[['well',x_var,y_var]],
                            hovertemplate="".join(
                            ["w:%{customdata[0]},x:%{customdata[1]}, y:%{customdata[2]}<extra></extra>"])
                            )
    line = go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode='lines', line=dict(color='blue'))
    fig.add_trace(scatter)
    fig.add_trace(line)
    fig.update_layout(  title_text= ('scatter plot'), width=600, height=600, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Joining main and additional dataframes for predictions
def join_add_df_prediction(base_dataframe, add_dataframe, target_var):
    """
    Both dataframes have contain 'well' & 'FORMATION_up' for joining
    """
    join_dataframe = base_dataframe.set_index(['well','FORMATION_up']).join(add_dataframe.set_index(['well','FORMATION_up'])).reset_index()
    col_names, gm_list = cat_finder(join_dataframe)
    df_corr = join_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(join_dataframe[gm_list], columns=gm_list, drop_first=True)
    mem_cell.rename(columns={'FORMATION_up_Balakhany X':'FORMATION_up_gm'},inplace=True)
    join_dataframe_gm = pd.concat([join_dataframe, mem_cell], axis=1)
    return df_corr, join_dataframe_gm
# Preparation dataframes for pairplot and for predictions
def join_df_prediction(base_dataframe, target_var):
    def columns_reorder(dataset, selected_column):
        new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
        dataset = dataset[new_order]
        return dataset
    def cat_finder(dataset):
        """
        cat_list: categorical columns to drop out
        get_dum_list: categorical columns to run via pd.get_dummies
        """
        cat_list = []
        gm_list = []
        for col in dataset.columns:
            # print(i)
            if dataset[col].dtype == 'string':
                cat_list.append(col)
                if col != 'well':
                    gm_list.append(col)
        # return {'cat_list':cat_list,
        #         'get_dum_list': gm_list}
        return cat_list, gm_list
    col_names, gm_list = cat_finder(base_dataframe)
    df_corr = base_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(base_dataframe[gm_list], columns=gm_list, drop_first=True)
    mem_cell.rename(columns={'FORMATION_up_Balakhany X':'FORMATION_up_gm'},inplace=True)
    dataframe = pd.concat([base_dataframe, mem_cell], axis=1)
    return df_corr, dataframe
# Function to calculate grid_search via train_split
def run_rfr_train_test_split(dataset, gs_set, scorer, target, rng, margin, logtxt_name, comment, xplot_flag, ftr_imp_flag):
    model_res = rfr_train_test_split(dataset, gs_set, scorer, target, rng, margin)
    write_res_file(logtxt_name, comment, target, 
                    model_res['train_ds'], model_res['metrics'], model_res['grid_search'])
    print('train_ds: ', model_res['train_ds'])
    print('metrics: ', model_res['metrics'])
    print('grid_search: ', model_res['grid_search'])
    model_res_hyper_par = model_res['grid_search'][0]
    if xplot_flag == 1:
        xplot_qc2(dataset['result_df'], 0.3, 0.05, margin)
    else:
        pass
    if ftr_imp_flag == 1:
        feature_imp_split(dataset['feature_imp'], 6, 4)
    else:
        pass
    return model_res_hyper_par
# Function to calculate target via 1-to-all
def run_rfr_1_to_all(dataset, hyperdict, target, rng, margin, logtxt_name, comment, xplot_flag, max_val, ftr_imp_flag):
    loop_res = rfr_loop(dataset, 'FORMATION_up', target, hyperdict, rng, margin)
    write_res_file(logtxt_name, comment, target, loop_res['train_ds'], loop_res['metrics'], loop_res['grid_search'])
    loop_res_pred = loop_res['result_df']
    print('train_ftrs: ',loop_res['train_ftrs'])
    print('metrics: ',loop_res['metrics'])
    if xplot_flag == 1:
        xplot_qc2_loop(loop_res['result_df'], max_val, rng, margin)
    else:
        pass
    if ftr_imp_flag == 1:
        feature_imp_split(loop_res['feature_imp'], 6, 4)
    else:
        pass
    return loop_res_pred

## Data preparation

### Well quality calculation & printing of diagrams 

In [ ]:
def well_clean_v2():
    #Counting of bad quality logs
    bal8_list = [   'Balakhany VIII sand', 'Balakhany VIII 20',   'Balakhany VIII 10',   'Balakhany VIII 25',
                    'Balakhany VIII 15',    'Balakhany VIII 5']
    well_tot8 = []
    well_zero8 = []
    well_clean8 = []
    for j in df_bal[(df_bal.FORMATION.isin(bal8_list) & (df_bal.PHIT>0))].well.unique():
        phit_zero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany VIII')]))
        phit_nonzero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany VIII') & (df_bal.PHIT > 0)]))
        well_tot8.append(j)
        if round((phit_nonzero/phit_zero),2)<=0.90:
            well_zero8.append(j)
        else:
            well_clean8.append(j)
            # well_display_ntd(df_bal, j, 'Balakhany VIII', 'NET', round((phit_nonzero/phit_zero),2), 1) #printing well plots with high quality logs

    bal10_list = ['Balakhany X', 'Balakhany X sand', 'Balakhany X 40', 'Balakhany X 20', 'Balakhany X 50']
    well_tot10 = []
    well_zero10 = []
    well_clean10 = []
    for j in df_bal[(df_bal.FORMATION.isin(bal10_list) & (df_bal.PHIT>0))].well.unique():
        phit_zero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany X')]))
        phit_nonzero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany X') & (df_bal.PHIT > 0)]))
        well_tot10.append(j)
        if round((phit_nonzero/phit_zero),2)<=0.90:
            # well_display_ntd(df_bal, j, 'Balakhany X', 'NET', round((phit_nonzero/phit_zero),2), 1)
            well_zero10.append(j)
        else:
            well_clean10.append(j)
    print('well_tot8', len(well_tot8))
    print('well_zero8', len(well_zero8))
    print('well_clean8', len(well_clean8))
    print('----------------------')
    print('well_tot10', len(well_tot10))
    print('well_zero10', len(well_zero10))
    print('well_clean10', len(well_clean10))

    # broken wells Bal8
    # A08, A19, H01Z, J05 
    # broken wells Bal10
    # C31, D25
    # high tst_interv Bal8
    # E30Z
    # small tst_interv Bal8
    # G01Z, E05, E01, E01Y, E11Z, E07, H01Y, H01Z, A14
    # Add wells after review Bal8 
    # D04Z, 
    # Remove wells from clean_list by any reasons
    remove_tst8 = ['A08','A19','J05','E30Z','G01Z', 'E05', 'E01', 'E01Y', 'E11Z', 'E07', 'H01Y', 'H01Z', 'A14']
    well_clean8_v2 = [i for i in well_clean8 if i not in remove_tst8]
    remove_tst10 = ['C31','D25', 'E21A']
    well_clean10_v2 = [i for i in well_clean10 if i not in remove_tst10]
    print('----------------------')
    print('well_clean8_v2: ', len(well_clean8_v2))
    print('well_clean10_v2: ', len(well_clean10_v2))
    return well_clean8_v2, well_clean10_v2
well_clean8_v2, well_clean10_v2 = well_clean_v2()

### NetThicknessDistribution

In [ ]:
def NetThicknessDistribution():
    # Limitation dataframe to cleaned wells for Bal8 & Bal10
    df_net_bal8 = df_bal[['well', 'MD', 'TST', 'NET', 'FORMATION_up', 'LPERM', 'PHIT', 'VSH']]
    df_net_bal8 = df_net_bal8[df_net_bal8.well.isin(well_clean8_v2) & (df_net_bal8.FORMATION_up=='Balakhany VIII')]
    df_net_bal10 = df_bal[['well', 'MD', 'TST', 'NET', 'FORMATION_up', 'LPERM', 'PHIT', 'VSH']]
    df_net_bal10 = df_net_bal10[df_net_bal10.well.isin(well_clean10_v2) & (df_net_bal10.FORMATION_up=='Balakhany X')]
    # Calculation dataframe with h_tst from MD to TST for Bal8
    df_list8 = []
    print('Calculation dataframe with h_tst from MD to TST for Bal8')
    for well in tqdm(df_net_bal8.well.unique()):
        df = ntd_calculation_brief(df_net_bal8, well, 'Balakhany VIII', 'NET')
        df_list8.append(df)
    ntd_net_8 = pd.concat(df_list8)
    df_list10 = []
    print('Calculation dataframe with h_tst from MD to TST for Bal10')
    for well in tqdm(df_net_bal10.well.unique()):
        df = ntd_calculation_brief(df_net_bal10, well, 'Balakhany X', 'NET')
        df_list10.append(df)
    ntd_net_10 = pd.concat(df_list10)
    ntd_net_final = pd.concat([ntd_net_8, ntd_net_10])
    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    print('Cleaning NET variable and making up NET_clp with clipped data')
    net_clp =  ntd_htst_cleaning(ntd_net_final, 1)
    df_bal_net = df_bal.set_index(['well','MD']).join(net_clp.drop(
        ['FORMATION_up','NET','TST', 'FORMATION', 'GR_N'], axis=1).set_index(['well','MD'])).reset_index()
    df_bal_net = df_bal_net[df_bal_net.NET_clp.notna()]
    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(df_bal_net.NET_clp))):
        if (df_bal_net.NET_clp.iloc[i] == 0 and  
            df_bal_net.NET_clp.iloc[i-1] == 1 and 
            df_bal_net.NET_clp.iloc[i+1] == 1):
            df_bal_net.NET_clp.iloc[i] = 1
    # NET-zero layers removing
    df_zero_bal = df_bal_net[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    df_zero_bal8 = df_zero_bal[df_zero_bal.well.isin(well_clean8_v2) & (df_zero_bal.FORMATION_up=='Balakhany VIII')]
    df_zero_list8 = []
    print('NET-zero layers removing Bal8')
    for well in tqdm(df_zero_bal8.well.unique()):
        df = ntd_calculation_zero(df_zero_bal8, well, 'Balakhany VIII', 'NET_clp')
        df_zero_list8.append(df)
    ntd_zero_8 = pd.concat(df_zero_list8)
    df_zero_bal10 = df_zero_bal[df_zero_bal.well.isin(well_clean10_v2) & (df_zero_bal.FORMATION_up=='Balakhany X')]
    df_zero_list10 = []
    print('NET-zero layers removing Bal10')
    for well in tqdm(df_zero_bal10.well.unique()):
        df = ntd_calculation_zero(df_zero_bal10, well, 'Balakhany X', 'NET_clp')
        df_zero_list10.append(df)
    ntd_zero_10 = pd.concat(df_zero_list10)
    ntd_zero = pd.concat([ntd_zero_8, ntd_zero_10])
    print('Run ntd_htst_zero_cleaning')
    net_clp2 = ntd_htst_zero_cleaning(ntd_zero, df_bal_net, 1, 'NET_clp', 'NET_clp2')
    #Joining NET_clp2 to main dataframe df_bal_net
    df_bal_net2 = df_bal_net.set_index(['well','MD']).join(net_clp2.drop(
        ['FORMATION_up','GR_N', 'NET','NET_clp', 'FORMATION','TST'], axis=1).set_index(['well','MD'])).reset_index()
    df_bal_net2 = df_bal_net2[df_bal_net2.NET_clp2.notna()]
    # Displaying of ramdom well for example
    well_display_net(df_bal_net2, 'J28', 'Balakhany VIII', 'NET_clp', 1, 'NET_clp2')
    return df_bal_net2
df_bal_net2 = NetThicknessDistribution()

## Full set of FU for Bal VIII / X FORMATION

In [6]:
def prep_fullset_bal_fu_calc_prop(dataset):
    """
    Calculation XY coordinates of Bal VIII/X 
    Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2
    Calculation average properties for Bal8 and Bal10 based on NET_clp2
    Joing KHtst/tst_interv and avg prop into one dataframe
    """
    # Calculation XY coordinates of Bal VIII/X
    df_bal_net2_kh_xy8 = dataset[   (dataset.well.isin(well_clean8_v2)) & 
                                    (dataset.FORMATION_up=='Balakhany VIII')][['well','FORMATION_up','X_mean','Y_mean','TVD_SCS','field']].groupby(
                                    ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index() 
    df_bal_net2_kh_xy10 = dataset[  (dataset.well.isin(well_clean10_v2)) & 
                                    (dataset.FORMATION_up=='Balakhany X')][['well','FORMATION_up','X_mean','Y_mean','TVD_SCS','field']].groupby(
                                    ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
    df_bal_net2_kh_xy = pd.concat([df_bal_net2_kh_xy8, df_bal_net2_kh_xy10]).sort_values(by='well')  

    df_net2_bal8 = dataset[[ 'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal8 = df_net2_bal8[    (df_net2_bal8.well.isin(well_clean8_v2)) & 
                                    (df_net2_bal8.FORMATION_up=='Balakhany VIII')]
    df_net2_bal10 = dataset[['well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal10 = df_net2_bal10[  (df_net2_bal10.well.isin(well_clean10_v2)) & 
                                    (df_net2_bal10.FORMATION_up=='Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2')
    df_recalc_list8 = []
    for well in tqdm(df_net2_bal8.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8.append(df)
    ntd_net2_8 = pd.concat(df_recalc_list8)
    ntd_net2_8.drop_duplicates(inplace=True)
    df_recalc_list10 = []
    for well in tqdm(df_net2_bal10.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10.append(df)
    ntd_net2_10 = pd.concat(df_recalc_list10)
    ntd_net2_10.drop_duplicates(inplace=True)
    ntd_net2_final = pd.concat([ntd_net2_8, ntd_net2_10])
    # Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2 and available FU Bal VIII/X
    df_kh8 = df_net2_bal8.groupby('well')[[ 'TVD_SCS','KHtst','FORMATION_up',
                                            'PHITHtst','VSHHtst','X_mean','Y_mean','field']].apply(lambda x: x.iloc[1]).reset_index()                              
    df_kh10 = df_net2_bal10.groupby('well')[[   'TVD_SCS','KHtst','FORMATION_up',
                                                'PHITHtst','VSHHtst','X_mean','Y_mean','field']].apply(lambda x: x.iloc[1]).reset_index()
    df_tst8 = df_net2_bal8.groupby('well')[['TST', 'TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[1]).reset_index()
    df_tst8['FORMATION_up'] = 'Balakhany VIII'
    df_tst10 = df_net2_bal10.groupby('well')[['TST','TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[1]).reset_index()
    df_tst10['FORMATION_up'] = 'Balakhany X'
    df_kh_fin = pd.concat([df_kh8, df_kh10])
    df_tst_fin = pd.concat([df_tst8, df_tst10])
    df_tst_fin.rename(columns={'TST':'tst_interv', 'TVD_SCS':'tvd_interv'}, inplace=True)
    df_tst_kh = df_tst_fin.set_index(['well','FORMATION_up']).join(df_kh_fin.set_index(['well','FORMATION_up'])).reset_index()
    # Calculation average properties for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    print('Calculation average properties for Bal8 and Bal10 based on NET_clp2')
    avg_prop_bal8 = avg_prop_calculation(ntd_net2_8, df_net2_bal8, 'Balakhany VIII')
    avg_prop_bal10 = avg_prop_calculation(ntd_net2_10, df_net2_bal10, 'Balakhany X')   
    avg_prop_final = pd.concat([avg_prop_bal8, avg_prop_bal10])
    avg_prop_final.sort_values(by='well', inplace=True)  
    # Joing KHtst/tst_interv and avg prop into one dataframe
    avg_prop_tst_kh = avg_prop_final.set_index(['well','FORMATION_up']).join(df_tst_kh.set_index(['well','FORMATION_up'])).reset_index().sort_values(by='well')
    avg_prop_tst_kh = avg_prop_tst_kh.round({   'htst_max': 0, 'htst_sum': 0, 'tst_interv': 0, 'tvd_interv': 0, 'TVD_SCS': 0, 
                                                'KHtst': 0, 'PHITHtst': 0, 'VSHHtst': 0, 'perm_wavg':1, 'phit_wavg':2})
    return avg_prop_tst_kh
def prep_dist_calc_brief(dataset, offset):
    avg_prop_targKH = dataset[['well', 'FORMATION_up', 'KHtst']]
    df_dist_kh_bal8 = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'KHtst')
    df_dist_kh_bal10 = dist_prop_calc(dataset, 'Balakhany X', offset, 'KHtst')
    df_dist_kh_bal_concat = pd.concat([df_dist_kh_bal8, df_dist_kh_bal10])
    df_dist_kh_bal_fin = df_dist_kh_bal_concat.set_index(['well', 'FORMATION_up']).join(avg_prop_targKH.set_index(['well', 'FORMATION_up'])).reset_index()
    df_dist_kh_bal_fin['dist1_op'] = 1/df_dist_kh_bal_fin.dist1
    df_dist_kh_bal_fin['dist2_op'] = 1/df_dist_kh_bal_fin.dist2
    df_dist_kh_bal_fin['dist3_op'] = 1/df_dist_kh_bal_fin.dist3
    return df_dist_kh_bal_fin
def prep_remove_balfu_calc_prop(dataset):
    """
    Calculation XY coordinates of Bal VIII sand/X sand
    Removing Bal VIII and Bal X from FORMATION
    Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    Calculation average properties for Bal8 and Bal10 based on NET_clp2.
    Joing KHtst/tst_interv and avg prop into one dataframe
    """
    # Calculation XY coordinates of Bal VIII/X
    df_bal_net2_kh_cut_xy8 = dataset[   (dataset.well.isin(well_clean8_v2)) & 
                                        (dataset.FORMATION_up=='Balakhany VIII') &
                                        (dataset.FORMATION != 'Balakhany VIII')][['well','FORMATION_up','X_mean','Y_mean','field']].groupby(
                                        ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index() 
    df_bal_net2_kh_cut_xy10 = dataset[  (dataset.well.isin(well_clean10_v2)) & 
                                        (dataset.FORMATION_up=='Balakhany X') &
                                        (dataset.FORMATION != 'Balakhany X')][['well','FORMATION_up','X_mean','Y_mean','field']].groupby(
                                        ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
    df_bal_net2_kh_cut_xy = pd.concat([df_bal_net2_kh_cut_xy8, df_bal_net2_kh_cut_xy10]).sort_values(by='well')  
    # Removing Bal VIII and Bal X from FORMATION to calc avg prop
    df_net2_bal8_cut = dataset[[ 'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal8_cut = df_net2_bal8_cut[    df_net2_bal8_cut.well.isin(well_clean8_v2) & 
                                            (df_net2_bal8_cut.FORMATION_up=='Balakhany VIII') &
                                            (df_net2_bal8_cut.FORMATION != 'Balakhany VIII')]
    df_net2_bal10_cut = dataset[['well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal10_cut = df_net2_bal10_cut[      df_net2_bal10_cut.well.isin(well_clean10_v2) & 
                                                (df_net2_bal10_cut.FORMATION_up=='Balakhany X') &
                                                (df_net2_bal10_cut.FORMATION != 'Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2 and renoved FU Bal VIII/X')
    df_recalc_list8_cut = []
    for well in tqdm(df_net2_bal8_cut.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8_cut, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8_cut.append(df)
    ntd_net2_8_cut = pd.concat(df_recalc_list8_cut)
    ntd_net2_8_cut.drop_duplicates(inplace=True)
    df_recalc_list10_cut = []
    for well in tqdm(df_net2_bal10_cut.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10_cut, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10_cut.append(df)
    ntd_net2_10_cut = pd.concat(df_recalc_list10_cut)
    ntd_net2_10_cut.drop_duplicates(inplace=True)
    ntd_net2_cut_final = pd.concat([ntd_net2_8_cut, ntd_net2_10_cut])
    # Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    df_kh8_cut = df_net2_bal8_cut.groupby('well')[[ 'TVD_SCS','KHtst','FORMATION_up',
                                                    'PHITHtst','VSHHtst','X_mean','Y_mean','field']].apply(lambda x: x.iloc[0]).reset_index()                              
    df_kh10_cut = df_net2_bal10_cut.groupby('well')[[   'TVD_SCS','KHtst','FORMATION_up',
                                                        'PHITHtst','VSHHtst','X_mean','Y_mean','field']].apply(lambda x: x.iloc[0]).reset_index()
    df_tst8_cut = df_net2_bal8_cut.groupby('well')[['TST', 'TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
    df_tst8_cut['FORMATION_up'] = 'Balakhany VIII'
    df_tst10_cut = df_net2_bal10_cut.groupby('well')[['TST', 'TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
    df_tst10_cut['FORMATION_up'] = 'Balakhany X'
    df_kh_cut_fin = pd.concat([df_kh8_cut, df_kh10_cut])
    df_tst_cut_fin = pd.concat([df_tst8_cut, df_tst10_cut])
    df_tst_cut_fin.rename(columns={'TST':'tst_interv', 'TVD_SCS':'tvd_interv'}, inplace=True)
    df_tst_kh_cut = df_tst_cut_fin.set_index(['well','FORMATION_up']).join(df_kh_cut_fin.set_index(['well','FORMATION_up'])).reset_index()
    # Calculation average properties for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    print('Calculation average properties for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X')
    avg_prop_bal8_cut = avg_prop_calculation(ntd_net2_8_cut, df_net2_bal8_cut, 'Balakhany VIII')
    avg_prop_bal10_cut = avg_prop_calculation(ntd_net2_10_cut, df_net2_bal10_cut, 'Balakhany X')   
    avg_prop_final_cut = pd.concat([avg_prop_bal8_cut, avg_prop_bal10_cut])
    avg_prop_final_cut.sort_values(by='well', inplace=True)  
    # Joing KHtst/tst_interv and avg prop into one dataframe
    avg_prop_tst_kh_cut = avg_prop_final_cut.set_index(['well','FORMATION_up']).join(df_tst_kh_cut.set_index(['well','FORMATION_up'])).reset_index().sort_values(by='well')
    avg_prop_tst_kh_cut = avg_prop_tst_kh_cut.round({   'htst_max': 0, 'htst_sum': 0, 'tst_interv': 0, 'tvd_interv': 0, 'TVD_SCS': 0, 
                                                        'KHtst': 0, 'PHITHtst': 3, 'VSHHtst': 3, 'perm_wavg':5, 'phit_wavg':5})
    return avg_prop_tst_kh_cut                  
def prep_dist_calc(dataset, offset, blacklist):
    """
    Calculation properties based for offset wells: KHtst and phit_wavg
    Dataset - dataset with log data and any other variables 
    Offset - I prefer 0 or 300m, but it could be various 
    Blacklist - indices of rows which need to be removed
    """
    df_dist_kh_bal8_cut = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'KHtst')
    df_dist_kh_bal10_cut = dist_prop_calc(dataset, 'Balakhany X', offset, 'KHtst')
    df_dist_kh_bal_cut_fin = pd.concat([df_dist_kh_bal8_cut, df_dist_kh_bal10_cut])
    df_dist_ph_bal8_cut = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'phit_wavg')
    df_dist_ph_bal10_cut = dist_prop_calc(dataset, 'Balakhany X', offset, 'phit_wavg')
    df_dist_ph_bal_cut_fin = pd.concat([df_dist_ph_bal8_cut, df_dist_ph_bal10_cut])
    df_dist_htst_bal8_cut = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'htst_sum')
    df_dist_htst_bal10_cut = dist_prop_calc(dataset, 'Balakhany X', offset, 'htst_sum')
    df_dist_htst_bal10_cut_fin = pd.concat([df_dist_htst_bal8_cut, df_dist_htst_bal10_cut])
    htst_dist = df_dist_htst_bal10_cut_fin[['well','FORMATION_up', 'htst_sum_1', 'htst_sum_2', 'htst_sum_3']]
    
    phit_dist = df_dist_ph_bal_cut_fin[['well','FORMATION_up', 'phit_wavg_1', 'phit_wavg_2', 'phit_wavg_3']]
    phit_dist_htst = phit_dist.set_index(['well','FORMATION_up']).join(htst_dist.set_index(['well','FORMATION_up'])).reset_index()
    df_dist_ph_kh_cut = df_dist_kh_bal_cut_fin.set_index(['well','FORMATION_up']).join(phit_dist_htst.set_index(['well','FORMATION_up']))
    avg_prop_tst_kh_dist_cut = dataset.set_index(['well','FORMATION_up']).join(df_dist_ph_kh_cut).reset_index()

    avg_prop_tst_kh_dist_cut['dist1_op'] = 1/avg_prop_tst_kh_dist_cut.dist1
    avg_prop_tst_kh_dist_cut['dist2_op'] = 1/avg_prop_tst_kh_dist_cut.dist2
    avg_prop_tst_kh_dist_cut['dist3_op'] = 1/avg_prop_tst_kh_dist_cut.dist3

    index_to_drop = blacklist
    avg_prop_tst_kh_cut_test = avg_prop_tst_kh_dist_cut.drop(index_to_drop).reset_index().drop('index', axis=1)
    return avg_prop_tst_kh_cut_test

In [ ]:
def fullset_bal8_10():
    # KHtst calculation and join to the main dataframe df_bal_net2
    df_kh = proph_calculation(df_bal_net2, 'NET_clp2')
    df_bal_net2_kh_init = df_bal_net2.set_index(['well','MD']).join(df_kh.drop(['FORMATION_up', 'TST'], axis=1).set_index(['well','MD'])).reset_index()
    # Cleaning KHtst dataset according to mentioned well lists
    df_bal_net2_kh8 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean8_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany VIII')]
    df_bal_net2_kh10 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean10_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany X')]
    df_bal_net2_kh = pd.concat([df_bal_net2_kh8, df_bal_net2_kh10])
    # Save df_bal_net2_kh to csv-file
    save_tocsv(df_bal_net2_kh,'ACG_wells_JOINT_BEST_v10_output.csv', 0)
    # Preparation full set of Bal8/10 and save it to csv
    avg_prop_tst_kh = prep_fullset_bal_fu_calc_prop(df_bal_net2_kh)
    save_tocsv(avg_prop_tst_kh,'avg_prop_tst_kh.csv', 0)
    # Dist calculation for full set of Bal8/10
    df_dist_kh_bal_fin = prep_dist_calc_brief(avg_prop_tst_kh, 0)
    return avg_prop_tst_kh, df_dist_kh_bal_fin, df_bal_net2_kh
avg_prop_tst_kh, df_dist_kh_bal_fin, df_bal_net2_kh = fullset_bal8_10()

In [9]:
# Display 3 offsets wells
def display_3offset_wells(well, formation, dataset_dist=df_dist_kh_bal_fin, dataset_logs=df_bal_net2_kh):
    """
    Pay attention dataset_dist=df_dist_kh_bal_fin, dataset_logs=df_bal_net2_kh
    well:       just well name
    formation:  just formation
    """
    def well_offset_selection(dataset_dist, fmname, well_target):
        try:
            well_df = dataset_dist[(dataset_dist.well == well_target) & (dataset_dist.FORMATION_up == fmname)][['well', 'well1', 'well2', 'well3',
                                                                                                                        'dist1', 'dist2', 'dist3',
                                                                                                                'KHtst','KHtst_1', 'KHtst_2', 'KHtst_3']]
            well1 = well_df['well1'].iloc[0]
            well2 = well_df['well2'].iloc[0]
            well3 = well_df['well3'].iloc[0]
            dist1 = well_df['dist1'].astype('int').iloc[0]
            dist2 = well_df['dist2'].astype('int').iloc[0]
            dist3 = well_df['dist3'].astype('int').iloc[0]
            kh = well_df['KHtst'].astype('int').iloc[0]
            kh1 = well_df['KHtst_1'].astype('int').iloc[0]
            kh2 = well_df['KHtst_2'].astype('int').iloc[0]
            kh3 = well_df['KHtst_3'].astype('int').iloc[0]
        except Exception as e:
            print(f'It looks like the desired formation is absent. The error is "{e}"')
        return {'target': well_target, 'w1':well1, 'w2':well2, 'w3':well3, 
                'dist': 0,'d1':dist1, 'd2':dist2,'d3':dist3,
                'kh':kh,'kh1':kh1, 'kh2':kh2, 'kh3':kh3}
    def display_tracks(dataset, wellname, fmname, ref_depth, depth_step, r, c, kh_value, dist):
        try:
            data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
            depth = data[ref_depth]
            grn = data['GR_N']
            vsh = data['VSH']
            rhob = data['RHOB'] 
            npss = data['NPSS']
            rdeep = data['RDEEP']
            phit = data['PHIT'] 
            net = data['NET_clp2']
            perm = data['LPERM']
            kh = data['KHtst']
            well_bal_tops = df_bal[(df_bal.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
            ax[r,c].plot(grn, depth, color='lightgreen', lw=2, zorder=10)
            ax[r,c].set_xlim(0, 150) 
            ax[r,c].grid(axis='y')
            ax[r,c].invert_yaxis()
            ax[r,c].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c].set_xticks([])
            ax[r,c].tick_params(axis='y', labelsize=8)
            ax[r,c].set_title(wellname + ' ' + fmname + ' kh:' + str(kh_value) + ' dist:' + str(dist), fontsize=12) 
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c].hlines(    well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
                # ax[r,c].text(10, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
            ax[r,c+1].plot(rhob, depth, color='red')
            ax[r,c+1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
            ax[r,c+1].set_xlim(1.65, 2.65)
            ax[r,c+1].grid(axis='y')
            ax[r,c+1].grid(axis='x')
            ax[r,c+1].invert_yaxis()
            ax[r,c+1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c+1].set_xticks([])
            ax[r,c+1].set_yticks([])
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c+1].hlines( well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                                xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
                ax[r,c+1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
            twin1 = ax[r,c+1].twiny()
            twin1.plot(npss, depth, color='blue')
            twin1.set_xlim(0.6, 0)
            twin1.set_xticks([])
            ax[r,c+2].plot(phit, depth, color='green', linestyle='dashed')
            ax[r,c+2].set_xlim(0.3, 0)
            ax[r,c+2].grid(axis='x')
            ax[r,c+2].grid(axis='y')
            ax[r,c+2].invert_yaxis()
            ax[r,c+2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c+2].set_xticks([])
            ax[r,c+2].set_yticks([])
            ax[r,c+2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c+2].hlines(    well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
            twin2 = ax[r,c+2].twiny()
            twin2.plot(net, depth, color='orange', linewidth=0.5)
            twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
            twin2.set_xlim(0, 1)
            twin2.set_xticks([])
            ax[r,c+3].plot(perm, depth, color='purple', alpha=0.66)
            ax[r,c+3].set_xscale('log')
            ax[r,c+3].set_xlim(0.1, 1000)
            ax[r,c+3].grid(axis='y')
            ax[r,c+3].grid(axis='x')
            ax[r,c+3].invert_yaxis()
            ax[r,c+3].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c+3].set_xticks([])
            ax[r,c+3].set_yticks([])
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c+3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.5)
            twin4 = ax[r,c+3].twiny()
            twin4.plot(kh, depth, color='black', alpha=1)
            twin4.set_xticks([])
        except Exception as e:
            print(f'It looks like the desired formation is absent. The error is "{e}"')
        return fig.show()
    def display_subplots():
        try:
            well_dist_dict = well_offset_selection(dataset_dist, fmname, well_target)
            display_tracks(dataset_logs, well_dist_dict['target'], fmname,'TST', 10, 0,0,well_dist_dict['kh'], well_dist_dict['dist'])
            display_tracks(dataset_logs, well_dist_dict['w1'], fmname,'TST', 10 ,0,4, well_dist_dict['kh1'], well_dist_dict['d1'])  
            display_tracks(dataset_logs, well_dist_dict['w2'], fmname,'TST', 10,1,0, well_dist_dict['kh2'], well_dist_dict['d2'])      
            display_tracks(dataset_logs, well_dist_dict['w3'], fmname,'TST', 10,1,4, well_dist_dict['kh3'], well_dist_dict['d3'])
        except Exception as e:
            print(f'It looks like the desired formation is absent. The error is "{e}"')
    well_target = well
    fmname = formation
    fig, ax = plt.subplots(2,8, figsize=(9,8), constrained_layout=True)
    return display_subplots()

### Experiments wiht NTD

### Maps & 3D view

In [ ]:
# Drawing maps of well trajectories
def well_traj_dataprep(dataset):
    map_data = dataset.dropna()
    map_data_top = map_data.groupby(['well','FORMATION_up'])[['X_traj','Y_traj']].apply(lambda x: x.iloc[0:-100:100]).reset_index()
    map_data_bot = map_data.groupby(['well','FORMATION_up'])[['X_traj','Y_traj']].apply(lambda x: x.iloc[-1]).reset_index()
    map_data_middle = map_data.groupby(['well','FORMATION_up'])[['X_mean', 'Y_mean', 'KHtst', 'TVD_SCS', 'Status']].apply(lambda x: x.iloc[0]).reset_index()
    map_trajectory_display = pd.concat([map_data_top, map_data_bot]).sort_values(by=['well','FORMATION_up']).drop('level_2', axis=1)
    return map_trajectory_display, map_data_middle
map_trajectory_display, map_data_middle = well_traj_dataprep(df_bal_net2_kh)

bal8_1510 = pd.read_csv(r'C:\jupyter\SPP\input\surfaces\petrel\bal8_1510_base.csv', sep=' ', names=['X','Y','geobody'])
def display_well_traj(trajectory, map_data_middle, petrel, fmname, mult, path, comment, print_flag):
    trajectory = trajectory[trajectory.FORMATION_up == fmname]
    map_data_middle = map_data_middle[map_data_middle.FORMATION_up == fmname]
    map_data_middle['KHtst'] = map_data_middle['KHtst'].round(0)
    traj = go.Scatter(  x=trajectory.X_traj, y=trajectory.Y_traj, 
                        mode='markers',
                        marker=dict(color='black', size=1),
                        customdata = trajectory[['well']],
                        hovertemplate="".join(["well:%{customdata[0]}<extra></extra>"])
                        )
    wells = go.Scatter( x=map_data_middle.X_mean, y=map_data_middle.Y_mean, 
                        mode='markers',
                        # marker=dict(symbol='diamond', color='red', size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                        marker=dict(color=map_data_middle.KHtst, size=map_data_middle.KHtst*mult, colorscale='RdYlGn',  showscale=True,
                                    line=dict(color='rgb(47, 57, 61)', width=0.5)),
                        customdata = map_data_middle[['well', 'KHtst']],
                        hovertemplate="".join(["well:%{customdata[0]},kh:%{customdata[1]}<extra></extra>"]))
    geobody_map = go.Scatter(   x=petrel['X'], y=petrel['Y'],
                                mode='markers',
                                marker=dict(size=5, color=petrel['geobody'],colorscale='Viridis', opacity=0.5))
    fig = go.Figure()
    fig.add_trace(geobody_map)
    fig.add_trace(traj)
    fig.add_trace(wells)
    fig.update_layout(  title_text= ('Map of traj and well mean points with'+ ' ' + fmname + ' 1510 polygons. Size of bubbles is KHtst.'),
                        autosize=True, width=1000, height=700, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    if print_flag == 'print':
        go_offline.plot(fig, filename=path + comment, validate=True, auto_open=False)
    else:
        pass
    return fig.show()
display_well_traj(map_trajectory_display, map_data_middle, bal8_1510, 'Balakhany VIII', 0.00125, 'plots/', 'Balakhany8_KHtst', 'dont_print')
# display_well_traj(map_trajectory_display, map_data_middle, 'Balakhany X', 0.003, 'plots/', 'Balakhany10_KHtst', 'dont_print')

## Removing Bal VIII / X from FORMATION

In [ ]:
def removing_fu_bal8_10():
    # KHtst calculation and join to the main dataframe df_bal_net2
    df_kh = proph_calculation(df_bal_net2, 'NET_clp2')
    df_bal_net2_kh_init = df_bal_net2.set_index(['well','MD']).join(df_kh.drop(['FORMATION_up', 'TST'], axis=1).set_index(['well','MD'])).reset_index()
    # Cleaning KHtst dataset according to mentioned well lists
    df_bal_net2_kh8 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean8_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany VIII')]
    df_bal_net2_kh10 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean10_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany X')]
    df_bal_net2_kh = pd.concat([df_bal_net2_kh8, df_bal_net2_kh10])
    # Removing upper part of Bal8/10 and property calc
    avg_prop_tst_kh_cut = prep_remove_balfu_calc_prop(df_bal_net2_kh)
    # Calculation of well properties and applying blacklist with indecies of outliers
    blacklist = [244, 240, 320, 324]
    avg_prop_tst_kh_cut0 = prep_dist_calc(avg_prop_tst_kh_cut, 0, blacklist)
    avg_prop_tst_kh_cut0 = avg_prop_tst_kh_cut0.convert_dtypes()
    well_offsets_df = avg_prop_tst_kh_cut0.copy()
    #Saving well propetries dataset to csv
    save_tocsv(avg_prop_tst_kh_cut0, 'avg_prop_tst_kh_dist0_cut.csv', 0)
    # Displaying features list of dataframe
    avg_prop_tst_kh_cut0.columns
    return avg_prop_tst_kh_cut0
avg_prop_tst_kh_cut0 = removing_fu_bal8_10()

In [ ]:
avg_prop_tst_kh_cut0.columns

### Data interpolation

In [ ]:
def interpolate_by_depth_fm(dataset_logs, formation_name, step):
    def interpolate_by_depth(one_well, formation_name, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        interp_X = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
        interp_Y = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
        interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
        interp_TVD = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
        interp_NET_clp2 = interp1d(one_well['TST'], one_well['NET_clp2'], kind='linear', fill_value="extrapolate")
        interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
        interp_KHtst = interp1d(one_well['TST'], one_well['KHtst'], kind='linear', fill_value="extrapolate")
        # Create a new DataFrame with the interpolated values for new TVD_SCS
        new_data = {
            'well': [well_name for _ in range(len(new_TST_values))],
            'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
            'tst_index': [_ for _ in range(len(new_TST_values))],
            'TST': new_TST_values,
            'X_traj': interp_X(new_TST_values),
            'Y_traj': interp_Y(new_TST_values),
            'PHIT': interp_PHIT(new_TST_values),
            'TVD_SCS': interp_TVD(new_TST_values),
            'NET_clp2': interp_NET_clp2(new_TST_values),
            'LPERM': interp_LPERM(new_TST_values),
            'KHtst': interp_KHtst(new_TST_values),
        }
        new_df = pd.DataFrame(new_data)
        return new_df
    df_lst = []
    print(f'Start interpolation of {formation_name}')
    for wellnames in tqdm(dataset_logs.well.unique()):
        well_sel = dataset_logs[dataset_logs.well == wellnames]
        well_interp = interpolate_by_depth(well_sel, formation_name, step)
        df_lst.append(well_interp)
    result = pd.concat(df_lst)
    return result
well_bal8 = df_bal_net2_kh[(df_bal_net2_kh.FORMATION_up == 'Balakhany VIII')]
well_bal10 = df_bal_net2_kh[(df_bal_net2_kh.FORMATION_up == 'Balakhany X')]
well_bal8_interp = interpolate_by_depth_fm(well_bal8, 'Balakhany VIII', 0.1)
well_bal10_interp = interpolate_by_depth_fm(well_bal10, 'Balakhany X', 0.1)

### PHIT_clipped

In [ ]:
well_bal8_interp_rn = well_bal8_interp.rename(columns={'PHIT':'PHIT_orig'})
well_bal10_interp_rn = well_bal10_interp.rename(columns={'PHIT':'PHIT_orig'})

def phit_rolling_averaging(input_dataset, samples_per_window):
    df_lst = []
    avg_report = []
    fmname = input_dataset['FORMATION_up'].iloc[0] 
    print(f'Start rolling averaging of {fmname}')
    for wellname in tqdm(input_dataset.well.unique()):
        dataset = input_dataset[input_dataset.well == wellname]
        window_size = int(len(dataset) / samples_per_window)
        dataset['PHIT'] = dataset['PHIT_orig'].rolling(window=window_size, center=True).mean()
        dataset =  dataset.dropna(subset=['PHIT'])
        df_lst.append(dataset)
        avg_report.append((wellname, len(dataset), window_size, samples_per_window))
    result = pd.concat(df_lst)
    avg_report_df = pd.DataFrame(avg_report, columns=['well','lenght_ds','window_size','samples_per_window'])
    return result, avg_report_df
samples_per_window = 100
well_bal8_interp_phavg, avg_report_df8 = phit_rolling_averaging(well_bal8_interp_rn, samples_per_window)
well_bal10_interp_phavg, avg_report_df10 = phit_rolling_averaging(well_bal10_interp_rn, samples_per_window)
well_bal8_interp_phavg['PHIT_clp'] = well_bal8_interp_phavg['PHIT']
well_bal10_interp_phavg['PHIT_clp'] = well_bal10_interp_phavg['PHIT']
well_bal8_interp_phavg['LPERM_clp'] = well_bal8_interp_phavg['LPERM']
well_bal10_interp_phavg['LPERM_clp'] = well_bal10_interp_phavg['LPERM']
well_bal8_interp_phavg.loc[well_bal8_interp_phavg.NET_clp2 == 0, 'PHIT_clp'] = 0.12
well_bal10_interp_phavg.loc[well_bal10_interp_phavg.NET_clp2 == 0, 'PHIT_clp'] = 0.12
well_bal8_interp_phavg.loc[well_bal8_interp_phavg.NET_clp2 == 0, 'LPERM_clp'] = 0.1
well_bal10_interp_phavg.loc[well_bal10_interp_phavg.NET_clp2 == 0, 'LPERM_clp'] = 0.1

In [ ]:
def cutting_block_lenght(dataset, block_lenght):
    df_lst = []
    fmname = dataset['FORMATION_up'].iloc[0]
    print(f'Start processing of dataset for {fmname} with block lenght {block_lenght}')
    for wellname in tqdm(dataset.well.unique()):
        data = dataset[dataset.well == wellname]
        tst_index_repaired = [i for i in range(0, len(data))]
        data['tst_index'] = tst_index_repaired
        new_index = [i for i in range(0, len(data), block_lenght)]
        data_cut = data[(data.tst_index < new_index[-1])]
        df_lst.append(data_cut)
    result = pd.concat(df_lst)
    return result
block_lenght = 100
well_bal8_interp_phavg_cut = cutting_block_lenght(well_bal8_interp_phavg, block_lenght)
well_bal10_interp_phavg_cut = cutting_block_lenght(well_bal10_interp_phavg, block_lenght)

In [ ]:
def exercise_phit_avg_kh(dataset, wellname):
    well_a01w = dataset[(dataset.well==wellname) & (dataset.FORMATION_up=='Balakhany VIII')]

    well_a01w['PHIT_clipped'] = well_a01w['PHIT']
    well_a01w.loc[well_a01w.NET_clp2 == 0, 'PHIT_clipped'] = 0
    well_a01w['LPERM_avg'] = 0.00000002*(np.exp(well_a01w.PHIT*105.56))
    well_a01w.loc[well_a01w['PHIT'] >= 0.2, 'LPERM_avg'] = (7.7925*((well_a01w.PHIT*100)**2))-(29881.0*well_a01w.PHIT)+2891.8
    well_a01w.loc[well_a01w['PHIT'] < 0.16, 'LPERM_avg'] = 0.0159*(np.exp(well_a01w.PHIT*21.27))
    well_a01w['khtst'] = well_a01w.LPERM_avg*0.1
    well_a01w['KHtst_avg'] = well_a01w.loc[::-1, 'khtst'].cumsum()[::-1]

    y = well_a01w.TST
    phit_orig = well_a01w.PHIT_orig
    phit_avg = well_a01w.PHIT
    phit_cliped = well_a01w.PHIT_clipped
    net = well_a01w.NET_clp2
    perm = well_a01w.LPERM
    perm_avg = well_a01w.LPERM_avg
    kh = well_a01w.KHtst
    kh_avg = well_a01w.KHtst_avg
    print(  'KH orig:', kh.iloc[0].round(0), 
            'KH avg:',kh_avg.iloc[0].round(0), 
            'KHavg/KHorig:',((kh.iloc[0].round(0)-kh_avg.iloc[0].round(0))/kh.iloc[0].round(0)).round(2))

    fig, ax = plt.subplots(1, 4, figsize=(6, 7))
    ax[0].plot(phit_orig, y, color='green')
    ax[0].plot(phit_avg, y, color='red')
    ax[0].set_xlim(0, 0.3)
    ax[0].invert_yaxis()
    ax[0].set_title(wellname)
    ax[1].plot(phit_cliped, y, color='red', zorder=1)
    ax[1].plot(net, y, color='orange', zorder=0)
    ax[1].set_xlim(0, 0.3)
    ax[1].invert_yaxis()
    ax[2].plot(perm, y, color='purple', lw=3)
    ax[2].plot(perm_avg, y, color='yellow')
    ax[2].invert_yaxis()
    ax[2].set_xscale('log')
    ax[3].plot(kh, y, color='black')
    ax[3].plot(kh_avg, y, color='gray')
    ax[3].invert_yaxis()
    fig.show()
exercise_phit_avg_kh(well_bal8_interp_phavg_cut, 'A01W')

###  PHIT & GRcube - martix plot 

In [ ]:
def well_letter_def(dataset):
    wells_letter = [wellname[0] for wellname in dataset.well.unique()]
    return set(wells_letter)
well_letter_def(well_bal8_interp_phavg_cut)

In [17]:
def gr_cube_upload():
    path = 'C:\\jupyter\\SPP\\input\\'
    vsh_cube_log = pd.read_parquet(path + 'ACG_GRcube_VSH_v3.parquet.gzip')
    vsh_cube_log = vsh_cube_log.replace(-9999.000, np.nan)
    vsh_cube_log = vsh_cube_log.dropna()
    vsh_cube_log.loc[vsh_cube_log.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
    vsh_cube_log.loc[vsh_cube_log.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
    vsh_cube_log = vsh_cube_log[vsh_cube_log.FORMATION_up.isin(['Balakhany VIII', 'Balakhany X'])]
    vsh_grcube = vsh_cube_log[['wellName', 'DEPT','VSH_GRcube', 'FORMATION_up']]
    vsh_grcube = vsh_grcube.rename(columns={'wellName':'well', 'DEPT':'MD'})
    return vsh_grcube
vsh_grcube = gr_cube_upload()
df_bal_net2_kh['MD'] = df_bal_net2_kh.MD.round(1)
df_bal_net2_kh_cube = df_bal_net2_kh.set_index(['well','MD', 'FORMATION_up']).join(vsh_grcube.set_index(['well','MD', 'FORMATION_up'])).reset_index()

In [ ]:
def vsh_gr_cube_recalc(dataset):
    def interpolate_by_depth_fm(dataset_logs, formation_name, step):
        def interpolate_by_depth(one_well, formation_name, step):
            one_well = one_well.sort_values(by='TST')
            well_name = one_well["well"].iloc[0]
            data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
            starting_tst = one_well["TST"].iloc[0]
            new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
            interp_X = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
            interp_Y = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
            interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
            interp_TVD = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
            interp_NET_clp2 = interp1d(one_well['TST'], one_well['NET_clp2'], kind='linear', fill_value="extrapolate")
            interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
            interp_KHtst = interp1d(one_well['TST'], one_well['KHtst'], kind='linear', fill_value="extrapolate")
            interp_VSH_GRcube = interp1d(one_well['TST'], one_well['VSH_GRcube'], kind='linear', fill_value="extrapolate")
            # Create a new DataFrame with the interpolated values for new TVD_SCS
            new_data = {
                'well': [well_name for _ in range(len(new_TST_values))],
                'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
                'tst_index': [_ for _ in range(len(new_TST_values))],
                'TST': new_TST_values,
                'X_traj': interp_X(new_TST_values),
                'Y_traj': interp_Y(new_TST_values),
                'PHIT': interp_PHIT(new_TST_values),
                'TVD_SCS': interp_TVD(new_TST_values),
                'NET_clp2': interp_NET_clp2(new_TST_values),
                'LPERM': interp_LPERM(new_TST_values),
                'KHtst': interp_KHtst(new_TST_values),
                'VSH_GRcube':interp_VSH_GRcube(new_TST_values)
            }
            new_df = pd.DataFrame(new_data)
            return new_df
        df_lst = []
        print(f'Start interpolation of {formation_name}')
        for wellnames in tqdm(dataset_logs.well.unique()):
            well_sel = dataset_logs[dataset_logs.well == wellnames]
            well_interp = interpolate_by_depth(well_sel, formation_name, step)
            df_lst.append(well_interp)
        result = pd.concat(df_lst)
        return result
    well_bal8 = dataset[(dataset.FORMATION_up == 'Balakhany VIII')]
    well_bal10 = dataset[(dataset.FORMATION_up == 'Balakhany X')]
    well_bal8_interp = interpolate_by_depth_fm(well_bal8, 'Balakhany VIII', 0.1)
    well_bal10_interp = interpolate_by_depth_fm(well_bal10, 'Balakhany X', 0.1)
    well_bal8_interp_rn = well_bal8_interp.rename(columns={'PHIT':'PHIT_orig'})
    well_bal10_interp_rn = well_bal10_interp.rename(columns={'PHIT':'PHIT_orig'})

    def phit_rolling_averaging(input_dataset, samples_per_window):
        df_lst = []
        avg_report = []
        fmname = input_dataset['FORMATION_up'].iloc[0] 
        print(f'Start rolling averaging of {fmname}')
        for wellname in tqdm(input_dataset.well.unique()):
            dataset = input_dataset[input_dataset.well == wellname]
            window_size = int(len(dataset) / samples_per_window)
            dataset['PHIT'] = dataset['PHIT_orig'].rolling(window=window_size, center=True).mean()
            dataset =  dataset.dropna(subset=['PHIT'])
            df_lst.append(dataset)
            avg_report.append((wellname, len(dataset), window_size, samples_per_window))
        result = pd.concat(df_lst)
        avg_report_df = pd.DataFrame(avg_report, columns=['well','lenght_ds','window_size','samples_per_window'])
        return result, avg_report_df
    samples_per_window = 100
    well_bal8_interp_phavg, avg_report_df8 = phit_rolling_averaging(well_bal8_interp_rn, samples_per_window)
    well_bal10_interp_phavg, avg_report_df10 = phit_rolling_averaging(well_bal10_interp_rn, samples_per_window)
    well_bal8_interp_phavg['PHIT_clp'] = well_bal8_interp_phavg['PHIT']
    well_bal10_interp_phavg['PHIT_clp'] = well_bal10_interp_phavg['PHIT']
    well_bal8_interp_phavg['LPERM_clp'] = well_bal8_interp_phavg['LPERM']
    well_bal10_interp_phavg['LPERM_clp'] = well_bal10_interp_phavg['LPERM']
    well_bal8_interp_phavg.loc[well_bal8_interp_phavg.NET_clp2 == 0, 'PHIT_clp'] = 0.12
    well_bal10_interp_phavg.loc[well_bal10_interp_phavg.NET_clp2 == 0, 'PHIT_clp'] = 0.12
    well_bal8_interp_phavg.loc[well_bal8_interp_phavg.NET_clp2 == 0, 'LPERM_clp'] = 0.1
    well_bal10_interp_phavg.loc[well_bal10_interp_phavg.NET_clp2 == 0, 'LPERM_clp'] = 0.1
    return well_bal8_interp_phavg, well_bal10_interp_phavg
well_bal8_interp_phavg, well_bal10_interp_phavg = vsh_gr_cube_recalc(df_bal_net2_kh_cube)

In [ ]:
def well_plots_phit_vsh_matrix(dataset, platform, variable, flag, max_var, comment):
    """
    flag = 'phit' or 'perm'
    """
    rows = 4
    columns = 9
    wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(platform)]
    fig, ax = plt.subplots(rows,columns, figsize=(16,rows*3))
    counter = 0
    y_real_list = []
    for j in range(0, rows):
        for i in range(0, columns):
            if counter < len(wells_letter):
                data = dataset[dataset.well==wells_letter[counter]]
                y_real_list.append(len(data))
                counter +=1
    max_ind = max(y_real_list)
    counter = 0
    for j in range(0, rows):
        for i in range(0, columns):
            if counter < len(wells_letter):
                well_data = dataset[dataset.well==wells_letter[counter]]
                ind = well_data[variable]
                y_real = [k for k in range(len(ind))]
                y_desired = [k for k in range(max_ind)]
                y_diff = len(y_desired) - len(y_real)
                values_to_add = [0.12 for k in range(y_diff)]
                x = well_data[variable]
                x_gr = well_data['VSH_GRcube']
                x_new = pd.concat([x, pd.Series(values_to_add)])
                x_gr_new = pd.concat([x_gr, pd.Series(values_to_add)])          
                if flag == 'phit':
                    ax[j,i].plot(x_new, y_desired, color='green', lw=1.5, alpha=1, zorder=1)
                    ax[j,i].set_xlim(0.1, 0.35)
                    # twin = ax[j,i].twiny()
                    # twin.plot(x_gr_new, y_desired, color='green', lw=2, alpha=0.5, zorder=0)
                    # twin.set_xlim(0, 1)
                if flag == 'perm':
                    ax[j,i].plot(x_new, y_desired, color='purple', lw=2, alpha=0.75)
                    ax[j,i].set_xscale('log')
                    ax[j,i].set_xlim(0.1, max_var)
                ax[j,i].set_title(wells_letter[counter] + comment)
                ax[j,i].invert_yaxis()
                ax[j,i].grid()
                counter +=1

    return plt.tight_layout()
# for letter in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']:
for letter in ['A']:
    well_plots_phit_vsh_matrix(well_bal8_interp_phavg, letter, 'PHIT_clp', 'phit', 0.35, ' bal8')

## Clustering job

### Data preparation for clustering

In [ ]:
def clustering_data_calculation(dataset):
    """
    """
    df_net2_bal8 = dataset[[    'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal8 = df_net2_bal8[    (df_net2_bal8.well.isin(well_clean8_v2)) & 
                                    (df_net2_bal8.FORMATION_up=='Balakhany VIII')]
    df_net2_bal10 = dataset[[   'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal10 = df_net2_bal10[  (df_net2_bal10.well.isin(well_clean10_v2)) & 
                                    (df_net2_bal10.FORMATION_up=='Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2')
    def ntd_calculation_brief(dataset,well,desired_fm, net_var):
        data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list8 = []
    for well in tqdm(df_net2_bal8.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8.append(df)
    ntd_net2_8 = pd.concat(df_recalc_list8)
    ntd_net2_8.drop_duplicates(inplace=True)
    df_recalc_list10 = []
    for well in tqdm(df_net2_bal10.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10.append(df)
    ntd_net2_10 = pd.concat(df_recalc_list10)
    ntd_net2_10.drop_duplicates(inplace=True)

    print('Calculation values for NTD Bal8 and Bal10')
    def ntd_properties_dataframe(dataset_ntd, dataset_logs, fmname):
        well_data = []
        well_formation = fmname
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()[:]):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_gperm = []
            well_top = []
            well_bot = []
            well_h = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_gperm.append(gmean(perm_lst))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val_bal8 = ntd_properties_dataframe(ntd_net2_8, df_net2_bal8, 'Balakhany VIII')
    ntd_val_bal10 = ntd_properties_dataframe(ntd_net2_10, df_net2_bal10, 'Balakhany X')
    ntd_val_final = pd.concat([ntd_val_bal8, ntd_val_bal10])
    return ntd_val_final
ntd_val_final = clustering_data_calculation(df_bal_net2_kh)
ntd_val_final8 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany VIII']
ntd_val_final10 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany X']

def nothtst_nan_fill(dataset_ntd, fmname):
    def nan_change_diff_fmbottom(dataset, wellname, fmname):
        row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
        row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
        return row_change
    df_list = []
    for wellname in dataset_ntd.well.unique():
        df = nan_change_diff_fmbottom(dataset_ntd, wellname, fmname)
        df_list.append(df)
    res_df_list = pd.concat(df_list)
    result = pd.concat([dataset_ntd, res_df_list])
    result = result.sort_values(by=['well','top_tst'])
    result_final = result.dropna(subset=['not_htst'], axis=0)
    return result_final
ntd_val_final8_clean = nothtst_nan_fill(ntd_val_final8, 'Balakhany VIII')
ntd_val_final10_clean = nothtst_nan_fill(ntd_val_final10, 'Balakhany X')

In [ ]:
#Clustering with block3
def blocks_for_clustering(df, block_size):
    def rows_selection(dataframe, block_size):
        selected_rows = dataframe.iloc[:block_size]   
        phit = []
        htst = []
        not_htst = []
        for ind, rows in selected_rows.iterrows():
            phit.append(rows['phit_avg'])
            htst.append(rows['htst'])
            not_htst.append(rows['not_htst'])
        def add_zero_values(var, block_size):
            if len(var) < block_size:
                zero_add = [np.nan for i in range(0, (block_size-len(var)))]
                res_var = np.concatenate([var,np.array(zero_add)], axis=0)
            else:
                res_var = var
            return res_var
        phit = add_zero_values(phit, block_size)
        htst = add_zero_values(htst, block_size)
        not_htst = add_zero_values(not_htst, block_size)
        res = np.concatenate([phit, htst, not_htst], axis=0) 
        res = res.reshape(1, block_size*3)
        return res
    def columns_names(name):
        columns_names = []
        for i in range(block_size):
            columns_names.append(name + '_' + str(i+1))
        return columns_names
    wellname = df['well'][0]
    fmname = df['FORMATION_up'][0]

    blocks = [df.iloc[i:i + block_size] for i in range(0, len(df), block_size)]
    phit_names = columns_names('phit')
    htst_names = columns_names('htst')
    not_htst_names = columns_names('not_htst')
    columns_names_list = phit_names + htst_names + not_htst_names

    res_lists = []
    for i in range(len(blocks)):
        res = rows_selection(blocks[i], block_size)
        res_lists.append(res)
    res = np.concatenate(res_lists)
    df_res = pd.DataFrame(res, columns = columns_names_list)
    df_res['well'] = wellname
    df_res['FORMATION_up'] = fmname
    return df_res
ntd_val_final8 = ntd_val_final8_clean
df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    df8 = ntd_val_final8[(ntd_val_final8.well==wellname)]
    res_df8 = blocks_for_clustering(df8, 3)
    df_lst.append(res_df8)
ntd_val_cluster = pd.concat(df_lst)
# ntd_val_cluster.to_csv('.\output\\ntd_val_cluster2.csv', index=False)
ntd_val_cluster = ntd_val_cluster.reset_index().drop('index', axis=1)
ntd_val_cluster.head(5)

In [ ]:
#Clustering with stride
def stride_for_clustering(df, block_size):
    def rows_selection(dataframe, block_size):
        selected_rows = dataframe.iloc[:block_size]   
        phit = []
        htst = []
        not_htst = []
        for ind, rows in selected_rows.iterrows():
            phit.append(rows['phit_avg'])
            htst.append(rows['htst'])
            not_htst.append(rows['not_htst'])
        def add_zero_values(var, block_size):
            if len(var) < block_size:
                zero_add = [np.nan for i in range(0, (block_size-len(var)))]
                res_var = np.concatenate([var,np.array(zero_add)], axis=0)
            else:
                res_var = var
            return res_var
        phit = add_zero_values(phit, block_size)
        htst = add_zero_values(htst, block_size)
        not_htst = add_zero_values(not_htst, block_size)
        res = np.concatenate([phit, htst, not_htst], axis=0) 
        res = res.reshape(1, block_size*3)
        return res
    def columns_names(name):
        columns_names = []
        for i in range(block_size):
            columns_names.append(name + '_' + str(i+1))
        return columns_names
    wellname = df['well'][0]
    fmname = df['FORMATION_up'][0]

    blocks = [df.iloc[i:i + block_size] for i in range(0, len(df), 1)]
    phit_names = columns_names('phit')
    htst_names = columns_names('htst')
    not_htst_names = columns_names('not_htst')
    columns_names_list = phit_names + htst_names + not_htst_names

    res_lists = []
    for i in range(len(blocks)):
        res = rows_selection(blocks[i], block_size)
        res_lists.append(res)
    res = np.concatenate(res_lists)
    df_res = pd.DataFrame(res, columns = columns_names_list)
    df_res['well'] = wellname
    df_res['FORMATION_up'] = fmname
    return df_res
ntd_val_final8 = ntd_val_final8_clean
df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    df8 = ntd_val_final8[(ntd_val_final8.well==wellname)]
    res_df8 = stride_for_clustering(df8, 3)
    df_lst.append(res_df8)
ntd_val_cluster_stride = pd.concat(df_lst)
# ntd_val_cluster_stride.to_csv('.\output\\ntd_val_cluster_stride.csv', index=False)
ntd_val_cluster_stride = ntd_val_cluster_stride.reset_index().drop('index', axis=1)
ntd_val_cluster_stride.head(7)

In [ ]:
#Clustering with whole well
def stride_for_clustering_wholewell(df, block_size):
    block_size = max_blocks_determination(ntd_val_final8)
    def rows_selection(dataframe, block_size):
        selected_rows = dataframe.iloc[:block_size]   
        phit = []
        htst = []
        not_htst = []
        for ind, rows in selected_rows.iterrows():
            phit.append(rows['phit_avg'])
            htst.append(rows['htst'])
            not_htst.append(rows['not_htst'])
        def add_zero_values(var, block_size):
            if len(var) < block_size:
                zero_add = [np.nan for i in range(0, (block_size-len(var)))]
                res_var = np.concatenate([var,np.array(zero_add)], axis=0)
            else:
                res_var = var
            return res_var
        phit = add_zero_values(phit, block_size)
        htst = add_zero_values(htst, block_size)
        not_htst = add_zero_values(not_htst, block_size)
        res = np.concatenate([phit, htst, not_htst], axis=0) 
        res = res.reshape(1, block_size*3)
        return res
    def columns_names(name):
        columns_names = []
        for i in range(block_size):
            columns_names.append(name + '_' + str(i+1))
        return columns_names
    wellname = df['well'][0]
    fmname = df['FORMATION_up'][0]

    blocks = [df.iloc[i:i + block_size] for i in range(0, len(df), 1)]
    phit_names = columns_names('phit')
    htst_names = columns_names('htst')
    not_htst_names = columns_names('not_htst')
    columns_names_list = phit_names + htst_names + not_htst_names

    res_lists = []
    for i in range(len(blocks)):
        res = rows_selection(blocks[i], block_size)
        res_lists.append(res)
    res = np.concatenate(res_lists)
    df_res = pd.DataFrame(res, columns = columns_names_list)
    df_res['well'] = wellname
    df_res['FORMATION_up'] = fmname
    return df_res
def max_blocks_determination(dataset):
    data_lst = []
    for wellname in  dataset.well.unique():
        layers_total = dataset[dataset.well == wellname].shape[0]
        data_lst.append(layers_total)
    return max(data_lst)
def clustering_wholewell_reduce(dataset):
    df_lst = []
    for wellname in dataset.well.unique():
        data = dataset[dataset.well == wellname].iloc[:1]
        df_lst.append(data)
    result = pd.concat(df_lst).reset_index()
    return result

ntd_val_final8 = ntd_val_final8_clean
block_size = max_blocks_determination(ntd_val_final8)
df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    df8 = ntd_val_final8[(ntd_val_final8.well==wellname)]
    res_df8 = stride_for_clustering_wholewell(df8, block_size)
    df_lst.append(res_df8)
ntd_stride_wholewell = pd.concat(df_lst).fillna(0)
ntd_stride_wholewell_cut = clustering_wholewell_reduce(ntd_stride_wholewell)
# ntd_stride_wholewell_cut.to_csv('.\output\\ntd_stride_wholewell.csv', index=False)
ntd_stride_wholewell_cut.head(3)

In [ ]:
#Clustering with stride
def stride_for_clustering(df, block_size):
    def rows_selection(dataframe, block_size):
        selected_rows = dataframe.iloc[:block_size]   
        phit = []
        htst = []
        not_htst = []
        for ind, rows in selected_rows.iterrows():
            phit.append(rows['phit_avg'])
            htst.append(rows['htst'])
            not_htst.append(rows['not_htst'])
        def add_zero_values(var, block_size):
            if len(var) < block_size:
                zero_add = [np.nan for i in range(0, (block_size-len(var)))]
                res_var = np.concatenate([var,np.array(zero_add)], axis=0)
            else:
                res_var = var
            return res_var
        phit = add_zero_values(phit, block_size)
        htst = add_zero_values(htst, block_size)
        not_htst = add_zero_values(not_htst, block_size)
        res = np.concatenate([phit, htst, not_htst], axis=0) 
        res = res.reshape(1, block_size*3)
        return res
    def columns_names(name):
        columns_names = []
        for i in range(block_size):
            columns_names.append(name + '_' + str(i+1))
        return columns_names
    wellname = df['well'][0]
    fmname = df['FORMATION_up'][0]

    blocks = [df.iloc[i:i + block_size] for i in range(0, len(df), 1)]
    phit_names = columns_names('phit')
    htst_names = columns_names('htst')
    not_htst_names = columns_names('not_htst')
    columns_names_list = phit_names + htst_names + not_htst_names

    res_lists = []
    for i in range(len(blocks)):
        res = rows_selection(blocks[i], block_size)
        res_lists.append(res)
    res = np.concatenate(res_lists)
    df_res = pd.DataFrame(res, columns = columns_names_list)
    df_res['well'] = wellname
    df_res['FORMATION_up'] = fmname
    return df_res

ntd_val_final8 = ntd_val_final8_clean
df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    df8 = ntd_val_final8[(ntd_val_final8.well==wellname)]
    res_df8 = stride_for_clustering(df8, 1)
    df_lst.append(res_df8)
ntd_val_cluster_stride = pd.concat(df_lst)
# ntd_val_cluster_stride.to_csv('.\output\\ntd_val_cluster_stride.csv', index=False)
ntd_val_cluster_stride = ntd_val_cluster_stride.reset_index().drop('index', axis=1)
ntd_val_cluster_stride.head(7)

In [ ]:
#Clustering with top-phi-bot
def top_phit_bot_clustering(dataset, wellname):
    data = dataset[dataset.well == wellname]
    data['top_htst'] = data['top_tst'] - data['fm_top_tst']
    data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
    data['bot_htst'] = data['not_htst']
    data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'top_htst','htst','bot_htst', 'fm_top_tst', 'fm_bot_tst']]
    return data

df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    res_df8 = top_phit_bot_clustering(ntd_val_final8_clean, wellname)
    df_lst.append(res_df8)
top_phi_bot_cluster = pd.concat(df_lst).reset_index().drop('index', axis=1)
# top_phi_bot_cluster.to_csv('.\output\\top_phi_bot_cluster.csv', index=False)
top_phi_bot_cluster.head(3)

In [ ]:
# Clustering with top-phi-bot - tough algorithm
def top_phit_bot_clustering(df, block_size):
    def rows_selection(dataframe, block_size):
        selected_rows = dataframe.iloc[:block_size]   
        phit = []
        top_htst = []
        htst = []
        bot_htst = []
        for ind, rows in selected_rows.iterrows():
            phit.append(rows['phit_avg'])
            htst.append(rows['htst'])
            bot_htst.append(rows['not_htst'])
        top_htst.append(dataframe['top_tst'].iloc[0] - dataframe['fm_top_tst'].iloc[0])
        for ind, rows in selected_rows.iterrows():
            top_htst.append(rows['not_htst'])
        top_htst = top_htst[:-1]
        def add_zero_values(var, block_size):
            if len(var) < block_size:
                zero_add = [np.nan for i in range(0, (block_size-len(var)))]
                res_var = np.concatenate([var,np.array(zero_add)], axis=0)
            else:
                res_var = var
            return res_var
        phit = add_zero_values(phit, block_size)
        htst = add_zero_values(htst, block_size)
        bot_htst = add_zero_values(bot_htst, block_size)
        top_htst = add_zero_values(top_htst, block_size)
        res = np.concatenate([phit, top_htst, htst, bot_htst], axis=0)
        res = res.reshape(1, -1) 
        return res
    def columns_names(name):
        columns_names = []
        for i in range(block_size):
            columns_names.append(name + '_' + str(i+1))
        return columns_names
    wellname = df['well'][0]
    fmname = df['FORMATION_up'][0]

    blocks = [df.iloc[i:i + block_size] for i in range(0, len(df), 1)]
    phit_names = columns_names('phit')
    htst_names = columns_names('htst')
    top_htst_names = columns_names('top_htst')
    bot_htst_names = columns_names('bot_htst')
    columns_names_list = phit_names + top_htst_names + htst_names + bot_htst_names

    res_lists = []
    for i in range(len(blocks)):
        res = rows_selection(blocks[i], block_size)
        res_lists.append(res)
    res = np.concatenate(res_lists)
    df_res = pd.DataFrame(res, columns = columns_names_list)
    df_res['well'] = wellname
    df_res['FORMATION_up'] = fmname
    return df_res

ntd_val_final8 = ntd_val_final8_clean
df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    df8 = ntd_val_final8[(ntd_val_final8.well==wellname)]
    res_df8 = top_phit_bot_clustering(df8, 1)
    df_lst.append(res_df8)
top_phit_bot_cluster = pd.concat(df_lst)
top_phit_bot_cluster.head(3)

### Clustering stride - resulting data analize

In [ ]:
# Uploading clustering data stride from Farid
ntd_val_clustering_stride_many_labels = pd.read_csv('.\inputoutput\\ntd_val_clustering_with_stride_labels.csv').drop('Unnamed: 0', axis=1)
ntd_val_clustering_stride_many_labels.columns

In [34]:
ntd_val_clust_stride = ntd_val_clustering_stride_many_labels[[  'phit_1', 'phit_2', 'phit_3', 'htst_1', 'htst_2', 'htst_3',
                                                                'not_htst_1', 'not_htst_2', 'not_htst_3', 'well', 'FORMATION_up',
                                                                'kmeans_3_labels']]
# ntd_val_clust_stride[ntd_val_clust_stride.well=='A01Y']

In [ ]:
ntd_val_clust_stride.well.unique()[:10]

In [ ]:
dataset = ntd_val_clust_stride
wellname = 'A02Z'

fig, ax = plt.subplots(1,9, figsize=(16,2.5))
for j in range(9):
    try:
        data = dataset[dataset.well == wellname].iloc[j]
        well_lst = []
        phit_lst = []
        htst_lst = []
        cluster_lst = []
        for i in range(3):
            phit_lst.append(data[i])
            phit_lst.append(0)
        for i in range(3):
            htst_lst.append(data[i+3])
            htst_lst.append(data[i+6])
        for i in range(6):
            cluster_lst.append(data[-1])
        htst_lst, phit_lst, cluster_lst
        well_collect_cluster = pd.DataFrame(zip(phit_lst, htst_lst, cluster_lst), columns=['phit', 'htst','cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_top = pd.DataFrame({'phit':[0], 'htst':[0], 'cluster':well_collect_cluster['cluster'].iloc[0],'depth':[0]})
        wellplot = pd.concat([df_top, well_collect_cluster]).reset_index().drop('index', axis=1)
        # wellplot = well_collect_cluster
        custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
        ax[j].plot(wellplot['phit'], wellplot['depth'], 
                drawstyle='steps-post', color=custom_palette[wellplot['cluster'].iloc[0]], alpha=1, lw=2)
        ax[j].set_xlim(0, 0.35)
        ax[j].invert_yaxis()
        ax[j].set_title(wellname)
        ax[j].tick_params(axis='both', which='major', labelsize=8)
        ax[j].grid()
        ax[j].tight_layout()
    except:
        pass

### Clustering 3 - resulting data analize

In [ ]:
# Uploading clustering data block3 from Farid
ntd_val_clustering2_with_labels = pd.read_csv('.\inputoutput\\ntd_val_clustering2_with_labels.csv').drop('Unnamed: 0', axis=1)
ntd_val_clustering2_with_labels.iloc[:,:11].head(3)

In [ ]:
def block_clustering_display(dataset, columns, rows, clustering, colors_dict, block, output_flag):
    fig, ax = plt.subplots(rows, columns, figsize=(17,10))
    counter = (block-1)*rows*columns
    # print('counter init ', counter)
    for j in range(0, rows):
        for i in range(0, columns):
            if counter < block*rows*columns+1:
                try:
                    data = dataset.iloc[counter]
                    title = dataset.iloc[counter]['well']
                    colors = dataset.iloc[counter][clustering]
                    y = [0, data.htst_1, data.not_htst_1, data.htst_2, data.not_htst_2, data.htst_3, data.not_htst_3]
                    x = [0, data.phit_1, 0, data.phit_2, 0, data.phit_3, 0]
                    # colors_dict = {2: 'red', 1: 'green', 0: 'blue'}
                    depth_summ_list = []
                    depth_counter = 0
                    for k in range(len(y)):
                        depth_counter += y[k]
                        depth_summ_list.append(depth_counter) 
                    ax[j,i].step(x, depth_summ_list, label=title, where='post', color=colors_dict[colors], linestyle='-', linewidth=2)
                    ax[j,i].set_ylim(-5,100)
                    ax[j,i].set_xlim(0,0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].legend(fontsize=8)
                    ax[j,i].grid()
                    counter +=1
                except:
                    pass
    plt.tight_layout()
    if output_flag == 'print':
        plt.savefig('.\plots\\clustering\\' + clustering + '_' + str(block) +'.png')
    else:
        pass
    # print('last value ', counter, block)

colors_dict_kmeans = {2: 'red', 1: 'green', 0: 'blue'}
colors_dict_dbscan = {-1: 'red', 0: 'green', 1: 'blue'}
colors_dict_aggl = {0: 'red', 1: 'green', 2: 'blue'}
colors_dict_gmm_labels = {1: 'red', 2: 'green', 0: 'blue'}
clustering = 'dbscan_labels'
colors_dict = colors_dict_dbscan

for i in range(1,len(ntd_val_clustering2_with_labels)//(9*4)+2):
    block_clustering_display(ntd_val_clustering2_with_labels, 9, 4, clustering, colors_dict, i, 'dont print')

In [39]:
def htst_sum_clustering(dataset, clustering_algorithm, cluster):
    cluster_num = dataset[dataset[clustering_algorithm]==cluster]
    cluster_num_gb = cluster_num.groupby(['well','FORMATION_up'])[['htst_1','htst_2','htst_3']].sum().reset_index()
    htst_sum_cluster_name = 'htst_sum_cluster_' + str(cluster)
    cluster_num_gb[htst_sum_cluster_name] = cluster_num_gb['htst_1'] + cluster_num_gb['htst_2'] + cluster_num_gb['htst_3']
    cluster_num = 'cluster_' + str(cluster)
    cluster_num_gb[cluster_num] = cluster
    return cluster_num_gb[['well','FORMATION_up',htst_sum_cluster_name,cluster_num]]

xy_bal8_init = df_bal_net2_kh[df_bal_net2_kh.FORMATION_up=='Balakhany VIII'][['well','FORMATION_up','X_mean','Y_mean']]
xy_bal8 = xy_bal8_init.groupby(['well','FORMATION_up']).apply(lambda x: 
                                                            x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
kmeans2 = htst_sum_clustering(ntd_val_clustering2_with_labels, 'kmeans_labels', 2)
kmeans1 = htst_sum_clustering(ntd_val_clustering2_with_labels, 'kmeans_labels', 1)
kmeans0 = htst_sum_clustering(ntd_val_clustering2_with_labels, 'kmeans_labels', 0)
kmeans = pd.merge(pd.merge( kmeans2, kmeans1, how='outer', on=['well','FORMATION_up']), 
                            kmeans0, how='outer', on=['well','FORMATION_up']) 
kmeans_xy = kmeans.set_index(['well','FORMATION_up']).join(xy_bal8.set_index(['well','FORMATION_up'])).reset_index()
kmeans_xy['htst_sum_cluster_2'].fillna(0, inplace=True)
kmeans_xy['htst_sum_cluster_1'].fillna(0, inplace=True)
kmeans_xy['htst_sum_cluster_0'].fillna(0, inplace=True)
kmeans_xy['X_mean'] = kmeans_xy['X_mean'].astype('int')/1000
kmeans_xy['Y_mean'] = kmeans_xy['Y_mean'].astype('int')/1000

In [ ]:
def pie_chart_map3():
       fig, ax = plt.subplots(figsize=(13,13))
       for ind, row in kmeans_xy[:].iterrows():
              ax.pie([row['htst_sum_cluster_2'], row['htst_sum_cluster_1'], row['htst_sum_cluster_0']], 
                     radius=0.3, center=(row['X_mean'], row['Y_mean']), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                     colors=['red', 'green', 'blue'], frame=True)
              plt.grid()
pie_chart_map3()

In [ ]:
def pie_chart_map3_pale(color_cluster):
       cluster_count = kmeans_xy.groupby('well')[['cluster_2','cluster_1','cluster_0']].count()
       cluster_count['cluster_sum'] = cluster_count.sum(axis=1)
       cluster_count = cluster_count.reset_index()
       cluster_count_1cluster = cluster_count[cluster_count.cluster_sum ==1]['well'].unique()
       cluster_count_2cluster = cluster_count[cluster_count.cluster_sum ==2]['well'].unique()
       cluster_count_3cluster = cluster_count[cluster_count.cluster_sum ==3]['well'].unique()
       kmeans_xy_1cluster = kmeans_xy[kmeans_xy.well.isin(cluster_count_1cluster)]
       kmeans_xy_2cluster = kmeans_xy[kmeans_xy.well.isin(cluster_count_2cluster)]
       kmeans_xy_3cluster = kmeans_xy[kmeans_xy.well.isin(cluster_count_3cluster)]

       fig, ax = plt.subplots(figsize=(13,13))
       for ind, row in kmeans_xy_2cluster.iterrows():
              ax.pie([row['htst_sum_cluster_2'], row['htst_sum_cluster_1'], row['htst_sum_cluster_0']], 
                     radius=0.3, center=(row['X_mean'], row['Y_mean']), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                     colors=['red', 'green', 'blue'], frame=True)
       for ind, row in kmeans_xy_3cluster.iterrows():
              ax.pie([row['htst_sum_cluster_2'], row['htst_sum_cluster_1'], row['htst_sum_cluster_0']], 
                     radius=0.3, center=(row['X_mean'], row['Y_mean']), wedgeprops={"linewidth": 1.5, "edgecolor": "black", "alpha":0.75},
                     colors=['red', 'green', 'blue'], frame=True)
       if color_cluster == 'blue':
              color_list = ['gray', 'gray', 'blue']
       if color_cluster == 'green':
              color_list = ['gray', 'green', 'gray']
       if color_cluster == 'red':
              color_list = ['red', 'gray', 'gray']
       for ind, row in kmeans_xy_1cluster.iterrows():
              ax.pie([row['htst_sum_cluster_2'], row['htst_sum_cluster_1'], row['htst_sum_cluster_0']], 
                     radius=0.3, center=(row['X_mean'], row['Y_mean']), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.25},
                     colors=color_list, frame=True)
       plt.grid()
pie_chart_map3_pale('blue')
pie_chart_map3_pale('green')
pie_chart_map3_pale('red')

In [ ]:
def cluster_phit_histo(datatset, cluster_num):
    kmeans = datatset[datatset.kmeans_labels == cluster_num]
    kmeans_phit = pd.concat([kmeans.phit_1, kmeans.phit_3, kmeans.phit_3]).reset_index().drop('index', axis=1)
    kmeans_phit['phit'] = kmeans_phit[0]
    kmeans_phit['cluster']=cluster_num
    kmeans_phit = kmeans_phit.drop(0, axis=1)
    return kmeans_phit
kmeans0_phit = cluster_phit_histo(ntd_val_clustering2_with_labels, 0)
kmeans1_phit = cluster_phit_histo(ntd_val_clustering2_with_labels, 1)
kmeans2_phit = cluster_phit_histo(ntd_val_clustering2_with_labels, 2)
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
kmeans_phit = pd.concat([kmeans2_phit, kmeans1_phit, kmeans0_phit])
sns.histplot(data=kmeans_phit, x='phit', hue='cluster', bins=25, kde=True, palette=custom_palette);

In [ ]:
def cluster_htst_histo(datatset, cluster_num):
    kmeans = datatset[datatset.kmeans_labels == cluster_num]
    kmeans_htst = pd.concat([kmeans.htst_1, kmeans.htst_2, kmeans.htst_3]).reset_index().drop('index', axis=1)
    kmeans_htst['htst'] = kmeans_htst[0]
    kmeans_htst['cluster']=cluster_num
    kmeans_htst = kmeans_htst.drop(0, axis=1)
    return kmeans_htst
kmeans0_htst = cluster_htst_histo(ntd_val_clustering2_with_labels, 0)
kmeans1_htst = cluster_htst_histo(ntd_val_clustering2_with_labels, 1)
kmeans2_htst = cluster_htst_histo(ntd_val_clustering2_with_labels, 2)
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
kmeans_htst = pd.concat([kmeans2_htst, kmeans1_htst, kmeans0_htst])
sns.histplot(data=kmeans_htst, x='htst', hue='cluster', bins=25, kde=True, palette=custom_palette);

In [ ]:
def cluster_nothtst_histo(datatset, cluster_num):
    kmeans = datatset[datatset.kmeans_labels == cluster_num]
    kmeans_nothtst = pd.concat([kmeans.not_htst_1, kmeans.not_htst_2, kmeans.not_htst_3]).reset_index().drop('index', axis=1)
    kmeans_nothtst['nothtst'] = kmeans_nothtst[0]
    kmeans_nothtst['cluster']=cluster_num
    kmeans_nothtst = kmeans_nothtst.drop(0, axis=1)
    return kmeans_nothtst
kmeans0_nothtst = cluster_nothtst_histo(ntd_val_clustering2_with_labels, 0)
kmeans1_nothtst = cluster_nothtst_histo(ntd_val_clustering2_with_labels, 1)
kmeans2_nothtst = cluster_nothtst_histo(ntd_val_clustering2_with_labels, 2)
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
kmeans_nothtst = pd.concat([kmeans2_nothtst, kmeans1_nothtst, kmeans0_nothtst])
sns.histplot(data=kmeans_nothtst, x='nothtst', hue='cluster', bins=25, kde=True, palette=custom_palette);

### Clustering 3 - clusters more > 3

In [ ]:
# Uploading clustering data block3 from Farid
ntd_val_clusters_morethen3 = pd.read_csv('.\inputoutput\\ntd_val_clustering2_with_labels_clusters345.csv').drop('Unnamed: 0', axis=1)
ntd_val_clusters_morethen3.columns

In [ ]:
def well_collecting_clusters(dataset, clustering):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        data_clusters = data[clustering].unique()
        well_lst = []
        phit_lst = []
        htst_lst = []
        cluster_lst = []
        for cluster_num in data_clusters:
            data_per_cluster = data[data[clustering] == cluster_num]
            for ind, row in data_per_cluster.iterrows():
                well_lst.append(wellname)
                cluster_lst.append(cluster_num)
                phit_lst.append(row['phit_1'])
                htst_lst.append(row['htst_1'])
                well_lst.append(wellname)
                phit_lst.append(0)
                htst_lst.append(row['not_htst_1'])
                cluster_lst.append(cluster_num)  

                well_lst.append(wellname)
                cluster_lst.append(cluster_num)
                phit_lst.append(row['phit_2'])
                htst_lst.append(row['htst_2'])
                well_lst.append(wellname)
                phit_lst.append(0)
                htst_lst.append(row['not_htst_2'])
                cluster_lst.append(cluster_num)

                well_lst.append(wellname)
                cluster_lst.append(cluster_num)
                phit_lst.append(row['phit_3'])
                htst_lst.append(row['htst_3'])
                well_lst.append(wellname)
                phit_lst.append(0)
                htst_lst.append(row['not_htst_3'])
                cluster_lst.append(cluster_num)   
        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, cluster_lst ), columns=['well','phit', 'htst', 'cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
ntd_val_kmeans4 = well_collecting_clusters(ntd_val_clusters_morethen3, 'kmeans_4_labels')
ntd_val_kmeans4.cluster.unique()

In [ ]:
def cluster_phit_histo4(datatset, cluster_col, cluster_num):
    kmeans = datatset[datatset[cluster_col] == cluster_num]
    kmeans_phit = pd.concat([kmeans.phit_1, kmeans.phit_3, kmeans.phit_3]).reset_index().drop('index', axis=1)
    kmeans_phit['phit'] = kmeans_phit[0]
    kmeans_phit['cluster']=cluster_num
    kmeans_phit = kmeans_phit.drop(0, axis=1)
    return kmeans_phit
kmeans0_phit = cluster_phit_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 0)
kmeans1_phit = cluster_phit_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 1)
kmeans2_phit = cluster_phit_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 2)
kmeans3_phit = cluster_phit_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 3)
custom_palette = {3: 'orange', 2: 'red', 1: 'green', 0: 'blue'}
kmeans_phit = pd.concat([kmeans3_phit, kmeans2_phit, kmeans1_phit, kmeans0_phit])
sns.histplot(data=kmeans_phit, x='phit', hue='cluster', bins=25, kde=True, palette=custom_palette);

In [ ]:
def cluster_htst_histo4(datatset, cluster_col, cluster_num):
    kmeans = datatset[datatset[cluster_col] == cluster_num]
    kmeans_htst = pd.concat([kmeans.htst_1, kmeans.htst_2, kmeans.htst_3]).reset_index().drop('index', axis=1)
    kmeans_htst['htst'] = kmeans_htst[0]
    kmeans_htst['cluster']=cluster_num
    kmeans_htst = kmeans_htst.drop(0, axis=1)
    return kmeans_htst
kmeans0_htst = cluster_htst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 0)
kmeans1_htst = cluster_htst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 1)
kmeans2_htst = cluster_htst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 2)
kmeans3_htst = cluster_htst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 3)
custom_palette = {3: 'orange', 2: 'red', 1: 'green', 0: 'blue'}
kmeans_htst = pd.concat([kmeans3_htst, kmeans2_htst, kmeans1_htst, kmeans0_htst])
sns.histplot(data=kmeans_htst, x='htst', hue='cluster', bins=25, kde=True, palette=custom_palette);

In [ ]:
def cluster_nothtst_histo4(datatset, cluster_col, cluster_num):
    kmeans = datatset[datatset[cluster_col] == cluster_num]
    kmeans_nothtst = pd.concat([kmeans.not_htst_1, kmeans.not_htst_2, kmeans.not_htst_3]).reset_index().drop('index', axis=1)
    kmeans_nothtst['nothtst'] = kmeans_nothtst[0]
    kmeans_nothtst['cluster']=cluster_num
    kmeans_nothtst = kmeans_nothtst.drop(0, axis=1)
    return kmeans_nothtst
kmeans0_nothtst = cluster_nothtst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 0)
kmeans1_nothtst = cluster_nothtst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 1)
kmeans2_nothtst = cluster_nothtst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 2)
kmeans3_nothtst = cluster_nothtst_histo4(ntd_val_clusters_morethen3, 'kmeans_4_labels', 3)
custom_palette = {3: 'orange', 2: 'red', 1: 'green', 0: 'blue'}
kmeans_nothtst = pd.concat([kmeans3_nothtst, kmeans2_nothtst, kmeans1_nothtst, kmeans0_nothtst])
sns.histplot(data=kmeans_nothtst, x='nothtst', hue='cluster', bins=25, kde=True, palette=custom_palette);

In [ ]:
def clusters_display_into_well(dataset, wellname):
    welldata = dataset[dataset.well == wellname]
    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
    fig, ax = plt.subplots(figsize=(2,4))
    ax.plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1)
    ax.set_xlim(0, 0.35)
    ax.invert_yaxis()
    ax.set_title(wellname)
    ax.grid()
    def clusters_indice(welldata):
        indices0 = welldata.index[(welldata['cluster'] == 0) & 
                                    (welldata['cluster'].shift(-1) != 0) | 
                                    (welldata['cluster'] == 0) & (welldata['cluster'].shift(+1) != 0)].tolist()
        indices1 = welldata.index[(welldata['cluster'] == 1) & 
                                    (welldata['cluster'].shift(-1) != 1) | 
                                    (welldata['cluster'] == 1) & (welldata['cluster'].shift(+1) != 1)].tolist()
        indices2 = welldata.index[(welldata['cluster'] == 2) & 
                                    (welldata['cluster'].shift(-1) != 2) | 
                                    (welldata['cluster'] == 2) & (welldata['cluster'].shift(+1) != 2)].tolist()
        indices3 = welldata.index[(welldata['cluster'] == 3) & 
                                    (welldata['cluster'].shift(-1) != 3) | 
                                    (welldata['cluster'] == 3) & (welldata['cluster'].shift(+1) != 3)].tolist()
        cluster_list = [indices0, indices1, indices2, indices3]
        return cluster_list
    cluster_list = clusters_indice(welldata)
    def clusters_rectangle(welldata, i, color):
            if cluster_list[i][0] > 0:
                cluster_xy = welldata['depth'].iloc[cluster_list[i][0]-1]
            else:
                cluster_xy = welldata['depth'].iloc[cluster_list[i][0]]
            try:
                # cluster_h = welldata['depth'].iloc[cluster_list[i][1]] - welldata['depth'].iloc[cluster_list[i][0]]
                ind1 = cluster_list[i][0]
                ind2 = cluster_list[i][1]
                cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
            except:
                pass
                # cluster_h = welldata['depth'].iloc[cluster_list[i][1]] - welldata['depth'].iloc[cluster_list[i][0]] 
            # cluster_h = welldata['depth'].iloc[cluster_list[i][1]+1] - welldata['depth'].iloc[cluster_list[i][0]] 
            rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
            ax.add_patch(rectangle)
    def clusters_rectangle2(welldata, i, color):
            cluster_xy = welldata['depth'].iloc[cluster_list[i][2]-1]
            # try:
            #     cluster_h = welldata['depth'].iloc[cluster_list[i][3]+1] - welldata['depth'].iloc[cluster_list[i][2]]
            # except:
            #     cluster_h = welldata['depth'].iloc[cluster_list[i][3]] - welldata['depth'].iloc[cluster_list[i][2]]
            try:
                ind1 = cluster_list[i][2]
                ind2 = cluster_list[i][3]
                cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
            except:
                pass
            rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
            ax.add_patch(rectangle)

    for i in range(len(cluster_list)):
        if len(cluster_list[i]) > 0 and i==0:
            clusters_rectangle(welldata, i, 'blue')
            # print('cluster 0', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'blue')
            except:
                pass
        if len(cluster_list[i]) > 0 and i==1:
            clusters_rectangle(welldata, i, 'green')
            # print('cluster 1', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'green')
            except:
                pass
        if len(cluster_list[i]) > 0 and i==2:
            clusters_rectangle(welldata, i, 'red')
            # print('cluster 2', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'red')
            except:
                pass
        if len(cluster_list[i]) > 0 and i==3:
            clusters_rectangle(welldata, i, 'orange')
            # print('cluster 2', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'orange')
            except:
                pass
    print(welldata)
clusters_display_into_well(ntd_val_kmeans4, 'B01Y')

In [ ]:
def coloring_clusters4_matrix_plot(dataset, letters_list, rows, columns, output_flag, clustering):
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()

                    def clusters_indice(welldata):
                        indices0 = welldata.index[(welldata['cluster'] == 0) & 
                                                  (welldata['cluster'].shift(-1) != 0) | 
                                                  (welldata['cluster'] == 0) & (welldata['cluster'].shift(+1) != 0)].tolist()
                        indices1 = welldata.index[(welldata['cluster'] == 1) & 
                                                  (welldata['cluster'].shift(-1) != 1) | 
                                                  (welldata['cluster'] == 1) & (welldata['cluster'].shift(+1) != 1)].tolist()
                        indices2 = welldata.index[(welldata['cluster'] == 2) & 
                                                  (welldata['cluster'].shift(-1) != 2) | 
                                                  (welldata['cluster'] == 2) & (welldata['cluster'].shift(+1) != 2)].tolist()
                        indices3 = welldata.index[(welldata['cluster'] == 3) & 
                                                  (welldata['cluster'].shift(-1) != 3) | 
                                                  (welldata['cluster'] == 3) & (welldata['cluster'].shift(+1) != 3)].tolist()
                        cluster_list = [indices0, indices1, indices2, indices3]
                        return cluster_list
                    cluster_list = clusters_indice(welldata)
                    def clusters_rectangle(welldata, k, color):
                        if cluster_list[k][0] > 0:
                            cluster_xy = welldata['depth'].iloc[cluster_list[k][0]-1]
                        else:
                            cluster_xy = welldata['depth'].iloc[cluster_list[k][0]]
                        try:
                            ind1 = cluster_list[k][0]
                            ind2 = cluster_list[k][1]
                            cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
                        except:
                            pass
                        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
                        ax[j,i].add_patch(rectangle)
                    def clusters_rectangle2(welldata, k, color):
                        cluster_xy = welldata['depth'].iloc[cluster_list[i][2]-1]
                        try:
                            ind1 = cluster_list[k][2]
                            ind2 = cluster_list[k][3]
                            cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
                        except:
                            pass
                        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
                        ax[j,i].add_patch(rectangle)
                    
                    for k in range(len(cluster_list)):
                        if len(cluster_list[k]) > 0 and k==0:
                            clusters_rectangle(welldata, k, 'blue')
                            try:
                                clusters_rectangle2(welldata, k, 'blue')
                            except:
                                pass
                        if len(cluster_list[k]) > 0 and k==1:
                            clusters_rectangle(welldata, k, 'green')
                            try:
                                clusters_rectangle2(welldata, k, 'green')
                            except:
                                pass
                        if len(cluster_list[k]) > 0 and k==2:
                            clusters_rectangle(welldata, k, 'red')
                            try:
                                clusters_rectangle2(welldata, k, 'red')
                            except:
                                pass
                        if len(cluster_list[k]) > 0 and k==3:
                            clusters_rectangle(welldata, k, 'orange')
                            try:
                                clusters_rectangle2(welldata, k, 'orange')
                            except:
                                pass
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\clustering_wells_3blocks_4\\' + clustering + '_' + str(letter) +'.png')
        else:
            pass
coloring_clusters4_matrix_plot(ntd_val_kmeans4, 
                              ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], 
                              4, 9, 'dont print', 'kmeans4')

### Coloring clustered samples into wells

In [ ]:
wellname = 'A01Y'
ntd_val_clustering2_with_labels = pd.read_csv('.\inputoutput\\ntd_val_clustering2_with_labels.csv').drop('Unnamed: 0', axis=1)
ntd_val_clustering2_with_labels[ntd_val_clustering2_with_labels.well==wellname]

In [ ]:
def clusters_per_sample_display(dataset, columns, rows, clustering, colors_dict, block):
    fig, ax = plt.subplots(rows, columns, figsize=(17,10))
    counter = (block-1)*rows*columns
    for j in range(0, rows):
        for i in range(0, columns):
            if counter < block*rows*columns+1:
                try:
                    data = dataset.iloc[counter]
                    title = dataset.iloc[counter]['well']
                    colors = dataset.iloc[counter][clustering]
                    y = [0, data.htst_1, data.not_htst_1, data.htst_2, data.not_htst_2, data.htst_3, data.not_htst_3]
                    x = [0, data.phit_1, 0, data.phit_2, 0, data.phit_3, 0]

                    depth_summ_list = []
                    depth_counter = 0
                    for k in range(len(y)):
                        depth_counter += y[k]
                        depth_summ_list.append(depth_counter) 
                    ax[j,i].step(x, depth_summ_list, label=title, where='post', color=colors_dict[colors], linestyle='-', linewidth=2)
                    ax[j,i].set_ylim(-5,100)
                    ax[j,i].set_xlim(0,0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].legend(fontsize=8)
                    counter +=1
                except:
                    pass
colors_dict = {2: 'red', 1: 'green', 0: 'blue'}
for i in range(1,len(ntd_val_clustering2_with_labels)//(9*4)+2)[:10]:
    clusters_per_sample_display(ntd_val_clustering2_with_labels, 9, 4, 'kmeans_labels', colors_dict, i)

In [ ]:
def well_collecting_clusters(dataset, clustering):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        data_clusters = data[clustering].unique()
        well_lst = []
        phit_lst = []
        htst_lst = []
        cluster_lst = []
        for cluster_num in data_clusters:
            data_per_cluster = data[data[clustering] == cluster_num]
            for ind, row in data_per_cluster.iterrows():
                well_lst.append(wellname)
                cluster_lst.append(cluster_num)
                phit_lst.append(row['phit_1'])
                htst_lst.append(row['htst_1'])
                well_lst.append(wellname)
                phit_lst.append(0)
                htst_lst.append(row['not_htst_1'])
                cluster_lst.append(cluster_num)  

                well_lst.append(wellname)
                cluster_lst.append(cluster_num)
                phit_lst.append(row['phit_2'])
                htst_lst.append(row['htst_2'])
                well_lst.append(wellname)
                phit_lst.append(0)
                htst_lst.append(row['not_htst_2'])
                cluster_lst.append(cluster_num)

                well_lst.append(wellname)
                cluster_lst.append(cluster_num)
                phit_lst.append(row['phit_3'])
                htst_lst.append(row['htst_3'])
                well_lst.append(wellname)
                phit_lst.append(0)
                htst_lst.append(row['not_htst_3'])
                cluster_lst.append(cluster_num)   
        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, cluster_lst ), columns=['well','phit', 'htst', 'cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
ntd_val_kmeans3 = well_collecting_clusters(ntd_val_clustering2_with_labels, 'kmeans_labels')
ntd_val_kmeans3[ntd_val_kmeans3.well=='A01Y']

In [ ]:
def clusters_display_into_well(dataset, wellname):
    welldata = dataset[dataset.well == wellname]
    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)

    fig, ax = plt.subplots(figsize=(2,4))
    ax.plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1)
    ax.set_xlim(0, 0.35)
    ax.invert_yaxis()
    ax.set_title(wellname)
    ax.grid()

    def clusters_indice(welldata):
        indices0 = welldata.index[(welldata['cluster'] == 0) & (welldata['cluster'].shift(-1) != 0) | (welldata['cluster'] == 0) & (welldata['cluster'].shift(+1) != 0)].tolist()
        indices1 = welldata.index[(welldata['cluster'] == 1) & (welldata['cluster'].shift(-1) != 1) | (welldata['cluster'] == 1) & (welldata['cluster'].shift(+1) != 1)].tolist()
        indices2 = welldata.index[(welldata['cluster'] == 2) & (welldata['cluster'].shift(-1) != 2) | (welldata['cluster'] == 2) & (welldata['cluster'].shift(+1) != 2)].tolist()
        cluster_list = [indices0, indices1, indices2]
        return cluster_list
    cluster_list = clusters_indice(welldata)
    def clusters_rectangle(welldata, i, color):
            if cluster_list[i][0] > 0:
                cluster_xy = welldata['depth'].iloc[cluster_list[i][0]-1]
            else:
                cluster_xy = welldata['depth'].iloc[cluster_list[i][0]]
            try:
                # cluster_h = welldata['depth'].iloc[cluster_list[i][1]] - welldata['depth'].iloc[cluster_list[i][0]]
                ind1 = cluster_list[i][0]
                ind2 = cluster_list[i][1]
                cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
            except:
                pass
                # cluster_h = welldata['depth'].iloc[cluster_list[i][1]] - welldata['depth'].iloc[cluster_list[i][0]] 
            # cluster_h = welldata['depth'].iloc[cluster_list[i][1]+1] - welldata['depth'].iloc[cluster_list[i][0]] 
            rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
            ax.add_patch(rectangle)
    def clusters_rectangle2(welldata, i, color):
            cluster_xy = welldata['depth'].iloc[cluster_list[i][2]-1]
            # try:
            #     cluster_h = welldata['depth'].iloc[cluster_list[i][3]+1] - welldata['depth'].iloc[cluster_list[i][2]]
            # except:
            #     cluster_h = welldata['depth'].iloc[cluster_list[i][3]] - welldata['depth'].iloc[cluster_list[i][2]]
            try:
                ind1 = cluster_list[i][2]
                ind2 = cluster_list[i][3]
                cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
            except:
                pass
            rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
            ax.add_patch(rectangle)

    for i in range(len(cluster_list)):
        if len(cluster_list[i]) > 0 and i==0:
            clusters_rectangle(welldata, i, 'blue')
            # print('cluster 0', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'blue')
            except:
                pass
        if len(cluster_list[i]) > 0 and i==1:
            clusters_rectangle(welldata, i, 'green')
            # print('cluster 1', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'green')
            except:
                pass
        if len(cluster_list[i]) > 0 and i==2:
            clusters_rectangle(welldata, i, 'red')
            # print('cluster 2', cluster_list[i], i, cluster_list[i][1])
            try:
                clusters_rectangle2(welldata, i, 'red')
            except:
                pass
    print(welldata)
clusters_display_into_well(ntd_val_kmeans3, 'C14Z')

In [ ]:
def coloring_clusters_matrix_plot(dataset, letters_list, rows, columns, output_flag, clustering):
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    
                    def clusters_indice(welldata):
                        indices0 = welldata.index[(welldata['cluster'] == 0) & (welldata['cluster'].shift(-1) != 0) | (welldata['cluster'] == 0) & (welldata['cluster'].shift(+1) != 0)].tolist()
                        indices1 = welldata.index[(welldata['cluster'] == 1) & (welldata['cluster'].shift(-1) != 1) | (welldata['cluster'] == 1) & (welldata['cluster'].shift(+1) != 1)].tolist()
                        indices2 = welldata.index[(welldata['cluster'] == 2) & (welldata['cluster'].shift(-1) != 2) | (welldata['cluster'] == 2) & (welldata['cluster'].shift(+1) != 2)].tolist()
                        cluster_list = [indices0, indices1, indices2]
                        return cluster_list
                    cluster_list = clusters_indice(welldata)
                    def clusters_rectangle(welldata, k, color):
                        if cluster_list[k][0] > 0:
                            cluster_xy = welldata['depth'].iloc[cluster_list[k][0]-1]
                        else:
                            cluster_xy = welldata['depth'].iloc[cluster_list[k][0]]
                        try:
                            ind1 = cluster_list[k][0]
                            ind2 = cluster_list[k][1]
                            cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
                        except:
                            pass
                        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
                        ax[j,i].add_patch(rectangle)
                    def clusters_rectangle2(welldata, k, color):
                        cluster_xy = welldata['depth'].iloc[cluster_list[i][2]-1]
                        try:
                            ind1 = cluster_list[k][2]
                            ind2 = cluster_list[k][3]
                            cluster_h = welldata['htst'].iloc[ind1:ind2+1].sum()
                        except:
                            pass
                        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
                        ax[j,i].add_patch(rectangle)
                    
                    for k in range(len(cluster_list)):
                        if len(cluster_list[k]) > 0 and k==0:
                            clusters_rectangle(welldata, k, 'blue')
                            try:
                                clusters_rectangle2(welldata, k, 'blue')
                            except:
                                pass
                        if len(cluster_list[k]) > 0 and k==1:
                            clusters_rectangle(welldata, k, 'green')
                            try:
                                clusters_rectangle2(welldata, k, 'green')
                            except:
                                pass
                        if len(cluster_list[k]) > 0 and k==2:
                            clusters_rectangle(welldata, k, 'red')
                            try:
                                clusters_rectangle2(welldata, k, 'red')
                            except:
                                pass
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\clustering_wells_3blocks\\' + clustering + '_' + str(letter) +'.png')
        else:
            pass
coloring_clusters_matrix_plot(ntd_val_kmeans3, 
                              ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], 
                              4, 9, 'dontprint', 'kmeans3')

### Clustering top_phi_bot layering

In [ ]:
# Uploading clustering data top_phi_bot from Farid
ntd_val_clustering_top_phi_bot = pd.read_csv('.\inputoutput\\ntd_val_clustering_one_layer_with_labels.csv').drop('Unnamed: 0', axis=1)
ntd_val_clustering_top_phi_bot.columns

In [ ]:
def well_collecting_clusters_top_phi_bot(dataset, clustering):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, cluster_lst ), columns=['well','phit', 'htst', 'cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
ntd_val_tpb_test = well_collecting_clusters_top_phi_bot(ntd_val_clustering_top_phi_bot, 'kmeans_3_labels')

def clusters_display_into_well_tpb3(dataset, wellname):
    def clusters_rectangle(data, i, color):
        # cluster_xy = data['depth'].iloc[i-2]
        cluster_xy = data['depth'].iloc[i-1]
        # cluster_h = data['depth'].iloc[i+1] - data['depth'].iloc[i-2]
        cluster_h = data['depth'].iloc[i] - data['depth'].iloc[i-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax.add_patch(rectangle)
    data = dataset[dataset.well == wellname]
    top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':data['cluster'].iloc[0],'depth':[0]})
    data = pd.concat([top, data])
    # print(data)
    fig, ax = plt.subplots(figsize=(2,4))
    ax.plot(data['phit'], data['depth'], drawstyle='steps-post', color='black', lw=0.75, alpha=0.75)
    ax.set_xlim(0, 0.35)
    ax.invert_yaxis()
    ax.set_title(wellname)
    ax.grid()
    for i in range(len(data)):
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 0:
            clusters_rectangle(data, i, 'blue')
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 1:
            clusters_rectangle(data, i, 'green')
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 2:
            clusters_rectangle(data, i, 'red')
clusters_display_into_well_tpb3(ntd_val_tpb_test, 'A01W')

In [ ]:
def coloring_clusters_matrix_tpb3(dataset, letters_list, rows, columns, clustering, output_flag):
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                            clusters_rectangle(welldata, k, 'blue')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                            clusters_rectangle(welldata, k, 'red')
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\clustering_wells_tpb\\' + clustering + '_' + str(letter) +'.png')
        else:
            pass
# ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
coloring_clusters_matrix_tpb3(ntd_val_tpb_test, ['A'], 4, 9, 'kmeans3_tpb', 'dontprint')

In [ ]:
def top_htst_concat(datatset, cluster_num):
    top_htst = datatset[(datatset.cluster.isna()) & (datatset.cluster.shift(-1)==cluster_num)]
    top_htst['cluster'] = cluster_num
    return top_htst

dataset = ntd_val_tpb_test
top_tpb0 = top_htst_concat(dataset, 0)
top_tpb1 = top_htst_concat(dataset, 1)
top_tpb2 = top_htst_concat(dataset, 2)
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
top_tpb3 = pd.concat([top_tpb0, top_tpb1, top_tpb2])
ax = sns.histplot(data=top_tpb3, x='htst', hue='cluster', bins=25, kde=True, palette=custom_palette)
ax.set_xticks(range(0, 60, 5)), ax.grid(True, axis='x')
ax.set(xlabel='top_htst', ylabel='Count');

In [ ]:
def bot_htst_concat(dataset, cluster_num):
    bot_htst = dataset[(dataset.cluster.isna()) & (dataset.cluster.shift(1)==cluster_num)]
    bot_htst['cluster'] = cluster_num
    return bot_htst
dataset = ntd_val_tpb_test
bot_tpb0 = bot_htst_concat(dataset, 0)
bot_tpb1 = bot_htst_concat(dataset, 1)
bot_tpb2 = bot_htst_concat(dataset, 2)
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
bot_tpb3 = pd.concat([bot_tpb0, bot_tpb1, bot_tpb2])
ax = sns.histplot(data=bot_tpb3, x='htst', hue='cluster', bins=25, kde=True, palette=custom_palette)
ax.set_xticks(range(0, 50, 5)), ax.grid(True, axis='x')
ax.set(xlabel='bot_htst', ylabel='Count');

In [ ]:
def topbot_htst_concat(dataset, cluster_num):
    top_htst = dataset[(dataset.cluster.isna()) & (dataset.cluster.shift(-1)==cluster_num)]
    bot_htst = dataset[(dataset.cluster.isna()) & (dataset.cluster.shift(1)==cluster_num)]
    topbot_tpb = pd.concat([top_htst, bot_htst])
    topbot_tpb['cluster'] = cluster_num
    return topbot_tpb
dataset = ntd_val_tpb_test
topbot_tpb0 = topbot_htst_concat(dataset, 0)
topbot_tpb1 = topbot_htst_concat(dataset, 1)
topbot_tpb2 = topbot_htst_concat(dataset, 2)
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
topbot_tpb3 = pd.concat([topbot_tpb0, topbot_tpb1, topbot_tpb2])
ax = sns.histplot(data=topbot_tpb3, x='htst', hue='cluster', bins=25, kde=True, palette=custom_palette)
ax.set_xticks(range(0, 60, 5)), ax.grid(True, axis='x')
ax.set(xlabel='top_bot_htst', ylabel='Count');

In [ ]:
dataset = ntd_val_tpb_test
phit0 = dataset[dataset.cluster==0][['phit', 'cluster']]
phit1 = dataset[dataset.cluster==1][['phit', 'cluster']]
phit2 = dataset[dataset.cluster==2][['phit', 'cluster']]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
phit_tpb3 = pd.concat([phit0, phit1, phit2])
ax = sns.histplot(data=phit_tpb3, x='phit', hue='cluster', bins=35, kde=True, palette=custom_palette)
ax.grid(True, axis='x')
ax.set_xticks(np.arange(0.12, 0.32, 0.02));

In [ ]:
dataset = ntd_val_tpb_test
htst0 = dataset[dataset.cluster==0]
htst1 = dataset[dataset.cluster==1]
htst2 = dataset[dataset.cluster==2]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
htst_tpb3 = pd.concat([htst0, htst1, htst2])
ax = sns.histplot(data=htst_tpb3, x='htst', hue='cluster', bins=26, kde=True, palette=custom_palette)
ax.grid(True, axis='x')
ax.set_xticks(np.arange(0, 60, 5));

### Clustering top_phi_bot layering v2 

#### Dataset preparation

In [ ]:
# algorithms = ["kmeans","gmm","agglomerative"]
# algorithm_model_map = {}
# cluster_max_span = 10
 
# for algo in algorithms:
#     for cluster_num in range(3,cluster_max_span+1):
#         labels = None
#         if algo == "kmeans":
#             kmeans = KMeans(n_clusters=cluster_num, random_state=42)
#             kmeans_labels = kmeans.fit_predict(normalized_data)
#             labels = kmeans_labels
 
#         elif algo == "gmm":
#             gmm = GaussianMixture(n_components=cluster_num, random_state=42)
#             gmm.fit(normalized_data)
#             gmm_labels = gmm.predict(normalized_data)
#             labels = gmm_labels
#         elif algo == "agglomerative":
#             agglomerative = AgglomerativeClustering(n_clusters=cluster_num)
#             agglomerative_labels = agglomerative.fit_predict(normalized_data)
#             labels = agglomerative_labels

#         na_dropped_data[f"{algo}_{cluster_num}_labels"]=labels
 
# na_dropped_data.to_csv("top_phi_bot_cluster_ntg10_with_labels.csv")

In [ ]:
def clustering_data_calculation(dataset):
    """
    """
    df_net2_bal8 = dataset[[    'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal8 = df_net2_bal8[    (df_net2_bal8.well.isin(well_clean8_v2)) & 
                                    (df_net2_bal8.FORMATION_up=='Balakhany VIII')]
    df_net2_bal10 = dataset[[   'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal10 = df_net2_bal10[  (df_net2_bal10.well.isin(well_clean10_v2)) & 
                                    (df_net2_bal10.FORMATION_up=='Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2')
    def ntd_calculation_brief(dataset,well,desired_fm, net_var):
        data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list8 = []
    for well in tqdm(df_net2_bal8.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8.append(df)
    ntd_net2_8 = pd.concat(df_recalc_list8)
    ntd_net2_8.drop_duplicates(inplace=True)
    df_recalc_list10 = []
    for well in tqdm(df_net2_bal10.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10.append(df)
    ntd_net2_10 = pd.concat(df_recalc_list10)
    ntd_net2_10.drop_duplicates(inplace=True)

    print('Calculation values for NTD Bal8 and Bal10')
    def ntd_properties_dataframe(dataset_ntd, dataset_logs, fmname):
        well_data = []
        well_formation = fmname
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()[:]):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_gperm = []
            well_top = []
            well_bot = []
            well_h = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_gperm.append(gmean(perm_lst))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val_bal8 = ntd_properties_dataframe(ntd_net2_8, df_net2_bal8, 'Balakhany VIII')
    ntd_val_bal10 = ntd_properties_dataframe(ntd_net2_10, df_net2_bal10, 'Balakhany X')
    ntd_val_final = pd.concat([ntd_val_bal8, ntd_val_bal10])
    return ntd_val_final
ntd_val_final = clustering_data_calculation(df_bal_net2_kh)
ntd_val_final8 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany VIII']
ntd_val_final10 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany X']

def nothtst_nan_fill(dataset_ntd, fmname):
    def nan_change_diff_fmbottom(dataset, wellname, fmname):
        row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
        row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
        return row_change
    df_list = []
    for wellname in dataset_ntd.well.unique():
        df = nan_change_diff_fmbottom(dataset_ntd, wellname, fmname)
        df_list.append(df)
    res_df_list = pd.concat(df_list)
    result = pd.concat([dataset_ntd, res_df_list])
    result = result.sort_values(by=['well','top_tst'])
    result_final = result.dropna(subset=['not_htst'], axis=0)
    return result_final
ntd_val_final8_clean = nothtst_nan_fill(ntd_val_final8, 'Balakhany VIII')
ntd_val_final10_clean = nothtst_nan_fill(ntd_val_final10, 'Balakhany X')

In [ ]:
#Clustering with top-phi-bot v2
def top_phit_bot_clustering(dataset, print_flag, path):
    def top_phit_bot_collection_run(dataset):
        def top_phit_bot_collection(dataset, wellname):
            data = dataset[dataset.well == wellname]
            data['top_htst'] = data['top_tst'] - data['fm_top_tst']
            data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
            data['bot_htst'] = data['not_htst']
            data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 
                         'top_htst','htst','bot_htst', 'fm_top_tst', 'fm_bot_tst']]
            return data
        df_lst = []
        for wellname in tqdm(dataset.well.unique()):
            res_df = top_phit_bot_collection(dataset, wellname)
            df_lst.append(res_df)
        top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
        return top_phi_bot_cluster
    top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

    def top_phit_bot_ntg_run(dataset):
        def top_phit_bot_ntg(dataset, wellname):
            ntg = []
            data = dataset[dataset.well == wellname].reset_index(drop=True)
            for ind, row in data.iterrows():
                if ind == 0:
                    ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                if ind != 0:
                    ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                if ind == len(data):
                    ntg.append(row['htst']/(row['top_htst'] + row['htst']))
            result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
            return result
        df_lst = []
        for wellname in dataset.well.unique():
            df = top_phit_bot_ntg(dataset, wellname)
            df_lst.append(df)
        top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
        return top_phi_bot_cluster_ntg
    top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)

    if print_flag == 'print':
        top_phi_bot_cluster_ntg.to_csv(path, index=False)
    else:
        pass
    return top_phi_bot_cluster_ntg
top_phi_bot_cluster8 = top_phit_bot_clustering(ntd_val_final8_clean, 'dont_print', '.\output\\top_phi_bot_cluster8.csv')
top_phi_bot_cluster10 = top_phit_bot_clustering(ntd_val_final10_clean, 'dont_print', '.\output\\top_phi_bot_cluster8.csv')
# top_phi_bot_cluster8 = top_phi_bot_cluster8[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg','htst','ntg']]
# top_phi_bot_cluster10 = top_phi_bot_cluster10[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg','htst','ntg']]
# top_phi_bot_cluster8.to_csv('.\output\\top_phi_bot_cluster_ntg8.csv')
# top_phi_bot_cluster10.to_csv('.\output\\top_phi_bot_cluster_ntg10.csv')

In [ ]:
cluster_top_phi_bot_v2 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\top_phi_bot_cluster_ntg8_with_labels.csv').drop(['Unnamed: 0.1','Unnamed: 0'], axis=1)
cluster_top_phi_bot_v2.columns

In [ ]:
cluster_top_phi_bot_v3 = cluster_top_phi_bot_v2.join(top_phi_bot_cluster8[['top_htst', 'bot_htst']])
top_phi_bot_cluster8[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg','htst','ntg']].head(3)

#### Kmeans

In [ ]:
def well_collecting_clusters_top_phi_bot_v2(dataset, clustering):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        ntg_lst = []
        vsh_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

            ntg_lst.append(0)
            ntg_lst.append(row['ntg'])

            vsh_lst.append(0)
            vsh_lst.append(row['vsh_avg'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, ntg_lst, vsh_lst, cluster_lst ), columns=['well','phit', 'htst', 
                                                                                                               'ntg', 'vsh', 'cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
tpb_kmeans_v3 = well_collecting_clusters_top_phi_bot_v2(cluster_top_phi_bot_v3, 'kmeans_3_labels')

def clusters_display_into_well_tpb3(dataset, wellname):
    def clusters_rectangle(data, i, color):
        # cluster_xy = data['depth'].iloc[i-2]
        cluster_xy = data['depth'].iloc[i-1]
        # cluster_h = data['depth'].iloc[i+1] - data['depth'].iloc[i-2]
        cluster_h = data['depth'].iloc[i] - data['depth'].iloc[i-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax.add_patch(rectangle)
    data = dataset[dataset.well == wellname]
    top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':data['cluster'].iloc[0],'depth':[0]})
    data = pd.concat([top, data])
    # print(data)
    fig, ax = plt.subplots(figsize=(2,4))
    ax.plot(data['phit'], data['depth'], drawstyle='steps-post', color='black', lw=0.75, alpha=0.75)
    ax.set_xlim(0, 0.35)
    ax.invert_yaxis()
    ax.set_title(wellname)
    ax.grid()
    for i in range(len(data)):
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 0:
            clusters_rectangle(data, i, 'blue')
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 1:
            clusters_rectangle(data, i, 'green')
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 2:
            clusters_rectangle(data, i, 'red')
clusters_display_into_well_tpb3(tpb_kmeans_v3, 'C16')

In [ ]:
def coloring_clusters_matrix_tpb3(dataset, letters_list, rows, columns, clustering, output_flag):
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                            clusters_rectangle(welldata, k, 'blue')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                            clusters_rectangle(welldata, k, 'red')
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\clustering_wells_tpb\\' + clustering + '_' + str(letter) +'.png')
        else:
            pass
# ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']        
coloring_clusters_matrix_tpb3(tpb_kmeans_v3, ['B'], 4, 9, 'kmeans3_tpb', 'dontprint')

In [ ]:
tpb_kmeans_v4 = tpb_kmeans_v3[tpb_kmeans_v3.phit !=0]
fig, axs = plt.subplots(1, 2, figsize=(18, 4))
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
sns.histplot(data=tpb_kmeans_v4, x='phit', hue='cluster', ax=axes[0], kde=True,  palette=custom_palette)

In [ ]:
# dataset = tpb_kmeans_v3
def histo_phit(dataset):
    phit0 = dataset[dataset.cluster==0][['phit', 'cluster']]
    phit1 = dataset[dataset.cluster==1][['phit', 'cluster']]
    phit2 = dataset[dataset.cluster==2][['phit', 'cluster']]
    custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
    phit_tpb3 = pd.concat([phit0, phit1, phit2])
    ax = sns.histplot(data=phit_tpb3, x='phit', hue='cluster', bins=35, kde=True, palette=custom_palette)
    ax.grid(True, axis='x')
    ax.set_xticks(np.arange(0.12, 0.32, 0.02));
histo_phit(tpb_kmeans_v3)

In [ ]:
dataset = tpb_kmeans_v3
ntg0 = dataset[dataset.cluster==0][['ntg', 'cluster']]
ntg1 = dataset[dataset.cluster==1][['ntg', 'cluster']]
ntg2 = dataset[dataset.cluster==2][['ntg', 'cluster']]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
ntg_tpb_v3 = pd.concat([ntg0, ntg1, ntg2])
ax = sns.histplot(data=ntg_tpb_v3, x='ntg', hue='cluster', bins=35, kde=True, palette=custom_palette)
ax.grid(True, axis='x')
# ax.set_xticks(np.arange(0.12, 0.32, 0.02));

In [ ]:
dataset = tpb_kmeans_v3
htst0 = dataset[dataset.cluster==0]
htst1 = dataset[dataset.cluster==1]
htst2 = dataset[dataset.cluster==2]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
htst_tpb_v3 = pd.concat([htst0, htst1, htst2])
ax = sns.histplot(data=htst_tpb_v3, x='htst', hue='cluster', bins=50, kde=True, palette=custom_palette)
ax.grid(True, axis='x')
ax.set_xticks(np.arange(0, 60, 5));

In [ ]:
dataset = tpb_kmeans_v3
vsh0 = dataset[dataset.cluster==0]
vsh1 = dataset[dataset.cluster==1]
vsh2 = dataset[dataset.cluster==2]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
vsh_tpb_v3 = pd.concat([vsh0, vsh1, vsh2])
ax = sns.histplot(data=vsh_tpb_v3, x='vsh', hue='cluster', bins=50, kde=True, palette=custom_palette)
ax.grid(True, axis='x');

#### Kmeans map

In [44]:
cluster_kmeans.to_csv('cluster_kmeans.csv', index=False)

In [ ]:
def cluster_data_to_map(dataset, dataset_coord, comment):
    def dataset_groupby(dataset):
        result = dataset.groupby(['well','cluster'])['htst'].sum().reset_index()
        return result
    tpb_test_v3_piechart = dataset_groupby(dataset)

    def cluster_transpose(dataset, wellname):
        result = dataset[dataset.well==wellname]
        result.loc[result.cluster == 0, 'cluster_0'] = result.htst
        result.loc[result.cluster == 1, 'cluster_1'] = result.htst
        result.loc[result.cluster == 2, 'cluster_2'] = result.htst
        result.fillna(0)
        result = result.groupby('well').sum().reset_index()
        result = result[['well', 'cluster_0', 'cluster_1','cluster_2']]
        return result
    df_lst = []
    for wellname in tpb_test_v3_piechart.well.unique():
        df = cluster_transpose(tpb_test_v3_piechart, wellname)
        df_lst.append(df)
    data_transpose = pd.concat(df_lst).reset_index(drop=True)

    def coordinates_calc(dataset_coord, fm):
        dataset_coord = dataset_coord[dataset_coord.FORMATION_up == fm]
        result = dataset_coord.groupby('well')[['X_mean','Y_mean']].apply(lambda x: x.iloc[0]).reset_index()
        return result 
    coord = coordinates_calc(dataset_coord, 'Balakhany VIII')
    data_transpose_coord = data_transpose.set_index('well').join(coord.set_index('well'), rsuffix='_coord').reset_index()

    def piechart_map(dataset_map):
        fig, ax = plt.subplots(figsize=(13,10))
        ax.scatter(bal8_1510['X']/1000, bal8_1510['Y']/1000, c=bal8_1510['geobody'])
        for ind, row in dataset_map.iterrows():
                ax.pie([row['cluster_0'], row['cluster_1'], row['cluster_2']], 
                        radius=0.3, center=(row['X_mean']/1000, row['Y_mean']/1000), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                        colors=['blue', 'green', 'red'], frame=True)
        # plt.grid()
        plt.title(comment)     
    piechart_map(data_transpose_coord)
    return data_transpose_coord

cluster_kmeans = cluster_data_to_map(tpb_kmeans_v3, df_bal_net2_kh, 'Well clustering by Kmeans & Bal8 1510 geobodies')

#### Gmm

In [ ]:
def well_collecting_clusters_top_phi_bot_v2(dataset, clustering):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        ntg_lst = []
        vsh_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

            ntg_lst.append(0)
            ntg_lst.append(row['ntg'])

            vsh_lst.append(0)
            vsh_lst.append(row['vsh_avg'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, ntg_lst, vsh_lst, cluster_lst ), columns=['well','phit', 'htst', 
                                                                                                               'ntg', 'vsh', 'cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
tpb_gmm_v3 = well_collecting_clusters_top_phi_bot_v2(cluster_top_phi_bot_v3, 'gmm_3_labels')

def clusters_display_into_well_tpb3(dataset, wellname):
    def clusters_rectangle(data, i, color):
        # cluster_xy = data['depth'].iloc[i-2]
        cluster_xy = data['depth'].iloc[i-1]
        # cluster_h = data['depth'].iloc[i+1] - data['depth'].iloc[i-2]
        cluster_h = data['depth'].iloc[i] - data['depth'].iloc[i-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax.add_patch(rectangle)
    data = dataset[dataset.well == wellname]
    top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':data['cluster'].iloc[0],'depth':[0]})
    data = pd.concat([top, data])
    # print(data)
    fig, ax = plt.subplots(figsize=(2,4))
    ax.plot(data['phit'], data['depth'], drawstyle='steps-post', color='black', lw=0.75, alpha=0.75)
    ax.set_xlim(0, 0.35)
    ax.invert_yaxis()
    ax.set_title(wellname)
    ax.grid()
    for i in range(len(data)):
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 0:
            clusters_rectangle(data, i, 'blue')
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 1:
            clusters_rectangle(data, i, 'green')
        if data['phit'].iloc[i] > 0 and data['cluster'].iloc[i] == 2:
            clusters_rectangle(data, i, 'red')
clusters_display_into_well_tpb3(tpb_gmm_v3, 'C16')

In [ ]:
def coloring_clusters_matrix_tpb3(dataset, letters_list, rows, columns, clustering, output_flag):
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                            clusters_rectangle(welldata, k, 'blue')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                            clusters_rectangle(welldata, k, 'red')
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\clustering_wells_tpb\\' + clustering + '_' + str(letter) +'.png')
        else:
            pass
# ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
coloring_clusters_matrix_tpb3(tpb_gmm_v3, ['A'], 4, 9, 'kmeans3_tpb', 'dontprint')

In [ ]:
dataset = tpb_gmm_v3
phit0 = dataset[dataset.cluster==0][['phit', 'cluster']]
phit1 = dataset[dataset.cluster==1][['phit', 'cluster']]
phit2 = dataset[dataset.cluster==2][['phit', 'cluster']]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
phit_tpb3 = pd.concat([phit0, phit1, phit2])
ax = sns.histplot(data=phit_tpb3, x='phit', hue='cluster', bins=35, kde=True, palette=custom_palette)
ax.grid(True, axis='x')
ax.set_xticks(np.arange(0.12, 0.32, 0.02));

In [ ]:
dataset = tpb_gmm_v3
ntg0 = dataset[dataset.cluster==0][['ntg', 'cluster']]
ntg1 = dataset[dataset.cluster==1][['ntg', 'cluster']]
ntg2 = dataset[dataset.cluster==2][['ntg', 'cluster']]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
ntg_tpb_v3 = pd.concat([ntg0, ntg1, ntg2])
ax = sns.histplot(data=ntg_tpb_v3, x='ntg', hue='cluster', bins=35, kde=True, palette=custom_palette)
ax.grid(True, axis='x')

In [ ]:
dataset = tpb_gmm_v3
htst0 = dataset[dataset.cluster==0]
htst1 = dataset[dataset.cluster==1]
htst2 = dataset[dataset.cluster==2]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
htst_tpb_v3 = pd.concat([htst0, htst1, htst2])
ax = sns.histplot(data=htst_tpb_v3, x='htst', hue='cluster', bins=30, kde=True, palette=custom_palette)
ax.grid(True, axis='x')
# ax.set_xlim(0,30);
ax.set_xticks(np.arange(0, 60, 5));

In [ ]:
dataset = tpb_gmm_v3
vsh0 = dataset[dataset.cluster==0]
vsh1 = dataset[dataset.cluster==1]
vsh2 = dataset[dataset.cluster==2]
custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
vsh_tpb_v3 = pd.concat([vsh0, vsh1, vsh2])
ax = sns.histplot(data=vsh_tpb_v3, x='vsh', hue='cluster', bins=50, kde=True, palette=custom_palette)
ax.grid(True, axis='x');

#### Gmm map

In [ ]:
def cluster_data_to_map(dataset, dataset_coord, comment):
    def dataset_groupby(dataset):
        result = dataset.groupby(['well','cluster'])['htst'].sum().reset_index()
        return result
    tpb_test_v3_piechart = dataset_groupby(dataset)

    def cluster_transpose(dataset, wellname):
        result = dataset[dataset.well==wellname]
        result.loc[result.cluster == 0, 'cluster_0'] = result.htst
        result.loc[result.cluster == 1, 'cluster_1'] = result.htst
        result.loc[result.cluster == 2, 'cluster_2'] = result.htst
        result.fillna(0)
        result = result.groupby('well').sum().reset_index()
        result = result[['well', 'cluster_0', 'cluster_1','cluster_2']]
        return result
    df_lst = []
    for wellname in tpb_test_v3_piechart.well.unique():
        df = cluster_transpose(tpb_test_v3_piechart, wellname)
        df_lst.append(df)
    data_transpose = pd.concat(df_lst).reset_index(drop=True)

    def coordinates_calc(dataset_coord, fm):
        dataset_coord = dataset_coord[dataset_coord.FORMATION_up == fm]
        result = dataset_coord.groupby('well')[['X_mean','Y_mean']].apply(lambda x: x.iloc[0]).reset_index()
        return result 
    coord = coordinates_calc(dataset_coord, 'Balakhany VIII')
    data_transpose_coord = data_transpose.set_index('well').join(coord.set_index('well'), rsuffix='_coord').reset_index()

    def piechart_map(dataset_map):
        fig, ax = plt.subplots(figsize=(13,13))
        for ind, row in dataset_map.iterrows():
                ax.pie([row['cluster_0'], row['cluster_1'], row['cluster_2']], 
                        radius=0.3, center=(row['X_mean']/1000, row['Y_mean']/1000), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                        colors=['blue', 'green', 'red'], frame=True)
        plt.grid()
        plt.title(comment)
    piechart_map(data_transpose_coord)

    return data_transpose_coord

cluster_gmm = cluster_data_to_map(tpb_gmm_v3, df_bal_net2_kh, 'Gmm')

#### Experiments with prediction

In [ ]:
def clustering_data_calculation(dataset):
    """
    """
    df_net2_bal8 = dataset[[    'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal8 = df_net2_bal8[    (df_net2_bal8.well.isin(well_clean8_v2)) & 
                                    (df_net2_bal8.FORMATION_up=='Balakhany VIII')]
    df_net2_bal10 = dataset[[   'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal10 = df_net2_bal10[  (df_net2_bal10.well.isin(well_clean10_v2)) & 
                                    (df_net2_bal10.FORMATION_up=='Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2')
    def ntd_calculation_brief(dataset,well,desired_fm, net_var):
        data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list8 = []
    for well in tqdm(df_net2_bal8.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8.append(df)
    ntd_net2_8 = pd.concat(df_recalc_list8)
    ntd_net2_8.drop_duplicates(inplace=True)
    df_recalc_list10 = []
    for well in tqdm(df_net2_bal10.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10.append(df)
    ntd_net2_10 = pd.concat(df_recalc_list10)
    ntd_net2_10.drop_duplicates(inplace=True)

    print('Calculation values for NTD Bal8 and Bal10')
    def ntd_properties_dataframe(dataset_ntd, dataset_logs, fmname):
        well_data = []
        well_formation = fmname
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()[:]):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_gperm = []
            well_top = []
            well_bot = []
            well_h = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_gperm.append(gmean(perm_lst))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val_bal8 = ntd_properties_dataframe(ntd_net2_8, df_net2_bal8, 'Balakhany VIII')
    ntd_val_bal10 = ntd_properties_dataframe(ntd_net2_10, df_net2_bal10, 'Balakhany X')
    ntd_val_final = pd.concat([ntd_val_bal8, ntd_val_bal10])
    return ntd_val_final
# ntd_val_final = clustering_data_calculation(df_bal_net2_kh)
# ntd_val_final8 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany VIII']
# ntd_val_final10 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany X']

def nothtst_nan_fill(dataset_ntd, fmname):
    def nan_change_diff_fmbottom(dataset, wellname, fmname):
        row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
        row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
        return row_change
    df_list = []
    for wellname in dataset_ntd.well.unique():
        df = nan_change_diff_fmbottom(dataset_ntd, wellname, fmname)
        df_list.append(df)
    res_df_list = pd.concat(df_list)
    result = pd.concat([dataset_ntd, res_df_list])
    result = result.sort_values(by=['well','top_tst'])
    result_final = result.dropna(subset=['not_htst'], axis=0)
    return result_final
# ntd_val_final8_clean = nothtst_nan_fill(ntd_val_final8, 'Balakhany VIII')
# ntd_val_final10_clean = nothtst_nan_fill(ntd_val_final10, 'Balakhany X')

def top_phit_bot_clustering(dataset, print_flag, path):
    def top_phit_bot_collection_run(dataset):
        def top_phit_bot_collection(dataset, wellname):
            data = dataset[dataset.well == wellname]
            data['top_htst'] = data['top_tst'] - data['fm_top_tst']
            data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
            data['bot_htst'] = data['not_htst']
            data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 
                         'top_htst','htst','bot_htst', 'fm_top_tst', 'fm_bot_tst']]
            return data
        df_lst = []
        for wellname in tqdm(dataset.well.unique()):
            res_df = top_phit_bot_collection(dataset, wellname)
            df_lst.append(res_df)
        top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
        return top_phi_bot_cluster
    top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

    def top_phit_bot_ntg_run(dataset):
        def top_phit_bot_ntg(dataset, wellname):
            ntg = []
            data = dataset[dataset.well == wellname].reset_index(drop=True)
            for ind, row in data.iterrows():
                if ind == 0:
                    ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                if ind != 0:
                    ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                if ind == len(data):
                    ntg.append(row['htst']/(row['top_htst'] + row['htst']))
            result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
            return result
        df_lst = []
        for wellname in dataset.well.unique():
            df = top_phit_bot_ntg(dataset, wellname)
            df_lst.append(df)
        top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
        return top_phi_bot_cluster_ntg
    top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)

    if print_flag == 'print':
        top_phi_bot_cluster_ntg.to_csv(path, index=False)
    else:
        pass
    return top_phi_bot_cluster_ntg
# top_phi_bot_cluster8 = top_phit_bot_clustering(ntd_val_final8_clean, 'dont_print', '.\output\\top_phi_bot_cluster8.csv')
# top_phi_bot_cluster10 = top_phit_bot_clustering(ntd_val_final10_clean, 'dont_print', '.\output\\top_phi_bot_cluster8.csv')

cluster_top_phi_bot8_v2 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\top_phi_bot_cluster_ntg8_with_labels.csv').drop(['Unnamed: 0.1','Unnamed: 0'], axis=1)
cluster_top_phi_bot8_v3 = cluster_top_phi_bot8_v2.join(top_phi_bot_cluster8[['top_htst', 'bot_htst']])

def dataset_for_spatial_prediction(dataset_full, dataset_cluster, offset_qty, cluster_algo, cluster_list):
    
    def joining_coordinates(dataset_full, dataset_cluster, cluster_algo, cluster_list):
        coordinates = dataset_full.groupby(['well','FORMATION_up'])[['X_mean','Y_mean']].apply(lambda x: x.iloc[0]).reset_index()
        dataset_cluster = dataset_cluster[(dataset_cluster[cluster_algo].isin(cluster_list))]
        result = dataset_cluster.set_index(['well','FORMATION_up']).join(coordinates.set_index(['well','FORMATION_up'])).reset_index()
        coordinates = result[['well','FORMATION_up', 'X_mean', 'Y_mean']].groupby(['well','FORMATION_up']).apply(lambda x: x.iloc[0]).reset_index(drop=True)
        return coordinates, result
    coordinates, dataset_cluster_xy = joining_coordinates(dataset_full, dataset_cluster, cluster_algo, cluster_list)
    coordinates = coordinates[~coordinates.well.isin(['A14Y'])]

    def well_distance_calculation(coordinates, fm):
        coordinates_fm = coordinates[coordinates.FORMATION_up == fm]
        df_distance_fm = pd.DataFrame(euclidean_distances(coordinates_fm[['X_mean', 'Y_mean']]), columns=list(coordinates_fm.well))
        well_name_rows = coordinates_fm.well.reset_index().drop(['index'], axis=1)
        result = df_distance_fm.join(well_name_rows).set_index('well').reset_index()
        return result
    well_dist_crosstable_8 = well_distance_calculation(coordinates, 'Balakhany VIII')

    def offset_well_names_dist(dataset, offset_qty):
        df_lst = []
        for ind in range(len(dataset.well.unique())):
            off_well_series = dataset.iloc[ind]
            off_well_selected = pd.DataFrame(off_well_series)[1:].sort_values(by=ind)[:offset_qty+1].T
            off_well_selected['well'] = off_well_selected.columns[0]
            off_well_selected = off_well_selected.drop(columns= off_well_selected.well, axis=1)

            dist_titles = ['dist_' + str(num+1) for num in range(offset_qty)]
            well_titles = ['well_' + str(num+1) for num in range(offset_qty)]

            col_names = []
            for i in range(len(off_well_selected.columns[:-1])):
                col = off_well_selected.columns[i]
                col_names.append(col)
                off_well_selected = off_well_selected.rename(columns={col:dist_titles[i]})

            off_well_names = pd.DataFrame(col_names).T
            col_names = []
            for i in range(len(off_well_names.columns)):
                col = off_well_names.columns[i]
                col_names.append(col)
                off_well_names = off_well_names.rename(columns={col:well_titles[i]})
            
            concat_well_data = pd.concat([off_well_names.reset_index(drop=True), off_well_selected.reset_index(drop=True)], axis=1)
            df_lst.append(concat_well_data)
        result = pd.concat(df_lst).reset_index(drop=True)
        return result
    well_dist_data8 = offset_well_names_dist(well_dist_crosstable_8, offset_qty)

    def offset_wells_features_calculation(dataset_dist, dataset_clusters, cluster_algo, cluster_list, fm):
        df_lst = []
        for wellname in dataset_dist.well.unique():
            data = dataset_dist[dataset_dist.well == wellname]
            cc = 0
            for j in data.columns:
                if 'well_' in j:
                    cc += 1
                    offset_wellname = data[j].values[0]
                    data_cluster = dataset_clusters[(dataset_clusters.well == offset_wellname) & 
                                                        (dataset_clusters[cluster_algo].isin(cluster_list))]
                    var_name = 'phit_wavg_' + str(cc)
                    data[var_name] = ((data_cluster['phit_avg'] * data_cluster['htst']).sum()) / (data_cluster['htst'].sum())
                    var_name = 'vsh_wavg_' + str(cc)
                    data[var_name] = ((data_cluster['vsh_avg'] * data_cluster['htst']).sum()) / (data_cluster['htst'].sum())
                    var_name = 'htst_sum_' + str(cc)
                    data[var_name] = data_cluster['htst'].sum()                
            df_lst.append(data)
        result = pd.concat(df_lst).reset_index(drop=True)
        result['FORMATION_up'] = fm
        return result
    well_features8 = offset_wells_features_calculation(well_dist_data8, cluster_top_phi_bot8_v3, cluster_algo, cluster_list, 'Balakhany VIII')

    def target_wells_variable_calculation(dataset_dist, dataset_clusters, cluster_algo, cluster_list, fm):
        df_lst = []
        for wellname in dataset_dist.well.unique():
            df = pd.DataFrame({'well': [wellname], 'FORMATION_up': [fm], 'phit_wavg_target': [0]})
            data = dataset_clusters[(dataset_clusters.well == wellname) & 
                                    (dataset_clusters[cluster_algo].isin(cluster_list))]
            df['phit_wavg_target'] = ((data['phit_avg'] * data['htst']).sum()) / (data['htst'].sum())
            df_lst.append(df)
        result = pd.concat(df_lst).reset_index(drop=True)
        return result
    well_target8 = target_wells_variable_calculation(well_dist_data8, cluster_top_phi_bot8_v3, cluster_algo, cluster_list, 'Balakhany VIII')
    
    dataset8 = well_target8.set_index(['well','FORMATION_up']).join(well_features8.set_index(['well','FORMATION_up'])).reset_index()

    result = {'dataset8':dataset8, 'cluster_xy':dataset_cluster_xy, 'well_dist8':well_dist_data8, 'coordinates':coordinates,
              'target8':well_target8, 'feature8':well_features8, 'dist_crosstable8':well_dist_crosstable_8}
    return result

dataset8 = dataset_for_spatial_prediction(df_bal_net2_kh, cluster_top_phi_bot8_v3, 3, 'kmeans_3_labels', [0,1,2])['dataset8']
dataset8.head(3)

In [ ]:
## Archiv models
# model3 = model_preiction_split(dataset8, SVR(), 0.05, 'dont_display')
# model6 = model_preiction_split(dataset8, SGDRegressor(random_state=42), 0.05, 'dont_display')
# model7 = model_preiction_split(dataset8, GaussianProcessRegressor(random_state=42), 0.05, 'dont_display')
# model8 = model_preiction_split(dataset8, DecisionTreeRegressor(random_state=42), 0.05, 'dont_display')
# model9 = model_preiction_split(dataset8, GradientBoostingRegressor(random_state=42), 0.05, 'dont_display')
# model11 = model_preiction_split(dataset8, MLPRegressor(random_state=1, max_iter=1000), 0.05, 'dont_display')

In [ ]:
def model_preiction_split(dataset, selected_model, tolerance, display_flag='display'):
    drop_lst_X = ['well','FORMATION_up', 'well_1', 'well_2', 'well_3']
    drop_lst_y = ['well','FORMATION_up']

    X = dataset.drop('phit_wavg_target', axis=1)
    y = dataset[['well','FORMATION_up', 'phit_wavg_target']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

    y_train_wnames = y_train[['well','FORMATION_up']].reset_index(drop=True)
    y_test_wnames = y_test[['well','FORMATION_up']].reset_index(drop=True)

    X_train = X_train.drop(drop_lst_X, axis=1)
    X_test = X_test.drop(drop_lst_X, axis=1)
    y_train = y_train.drop(drop_lst_y, axis=1)
    y_test = y_test.drop(drop_lst_y, axis=1)

    model = Pipeline([("scaler",StandardScaler()),("model", selected_model)])
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_train = np.array(y_train).flatten()
    y_test = np.array(y_test).flatten()
    train = pd.DataFrame(zip(y_train,y_pred_train), columns=['y_orig', 'y_pred'])
    train = pd.concat([y_train_wnames, train], axis=1)
    test = pd.DataFrame(zip(y_test,y_pred_test), columns=['y_orig', 'y_pred'])
    test = pd.concat([y_test_wnames, test], axis=1)

    train['up'] = train['y_orig']*(1 + tolerance)
    train['down'] = train['y_orig']*(1 - tolerance)
    train['qc'] = 'out'
    train['dataset'] = 'train'
    train.loc[(train['y_pred'] <= train.up) & (train['y_pred'] >= train.down), 'qc'] = 'in'
    trainqc = train.qc.value_counts(normalize=True)

    test['up'] = test['y_orig']*(1 + tolerance)
    test['down'] = test['y_orig']*(1 - tolerance)
    test['qc'] = 'out'
    test['dataset'] = 'test'
    test.loc[(test['y_pred'] <= test.up) & (test['y_pred'] >= test.down), 'qc'] = 'in'
    testqc = test.qc.value_counts(normalize=True)
    df = pd.concat([train, test])
    df['y_pred'] = df['y_pred'].astype('float')

    result = {'result':df, 'testqc':testqc, 'trainqc':trainqc, 'train_df':X_train.columns}
    if display_flag == 'display':
        print('model:\n', model.named_steps['model'])
        print('features dataset: \n', list(X_train.columns))
        print(f'result test:\n with tolerance {tolerance} "in":', result['testqc']['in'].round(2))
    else:
        print('test "in":', result['testqc']['in'].round(2), model.named_steps['model'])
    return result

dataset8 = dataset_for_spatial_prediction(df_bal_net2_kh, cluster_top_phi_bot8_v3, 3, 'kmeans_3_labels', [0,1,2])['dataset8']
tolerance = 0.05
model1 = model_preiction_split(dataset8, RandomForestRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model2 = model_preiction_split(dataset8, BayesianRidge(), tolerance, 'dont_display')
model4 = model_preiction_split(dataset8, XGBRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model5 = model_preiction_split(dataset8, CatBoostRegressor(random_state=42, verbose=False), tolerance, 'dont_display')
model10 = model_preiction_split(dataset8, AdaBoostRegressor(random_state=42), tolerance, 'dont_display')

In [ ]:
model1['train_df']

In [ ]:
def xplot_qc2(data, y_orig, y_pred, max_val, rng, margin, round, comment):
    data = data.round({y_orig: round, y_pred: round})
    ds_train = data[data.dataset == 'train']
    ds_test = data[data.dataset == 'test']
    up_range = rng + 1
    dwn_range = 1 - rng
    colors = {'in': 'green', 'out': 'red'}
    qc_colors_tr = [colors[qc] for qc in ds_train.qc]
    qc_colors_ts = [colors[qc] for qc in ds_test.qc]
    scatter_train = go.Scatter( x=ds_train['y_orig'], y=ds_train['y_pred'],
                                mode='markers',
                                marker=dict(color=qc_colors_tr, size=6, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                customdata = ds_train[['well',y_orig, y_pred, 'FORMATION_up']],
                                hovertemplate="".join(
                                ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]}, f:%{customdata[3]}<extra></extra>"])
                                )
    scatter_test = go.Scatter(  x=ds_test[y_orig], y=ds_test[y_pred], 
                                mode='markers',
                                marker=dict(color=qc_colors_ts, size=6, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                customdata = data[['well', y_orig, y_pred, 'FORMATION_up']],
                                hovertemplate="".join(
                                ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]}, f:%{customdata[3]}<extra></extra>"])
                                )
    line_trace_up = go.Scatter(x=[0, max_val], y=[0 + margin, max_val*up_range + margin], mode='lines+markers', line=dict(color='blue'))
    line_trace_dw = go.Scatter(x=[0, max_val], y=[0 - margin, max_val*dwn_range - margin], mode='lines+markers', marker=dict(color='blue'))
    fig = make_subplots(rows=1, cols=2, subplot_titles=('train ds', 'test ds'))
    fig.add_trace(scatter_train,  row=1, col=1)
    fig.add_trace(line_trace_up,  row=1, col=1)
    fig.add_trace(line_trace_dw,  row=1, col=1)
    fig.update_xaxes(title_text='actual', row=1, col=1)
    fig.update_yaxes(title_text='predict', row=1, col=1)
    fig.add_trace(scatter_test,  row=1, col=2)
    fig.add_trace(line_trace_up,  row=1, col=2)
    fig.add_trace(line_trace_dw,  row=1, col=2)
    fig.update_xaxes(title_text='actual', row=1, col=2)
    fig.update_yaxes(title_text='predict', row=1, col=2)
    fig.update_layout(  title_text= (comment), width=700, height=350, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()

dataset8 = dataset_for_spatial_prediction(df_bal_net2_kh, cluster_top_phi_bot8_v3, 3, 'kmeans_3_labels', [0,1,2])['dataset8']
tolerance = 0.05
model1 = model_preiction_split(dataset8, RandomForestRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model2 = model_preiction_split(dataset8, BayesianRidge(), tolerance, 'dont_display')
model4 = model_preiction_split(dataset8, XGBRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model5 = model_preiction_split(dataset8, CatBoostRegressor(random_state=42, verbose=False), tolerance, 'dont_display')
model10 = model_preiction_split(dataset8, AdaBoostRegressor(random_state=42), tolerance, 'dont_display')

xplot_qc2(model1['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'RandomForestRegressor [0,1,2]')
xplot_qc2(model2['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'BayesianRidge [0,1,2]')
xplot_qc2(model4['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'XGBRegressor [0,1,2]')
xplot_qc2(model5['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'CatBoostRegressor [0,1,2]')
xplot_qc2(model10['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'AdaBoostRegressor [0,1,2]')

In [ ]:
dataset8 = dataset_for_spatial_prediction(df_bal_net2_kh, cluster_top_phi_bot8_v3, 3, 'kmeans_3_labels', [0])['dataset8']
tolerance = 0.05
model1 = model_preiction_split(dataset8, RandomForestRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model2 = model_preiction_split(dataset8, BayesianRidge(), tolerance, 'dont_display')
model4 = model_preiction_split(dataset8, XGBRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model5 = model_preiction_split(dataset8, CatBoostRegressor(random_state=42, verbose=False), tolerance, 'dont_display')
model10 = model_preiction_split(dataset8, AdaBoostRegressor(random_state=42), tolerance, 'dont_display')

xplot_qc2(model1['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'RandomForestRegressor [0]')
xplot_qc2(model2['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'BayesianRidge [0]')
xplot_qc2(model4['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'XGBRegressor [0]')
xplot_qc2(model5['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'CatBoostRegressor [0]')
xplot_qc2(model10['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'AdaBoostRegressor [0]')

In [ ]:
dataset8 = dataset_for_spatial_prediction(df_bal_net2_kh, cluster_top_phi_bot8_v3, 3, 'kmeans_3_labels', [1])['dataset8']
tolerance = 0.05
model1 = model_preiction_split(dataset8, RandomForestRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model2 = model_preiction_split(dataset8, BayesianRidge(), tolerance, 'dont_display')
model4 = model_preiction_split(dataset8, XGBRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model5 = model_preiction_split(dataset8, CatBoostRegressor(random_state=42, verbose=False), tolerance, 'dont_display')
model10 = model_preiction_split(dataset8, AdaBoostRegressor(random_state=42), tolerance, 'dont_display')

xplot_qc2(model1['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'RandomForestRegressor [1]')
xplot_qc2(model2['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'BayesianRidge [1]')
xplot_qc2(model4['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'XGBRegressor [1]')
xplot_qc2(model5['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'CatBoostRegressor [1]')
xplot_qc2(model10['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'AdaBoostRegressor [1]')

In [ ]:
dataset8 = dataset_for_spatial_prediction(df_bal_net2_kh, cluster_top_phi_bot8_v3, 3, 'kmeans_3_labels', [2])['dataset8']
tolerance = 0.05
model1 = model_preiction_split(dataset8, RandomForestRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model2 = model_preiction_split(dataset8, BayesianRidge(), tolerance, 'dont_display')
model4 = model_preiction_split(dataset8, XGBRegressor(n_jobs=-1, random_state=42), tolerance, 'dont_display')
model5 = model_preiction_split(dataset8, CatBoostRegressor(random_state=42, verbose=False), tolerance, 'dont_display')
model10 = model_preiction_split(dataset8, AdaBoostRegressor(random_state=42), tolerance, 'dont_display')

xplot_qc2(model1['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'RandomForestRegressor [2]')
xplot_qc2(model2['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'BayesianRidge [2]')
xplot_qc2(model4['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'XGBRegressor [2]')
xplot_qc2(model5['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'CatBoostRegressor [2]')
xplot_qc2(model10['result'], 'y_orig', 'y_pred', 0.3, tolerance, 0, 3, 'AdaBoostRegressor [2]')

### Clustering top_phi_bot cross-clustering

In [ ]:
ntd_val_clustering_top_phi_bot = pd.read_csv('.\inputoutput\\ntd_val_clustering_one_layer_with_labels.csv').drop('Unnamed: 0', axis=1)
tpb_crosscluster = ntd_val_clustering_top_phi_bot.copy()
ntd_val_clustering_top_phi_bot.columns

In [ ]:
tpb_crosscluster.head(3)

In [87]:
def well_collecting_clusters_top_phi_bot(dataset, clustering):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        topbot_htst_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

            topbot_htst_lst.append(0)
            topbot_htst_lst.append((row['top_htst'] + row['bot_htst']))

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, topbot_htst_lst, cluster_lst ), columns=['well','phit', 'htst', 'topbot_htst','cluster'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
tpb_crosscluster_kmeans = well_collecting_clusters_top_phi_bot(tpb_crosscluster, 'kmeans_3_labels')
tpb_crosscluster_gmm = well_collecting_clusters_top_phi_bot(tpb_crosscluster, 'gmm_3_labels')

tpb_crosscluster_kmeans = tpb_crosscluster_kmeans.rename(columns = {'cluster':'cluster_kmeans'}).reset_index(drop=True)
tpb_crosscluster_gmm = tpb_crosscluster_gmm.rename(columns = {'cluster':'cluster_gmm'}).reset_index(drop=True)
tpb_crosscluster_final = tpb_crosscluster_kmeans.join(tpb_crosscluster_gmm['cluster_gmm'])
tpb_crosscluster_final = tpb_crosscluster_final[['well', 'phit', 'htst', 'topbot_htst', 'depth', 'cluster_kmeans', 'cluster_gmm']]

In [ ]:
tpb_crosscluster_final.head(3)

In [ ]:
blue_big = tpb_crosscluster_final[  (tpb_crosscluster_final.cluster_kmeans == 0) & 
                                    (tpb_crosscluster_final.cluster_gmm == 0) &
                                    (tpb_crosscluster_final.htst >= 12.5) &
                                    (tpb_crosscluster_final.phit >=0.21)]
green = tpb_crosscluster_final[     (tpb_crosscluster_final.cluster_gmm == 1) &
                                    (tpb_crosscluster_final.topbot_htst >=7.5)]
red = tpb_crosscluster_final[       (tpb_crosscluster_final.cluster_kmeans == 2) &
                                    (tpb_crosscluster_final.topbot_htst < 7.5) &
                                    (tpb_crosscluster_final.phit < 0.21)]
all = tpb_crosscluster_final[tpb_crosscluster_final.phit != 0]
plt.scatter(blue_big['phit'], blue_big['htst'], alpha=0.75, ec='black', zorder=4)
plt.scatter(green['phit'], green['htst'], c='green', alpha=0.75, ec='black',zorder=3)
plt.scatter(red['phit'], red['htst'], c='red', alpha=0.75, ec='black',zorder=2)
plt.scatter(all['phit'], all['htst'], c='gray', alpha=0.1, ec='black', zorder=1)

In [ ]:
tpb_crosscluster_final['cluster'] = np.nan
tpb_crosscluster_final.loc[ ((tpb_crosscluster_final.cluster_kmeans == 0) & 
                            (tpb_crosscluster_final.cluster_gmm == 0) &
                            (tpb_crosscluster_final.htst >= 12.5) &
                            (tpb_crosscluster_final.phit >=0.21)), 'cluster'] = 0
tpb_crosscluster_final.loc[ ((tpb_crosscluster_final.cluster_kmeans == 1) & 
                            (tpb_crosscluster_final.cluster_gmm == 1) &
                            (tpb_crosscluster_final.htst >= 7.5)), 'cluster'] = 1
tpb_crosscluster_final.loc[ ((tpb_crosscluster_final.cluster_kmeans == 2) & 
                            (tpb_crosscluster_final.topbot_htst < 7.5) &
                            (tpb_crosscluster_final.phit < 0.21)), 'cluster'] = 2
tpb_crosscluster_final[tpb_crosscluster_final.well == 'A01W']

In [ ]:
def coloring_clusters_matrix_tpb3(dataset, letters_list, rows, columns, clustering, output_flag):
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                            clusters_rectangle(welldata, k, 'blue')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                            clusters_rectangle(welldata, k, 'red')
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\clustering_wells_tpb\\' + clustering + '_' + str(letter) +'.png')
        else:
            pass
coloring_clusters_matrix_tpb3(tpb_crosscluster_final, ['A','B','C'], 4, 9, 'cluster_cross', 'dontprint')

### Clustering whole well

In [92]:
# Uploading clustering data whole well from Farid
ntd_val_clust_wholewell_init = pd.read_csv('.\inputoutput\\ntd_val_clustering_wholewell_stride_with_labels.csv').drop('Unnamed: 0', axis=1)
# ntd_val_clust_wholewell_init.columns

In [ ]:
ntd_val_clust_wholewell = ntd_val_clust_wholewell_init[['phit_1', 'phit_2', 'phit_3', 'phit_4', 'phit_5', 'phit_6', 'phit_7',
                                                        'phit_8', 'phit_9', 'phit_10', 'phit_11', 'phit_12', 'phit_13',
                                                        'phit_14', 'htst_1', 'htst_2', 'htst_3', 'htst_4', 'htst_5', 'htst_6',
                                                        'htst_7', 'htst_8', 'htst_9', 'htst_10', 'htst_11', 'htst_12',
                                                        'htst_13', 'htst_14', 'not_htst_1', 'not_htst_2', 'not_htst_3',
                                                        'not_htst_4', 'not_htst_5', 'not_htst_6', 'not_htst_7', 'not_htst_8',
                                                        'not_htst_9', 'not_htst_10', 'not_htst_11', 'not_htst_12',
                                                        'not_htst_13', 'not_htst_14', 'well', 'FORMATION_up', 'kmeans_3_labels',
                                                        'kmeans_4_labels', 'kmeans_5_labels']]
xy_bal8_init = df_bal_net2_kh[df_bal_net2_kh.FORMATION_up=='Balakhany VIII'][['well','FORMATION_up','X_mean','Y_mean']]
xy_bal8 = xy_bal8_init.groupby(['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
ntd_val_clust_wholewell_xy = ntd_val_clust_wholewell.set_index(['well','FORMATION_up']).join(xy_bal8.set_index(['well','FORMATION_up'])).reset_index()
ntd_val_clust_wholewell_xy

In [ ]:
fig, ax = plt.subplots(figsize=(13,10))
data = ntd_val_clust_wholewell_xy
color_dict = {0: 'blue', 1: 'green', 2: 'red'}
colors = [color_dict[val] for val in data['kmeans_3_labels']]
ax.scatter(data['X_mean'], data['Y_mean'], c=colors, marker='o', s=150, edgecolors='gray', alpha=0.75);

## Prediction based on layering plot

### Data preparation for prediction

In [ ]:
dataset = df_bal_net2_kh
def clustering_data_calculation(dataset):
    """
    """
    df_net2_bal8 = dataset[[    'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal8 = df_net2_bal8[    (df_net2_bal8.well.isin(well_clean8_v2)) & 
                                    (df_net2_bal8.FORMATION_up=='Balakhany VIII')]
    df_net2_bal10 = dataset[[   'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X_mean','Y_mean','field']]
    df_net2_bal10 = df_net2_bal10[  (df_net2_bal10.well.isin(well_clean10_v2)) & 
                                    (df_net2_bal10.FORMATION_up=='Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2')
    def ntd_calculation_brief(dataset,well,desired_fm, net_var):
        data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list8 = []
    for well in tqdm(df_net2_bal8.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8.append(df)
    ntd_net2_8 = pd.concat(df_recalc_list8)
    ntd_net2_8.drop_duplicates(inplace=True)
    df_recalc_list10 = []
    for well in tqdm(df_net2_bal10.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10.append(df)
    ntd_net2_10 = pd.concat(df_recalc_list10)
    ntd_net2_10.drop_duplicates(inplace=True)

    print('Calculation values for NTD Bal8 and Bal10')
    def ntd_properties_dataframe(dataset_ntd, dataset_logs, fmname):
        well_data = []
        well_formation = fmname
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()[:]):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_gperm = []
            well_top = []
            well_bot = []
            well_h = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_gperm.append(gmean(perm_lst))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val_bal8 = ntd_properties_dataframe(ntd_net2_8, df_net2_bal8, 'Balakhany VIII')
    ntd_val_bal10 = ntd_properties_dataframe(ntd_net2_10, df_net2_bal10, 'Balakhany X')
    ntd_val_final = pd.concat([ntd_val_bal8, ntd_val_bal10])
    return ntd_val_final
ntd_val_final = clustering_data_calculation(df_bal_net2_kh)
ntd_val_final8 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany VIII']
ntd_val_final10 = ntd_val_final[ntd_val_final.FORMATION_up == 'Balakhany X']

def nothtst_nan_fill(dataset_ntd, fmname):
    def nan_change_diff_fmbottom(dataset, wellname, fmname):
        row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
        row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
        return row_change
    df_list = []
    for wellname in dataset_ntd.well.unique():
        df = nan_change_diff_fmbottom(dataset_ntd, wellname, fmname)
        df_list.append(df)
    res_df_list = pd.concat(df_list)
    result = pd.concat([dataset_ntd, res_df_list])
    result = result.sort_values(by=['well','top_tst'])
    result_final = result.dropna(subset=['not_htst'], axis=0)
    return result_final
ntd_val_final8_clean = nothtst_nan_fill(ntd_val_final8, 'Balakhany VIII')
ntd_val_final10_clean = nothtst_nan_fill(ntd_val_final10, 'Balakhany X')

def top_phit_bot_clustering(dataset, wellname):
    data = dataset[dataset.well == wellname]
    data['top_htst'] = data['top_tst'] - data['fm_top_tst']
    data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
    data['bot_htst'] = data['not_htst']
    data['depth'] = data['top_htst'] + data['htst']/2
    data['depth'] = data['depth'].cumsum()
    data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'top_htst','htst','bot_htst', 'depth']]
    return data
df_lst = []
for wellname in tqdm(ntd_val_final8.well.unique()):
    res_df8 = top_phit_bot_clustering(ntd_val_final8_clean, wellname)
    df_lst.append(res_df8)
top_phi_bot_cluster = pd.concat(df_lst).reset_index().drop('index', axis=1)
top_phi_bot_cluster.head(3)

In [ ]:
xy_bal8_init = df_bal_net2_kh[df_bal_net2_kh.FORMATION_up=='Balakhany VIII'][['well','FORMATION_up','X_mean','Y_mean']]
xy_bal8 = xy_bal8_init.groupby(['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
top_phi_bot8_xy = top_phi_bot_cluster.set_index(['well','FORMATION_up']).join(xy_bal8.set_index(['well','FORMATION_up'])).reset_index()
top_phi_bot8_xy.head(3)

### Prediction - base line

In [ ]:
def join_df_prediction(base_dataframe, target_var):
    def columns_reorder(dataset, selected_column):
        new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
        dataset = dataset[new_order]
        return dataset
    def cat_finder(dataset):
        """
        cat_list: categorical columns to drop out
        get_dum_list: categorical columns to run via pd.get_dummies
        """
        cat_list = []
        gm_list = []
        for col in dataset.columns:
            if dataset[col].dtype == 'string' or dataset[col].dtype == 'object':
                cat_list.append(col)
                if col != 'well':
                    gm_list.append(col)
        return cat_list, gm_list
    col_names, gm_list = cat_finder(base_dataframe)
    df_corr = base_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(base_dataframe[gm_list], columns=gm_list, drop_first=True)
    dataframe = pd.concat([base_dataframe, mem_cell], axis=1)
    return df_corr, dataframe
def pairplot_special(dataset, xsize, ysize, flag=1):
    if flag == 1:
        def corrfunc(x, y, **kws):
            r, _ = stats.pearsonr(x, y)
            ax = plt.gca()
            ax.annotate("r = {:.2f}".format(r),
                        xy=(.1, .9), xycoords=ax.transAxes)
        sns.set_context(rc={'axes.labelsize':10, 'lines.linewidth': 0.75})
        g = sns.PairGrid(dataset)
        g.fig.set_size_inches(xsize,ysize)
        g.set(xticklabels=[], yticklabels=[]) 
        g.map_upper(plt.scatter, s=10, alpha=0.5)
        g.map_diag(sns.distplot, kde=False)
        g.map_lower(sns.kdeplot, cmap="Blues_d")
        g.map_lower(corrfunc)
    else:
        pass
training_data_tpb8 = top_phi_bot8_xy.copy(deep=True)
training_data_tpb8 = training_data_tpb8[[ 'well',
                                        'FORMATION_up',
                                        'X_mean', 'Y_mean',
                                        'phit_avg', 
                                        'vsh_avg',
                                        'htst',
                                        'depth',
                                        ]]
def clusters_joining(ntd_val_kmeans3, training_data_tpb):
    cluster_nums = ntd_val_kmeans3[(ntd_val_kmeans3.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
    cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
    cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
    training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
    training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
    training_data_tpb_gm = training_data_tpb_gm[training_data_tpb_gm.cluster.notna()]
    return training_data_tpb_gm
training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb8)
training_data_tpb_corr, training_data_tpb = join_df_prediction(training_data_tpb_gm, 'phit_avg')
pairplot_special(training_data_tpb_corr, 7, 7, 1)

In [98]:
training_data_tpb_notcl = training_data_tpb.drop('cluster', axis=1)

In [ ]:
# Run train-split block to get an optimized hyper parameters for 1-to-all function
gs_set = {  'bootstrap': [True, False],
            'max_depth': [10, 30, 100, 150],
            'min_samples_leaf': [2, 5, 10],
            'min_samples_split': [2, 5, 10],
            'n_estimators': [100, 150, 200]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict = run_rfr_train_test_split(training_data_tpb_notcl, gs_set, scorer, 'phit_avg', 
                                     0.05, 0, 'clusters_baseline.txt', 'clusters_baseline_grid_search', 0, 0)

In [100]:
training_data_tpb_notcl = training_data_tpb_notcl.reset_index().drop('index', axis=1)

In [ ]:
# Run 1-to-all block to to predict target vaiable
phit_pred_cluster_baseline = run_rfr_1_to_all(training_data_tpb_notcl, hyperdict, 'phit_avg', 
                                         0.05, 0, 'clusters_baseline.txt', 'clusters_baseline_1-to-all', 1, 0.3, 0)
phit_pred_cluster_baseline =  phit_pred_cluster_baseline.rename(columns={'predict':'phit_cluster_bl'})[
                                            ['well','FORMATION_up','phit_cluster_bl','qc']].sort_values(
                                            by=['well','FORMATION_up']).reset_index().drop('index', axis=1)
# Result 0.511 v/v

In [ ]:
def phit_orig_pred_bl_joining(top_phi_bot_xy, phit_pred_cluster_baseline):
    add_info1 = top_phi_bot_xy[['well','phit_avg','top_htst', 'htst', 'bot_htst']]
    add_info1['phit_avg'] = add_info1['phit_avg'].round(5)
    phit_pred_cluster_bl_collect = phit_pred_cluster_baseline.join(training_data_tpb_notcl.drop(['well','FORMATION_up','htst'],axis=1))
    phit_cluster_orig_pred = phit_pred_cluster_bl_collect.set_index(['well','phit_avg']).join(add_info1.set_index(['well','phit_avg'])).reset_index()
    return phit_cluster_orig_pred
def phit_orig_pred_bl_collecting(dataset):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        phit_pred_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            phit_pred_lst.append(np.nan)
            phit_pred_lst.append(row['phit_cluster_bl'])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

        phit_lst.append(0)
        phit_pred_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, phit_pred_lst ), 
                                            columns=['well','phit_orig', 'htst', 'phit_pred'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
phit_cluster_orig_pred = phit_orig_pred_bl_joining(top_phi_bot8_xy, phit_pred_cluster_baseline)
phit_orig_pred_cluster_bl = phit_orig_pred_bl_collecting(phit_cluster_orig_pred)

# ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
def coloring_clusters_matrix_plot(dataset, letters_list, rows, columns):
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit_orig':[0], 
                                           'htst':[0], 'phit_pred':[0], 'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit_orig'], welldata['depth'], drawstyle='steps-post', color='gray', alpha=1, lw=1.5)
                    ax[j,i].plot(welldata['phit_pred'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=2)
                    ax[j,i].set_xlim(0.1, 0.3)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)   
                    fig.tight_layout()
                    counter +=1
coloring_clusters_matrix_plot(phit_orig_pred_cluster_bl, ['A'], 4, 9)

### Prediction by clusters

#### Clusters 0,1,2 - convolution to points + offsets 

In [ ]:
def xgboost_cluster(dataset, dataset_clustering, cluster_num):
    def conv_cluster_data_to_sample(dataset, dataset_clustering, cluster_num):
        training_data_tpb8 = dataset.copy(deep=True)
        training_data_tpb8 = training_data_tpb8[[ 'well',
                                                'FORMATION_up',
                                                'X_mean', 'Y_mean',
                                                'phit_avg', 
                                                'vsh_avg',
                                                'top_htst',
                                                'htst',
                                                'bot_htst',
                                                'depth',
                                                ]]
        def clusters_joining(dataset_clustering, training_data_tpb):
            cluster_nums = dataset_clustering[(dataset_clustering.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
            cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
            cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
            training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
            training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
            return training_data_tpb_gm
        training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb8)
        training_data_tpb_gm0 = training_data_tpb_gm[training_data_tpb_gm.cluster.isin(cluster_num)]

        def closest_wells_per_cluster(training_data_tpb_gm0):
            def dist_prop_calc_xy(dataset, dist_formation, dist_cutoff, value):
                """
                dataset have to contain 'X_mean', 'Y_mean', 'TVD_SCS' and 'KHtst', if you assing value as KHtst
                """
                data = dataset[(dataset.FORMATION_up == dist_formation)]
                row_name = data.well.reset_index().drop(['index'], axis=1)
                distance_fm = pd.DataFrame(euclidean_distances(data[['X_mean', 'Y_mean']]), columns=list(data.well))
                distance_fm_well = distance_fm.join(row_name).set_index('well')
                distance_fm_well.reset_index(inplace=True)
                def well_kh_accum(wells, dataset, kh_formation):
                    well_kh_accum = []
                    well_x_accum = []
                    well_y_accum = []
                    for i in wells:
                        well_kh_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)][value].reset_index())    
                        well_x_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['X_mean'].reset_index())
                        well_y_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['Y_mean'].reset_index())
                    well_kh3 = pd.concat(well_kh_accum).T[1:]
                    well_kh3.columns = [value + '_1',value + '_2', value + '_3']
                    well_x3 = pd.concat(well_x_accum).T[1:]
                    well_x3.columns = ['x1','x2','x3']
                    well_y3 = pd.concat(well_y_accum).T[1:]
                    well_y3.columns = ['y1','y2','y3']
                    final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                                        well_x3.reset_index().drop('index',axis=1), 
                                        well_y3.reset_index().drop('index',axis=1)], axis=1)
                    return final
                df_collect = []
                for num, well_name in enumerate(distance_fm_well.well[:]):
                    well_dist3 = distance_fm_well[distance_fm_well.well == well_name].T[1:].sort_values(by=num)
                    well_dist3_s2 = well_dist3[well_dist3[num] > dist_cutoff][:3].reset_index()
                    well_dist3_tuple = tuple(well_dist3_s2['index'])
                    well_dist3_res = well_dist3_s2.T[1:].reset_index().drop('index', axis=1)   
                    well_name3_res = well_dist3_s2.T[:1].reset_index().drop('index', axis=1)
                    well_kh3_res = well_kh_accum(well_dist3_tuple,dataset, dist_formation)
                    well_dist3_res.columns =['dist1', 'dist2', 'dist3']
                    well_name3_res.columns =['well1', 'well2', 'well3']
                    concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
                    result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
                    df_collect.append(result)     
                df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
                df_well_kh_dist['FORMATION_up'] = dist_formation
                return df_well_kh_dist
            tpb8_cluster0 = training_data_tpb_gm0.groupby('well').apply(lambda x: x.iloc[0]).drop('well',axis=1).reset_index()
            list_well_offsets = dist_prop_calc_xy(tpb8_cluster0, 'Balakhany VIII', 0, 'phit_avg')[
                            ['well','FORMATION_up','well1', 'well2', 'well3','dist1', 'dist2', 'dist3']]
            return list_well_offsets
        list_wells_offsets0 = closest_wells_per_cluster(training_data_tpb_gm0)

        def loop_cluster_data_to_sample(dataset, list_wells_offsets_cluster, wellname):
            target_train = dataset[dataset.well == wellname]
            offset_wells = list_wells_offsets_cluster[list_wells_offsets_cluster.well == wellname]

            ftr1 = dataset[dataset.well == offset_wells.well1.iloc[0]]
            ftr2 = dataset[dataset.well == offset_wells.well2.iloc[0]]
            ftr3 = dataset[dataset.well == offset_wells.well3.iloc[0]]

            def conv_cluster_target_to_sample(dataset):
                def w_avg(phit, h):
                    result = phit*h
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg_target': [phit_wavg_cl]})
                return result

            target_wavg = conv_cluster_target_to_sample(target_train)

            def conv_cluster_feature_to_sample(dataset, offset_wells, well_seq, num):
                suff = '_' + str(num)
                dist = offset_wells[well_seq].iloc[0]
                def w_avg(phit, h):
                    result = phit*h
                    return result
                def summ_h(x1,x2):
                    result = x1+x2
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                dataset['vshh'] =  dataset.apply(lambda x: w_avg(x['vsh_avg'], x['htst']), axis=1)
                dataset['sumh'] =  dataset.apply(lambda x: summ_h(x['top_htst'], x['htst']), axis=1)
                vsh_wavg_cl = dataset['vshh'].sum() / dataset['htst'].sum()
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                gross = dataset['sumh'].sum() + dataset['bot_htst'].iloc[-1]
                net = dataset['htst'].sum()
                ntg = net/gross
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg' + suff: [phit_wavg_cl],
                                        'net' + suff:[net], 
                                        'gross' + suff:[gross], 
                                        'ntg' + suff:[ntg], 
                                        'vsh_wavg' + suff:[vsh_wavg_cl], 
                                        'dist' + suff:dist})
                return result

            ftr1_wavg = conv_cluster_feature_to_sample(ftr1, offset_wells, 'dist1', 1)
            ftr2_wavg = conv_cluster_feature_to_sample(ftr2, offset_wells, 'dist2', 2)
            ftr3_wavg = conv_cluster_feature_to_sample(ftr3, offset_wells, 'dist3', 3)

            return target_wavg, ftr1_wavg, ftr2_wavg, ftr3_wavg
        df_lst = []
        for wellname in training_data_tpb_gm0.well.unique():
            t, f1, f2, f3 = loop_cluster_data_to_sample(training_data_tpb_gm0, list_wells_offsets0, wellname)
            result_dataset = pd.concat([t.iloc[:,3:], f1.iloc[:,3:], f2.iloc[:,3:], f3.iloc[:,3:]], axis=1)
            df_lst.append(result_dataset)
        data_Xy = pd.concat(df_lst).reset_index(drop=True)
        return data_Xy
    data_Xy = conv_cluster_data_to_sample(dataset, dataset_clustering, cluster_num)
    X = data_Xy.iloc[:,1:]
    y = data_Xy['phit_wavg_target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

    model = XGBRegressor()

    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    train = pd.DataFrame(zip(y_train,y_pred_train), columns=['y_orig', 'y_pred'])
    test = pd.DataFrame(zip(y_test,y_pred_test), columns=['y_orig', 'y_pred'])
    tolerance = 0.05
    def test_processing(tolerance):
        test['up'] = test['y_orig']*(1 + tolerance)
        test['down'] = test['y_orig']*(1 - tolerance)
        test['qc'] = 'out'
        test.loc[(test['y_pred'] <= test.up) & (test['y_pred'] >= test.down), 'qc'] = 'in'
        return test
    test = test_processing(tolerance)
    metric = test.qc.value_counts(normalize=True)['in'].round(2)
    def train_test_display(tolerance, min_phit):
        fig, ax = plt.subplots()
        ax.scatter(train['y_orig'], train['y_pred'], label = 'train', ec='gray', alpha=0.5)
        ax.scatter(test['y_orig'], test['y_pred'], label = 'test', ec='gray', alpha=0.9)
        line = mlines.Line2D([min_phit, 0.28], [min_phit, 0.28], color='red', ls='--')
        line1 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 + tolerance), 0.28*(1 + tolerance)], color='blue', ls='--')
        line2 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 - tolerance), 0.28*((1 - tolerance))], color='blue', ls='--')
        ax.add_line(line), ax.add_line(line1), ax.add_line(line2), ax.grid(), ax.legend(), ax.set_ylabel('pred'), ax.set_xlabel('orig'),
        ax.set_xlim(0.14, 0.28), ax.set_ylim(0.14, 0.28) 
        ax.set_title(f'Cluster # {cluster_num} metric "test in": {metric}')
    train_test_display(tolerance, 0.15)
    return  data_Xy, train, test

data_Xy, train, test = xgboost_cluster(top_phi_bot8_xy, ntd_val_kmeans3, [0,1,2])

In [ ]:
def catboost_cluster(dataset, dataset_clustering, cluster_num):
    def conv_cluster_data_to_sample(dataset, dataset_clustering, cluster_num):
        training_data_tpb8 = dataset.copy(deep=True)
        training_data_tpb8 = training_data_tpb8[[ 'well',
                                                'FORMATION_up',
                                                'X_mean', 'Y_mean',
                                                'phit_avg', 
                                                'vsh_avg',
                                                'top_htst',
                                                'htst',
                                                'bot_htst',
                                                'depth',
                                                ]]
        def clusters_joining(dataset_clustering, training_data_tpb):
            cluster_nums = dataset_clustering[(dataset_clustering.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
            cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
            cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
            training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
            training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
            return training_data_tpb_gm
        training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb8)
        training_data_tpb_gm0 = training_data_tpb_gm[training_data_tpb_gm.cluster.isin(cluster_num)]

        def closest_wells_per_cluster(training_data_tpb_gm0):
            def dist_prop_calc_xy(dataset, dist_formation, dist_cutoff, value):
                """
                dataset have to contain 'X_mean', 'Y_mean', 'TVD_SCS' and 'KHtst', if you assing value as KHtst
                """
                data = dataset[(dataset.FORMATION_up == dist_formation)]
                row_name = data.well.reset_index().drop(['index'], axis=1)
                distance_fm = pd.DataFrame(euclidean_distances(data[['X_mean', 'Y_mean']]), columns=list(data.well))
                distance_fm_well = distance_fm.join(row_name).set_index('well')
                distance_fm_well.reset_index(inplace=True)
                def well_kh_accum(wells, dataset, kh_formation):
                    well_kh_accum = []
                    well_x_accum = []
                    well_y_accum = []
                    for i in wells:
                        well_kh_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)][value].reset_index())    
                        well_x_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['X_mean'].reset_index())
                        well_y_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['Y_mean'].reset_index())
                    well_kh3 = pd.concat(well_kh_accum).T[1:]
                    well_kh3.columns = [value + '_1',value + '_2', value + '_3']
                    well_x3 = pd.concat(well_x_accum).T[1:]
                    well_x3.columns = ['x1','x2','x3']
                    well_y3 = pd.concat(well_y_accum).T[1:]
                    well_y3.columns = ['y1','y2','y3']
                    final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                                        well_x3.reset_index().drop('index',axis=1), 
                                        well_y3.reset_index().drop('index',axis=1)], axis=1)
                    return final
                df_collect = []
                for num, well_name in enumerate(distance_fm_well.well[:]):
                    well_dist3 = distance_fm_well[distance_fm_well.well == well_name].T[1:].sort_values(by=num)
                    well_dist3_s2 = well_dist3[well_dist3[num] > dist_cutoff][:3].reset_index()
                    well_dist3_tuple = tuple(well_dist3_s2['index'])
                    well_dist3_res = well_dist3_s2.T[1:].reset_index().drop('index', axis=1)   
                    well_name3_res = well_dist3_s2.T[:1].reset_index().drop('index', axis=1)
                    well_kh3_res = well_kh_accum(well_dist3_tuple,dataset, dist_formation)
                    well_dist3_res.columns =['dist1', 'dist2', 'dist3']
                    well_name3_res.columns =['well1', 'well2', 'well3']
                    concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
                    result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
                    df_collect.append(result)     
                df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
                df_well_kh_dist['FORMATION_up'] = dist_formation
                return df_well_kh_dist
            tpb8_cluster0 = training_data_tpb_gm0.groupby('well').apply(lambda x: x.iloc[0]).drop('well',axis=1).reset_index()
            list_well_offsets = dist_prop_calc_xy(tpb8_cluster0, 'Balakhany VIII', 0, 'phit_avg')[
                            ['well','FORMATION_up','well1', 'well2', 'well3','dist1', 'dist2', 'dist3']]
            return list_well_offsets
        list_wells_offsets0 = closest_wells_per_cluster(training_data_tpb_gm0)

        def loop_cluster_data_to_sample(dataset, list_wells_offsets_cluster, wellname):
            target_train = dataset[dataset.well == wellname]
            offset_wells = list_wells_offsets_cluster[list_wells_offsets_cluster.well == wellname]

            ftr1 = dataset[dataset.well == offset_wells.well1.iloc[0]]
            ftr2 = dataset[dataset.well == offset_wells.well2.iloc[0]]
            ftr3 = dataset[dataset.well == offset_wells.well3.iloc[0]]

            def conv_cluster_target_to_sample(dataset):
                def w_avg(phit, h):
                    result = phit*h
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg_target': [phit_wavg_cl]})
                return result

            target_wavg = conv_cluster_target_to_sample(target_train)

            def conv_cluster_feature_to_sample(dataset, offset_wells, well_seq, num):
                suff = '_' + str(num)
                dist = offset_wells[well_seq].iloc[0]
                def w_avg(phit, h):
                    result = phit*h
                    return result
                def summ_h(x1,x2):
                    result = x1+x2
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                dataset['vshh'] =  dataset.apply(lambda x: w_avg(x['vsh_avg'], x['htst']), axis=1)
                dataset['sumh'] =  dataset.apply(lambda x: summ_h(x['top_htst'], x['htst']), axis=1)
                vsh_wavg_cl = dataset['vshh'].sum() / dataset['htst'].sum()
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                gross = dataset['sumh'].sum() + dataset['bot_htst'].iloc[-1]
                net = dataset['htst'].sum()
                ntg = net/gross
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg' + suff: [phit_wavg_cl],
                                        'net' + suff:[net], 
                                        'gross' + suff:[gross], 
                                        'ntg' + suff:[ntg], 
                                        'vsh_wavg' + suff:[vsh_wavg_cl], 
                                        'dist' + suff:dist})
                return result

            ftr1_wavg = conv_cluster_feature_to_sample(ftr1, offset_wells, 'dist1', 1)
            ftr2_wavg = conv_cluster_feature_to_sample(ftr2, offset_wells, 'dist2', 2)
            ftr3_wavg = conv_cluster_feature_to_sample(ftr3, offset_wells, 'dist3', 3)

            return target_wavg, ftr1_wavg, ftr2_wavg, ftr3_wavg
        df_lst = []
        for wellname in training_data_tpb_gm0.well.unique():
            t, f1, f2, f3 = loop_cluster_data_to_sample(training_data_tpb_gm0, list_wells_offsets0, wellname)
            result_dataset = pd.concat([t.iloc[:,3:], f1.iloc[:,3:], f2.iloc[:,3:], f3.iloc[:,3:]], axis=1)
            df_lst.append(result_dataset)
        data_Xy = pd.concat(df_lst).reset_index(drop=True)
        return data_Xy
    data_Xy = conv_cluster_data_to_sample(dataset, dataset_clustering, cluster_num)
    X = data_Xy.iloc[:,1:]
    y = data_Xy['phit_wavg_target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

    # model = XGBRegressor(n_jobs=-1, random_state=42)
    model = cb.CatBoostRegressor()

    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    train = pd.DataFrame(zip(y_train,y_pred_train), columns=['y_orig', 'y_pred'])
    test = pd.DataFrame(zip(y_test,y_pred_test), columns=['y_orig', 'y_pred'])
    tolerance = 0.05
    def test_processing(tolerance):
        test['up'] = test['y_orig']*(1 + tolerance)
        test['down'] = test['y_orig']*(1 - tolerance)
        test['qc'] = 'out'
        test.loc[(test['y_pred'] <= test.up) & (test['y_pred'] >= test.down), 'qc'] = 'in'
        return test
    test = test_processing(tolerance)
    metric = test.qc.value_counts(normalize=True)['in'].round(2)
    def train_test_display(tolerance, min_phit):
        fig, ax = plt.subplots()
        ax.scatter(train['y_orig'], train['y_pred'], label = 'train', ec='gray', alpha=0.5)
        ax.scatter(test['y_orig'], test['y_pred'], label = 'test', ec='gray', alpha=0.9)
        line = mlines.Line2D([min_phit, 0.28], [min_phit, 0.28], color='red', ls='--')
        line1 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 + tolerance), 0.28*(1 + tolerance)], color='blue', ls='--')
        line2 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 - tolerance), 0.28*((1 - tolerance))], color='blue', ls='--')
        ax.add_line(line), ax.add_line(line1), ax.add_line(line2), ax.grid(), ax.legend(), ax.set_ylabel('pred'), ax.set_xlabel('orig'),
        ax.set_xlim(0.14, 0.28), ax.set_ylim(0.14, 0.28) 
        ax.set_title(f'Cluster # {cluster_num} metric "test in": {metric}')
    train_test_display(tolerance, 0.15)
    return  data_Xy, train, test
data_Xy, train, test = catboost_cluster(top_phi_bot8_xy, ntd_val_kmeans3, [0,1,2])

In [115]:
def phit_orig_pred_bl_joining(top_phi_bot_xy, phit_pred_cluster_baseline):
    add_info1 = top_phi_bot_xy[['well','phit_avg','top_htst', 'htst', 'bot_htst']]
    add_info1['phit_avg'] = add_info1['phit_avg'].round(5)
    phit_pred_cluster_bl_collect = phit_pred_cluster_baseline.join(training_data_tpb_notcl.drop(['well','FORMATION_up','htst'],axis=1))
    phit_cluster_orig_pred = phit_pred_cluster_bl_collect.set_index(['well','phit_avg']).join(add_info1.set_index(['well','phit_avg'])).reset_index()
    return phit_cluster_orig_pred
def phit_orig_pred_bl_collecting(dataset):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        phit_pred_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            phit_pred_lst.append(np.nan)
            phit_pred_lst.append(row['phit_cluster_bl'])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

        phit_lst.append(0)
        phit_pred_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, phit_pred_lst ), 
                                            columns=['well','phit_orig', 'htst', 'phit_pred'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
phit_cluster_orig_pred = phit_orig_pred_bl_joining(top_phi_bot8_xy, phit_pred_cluster_baseline)
phit_orig_pred_cluster_bl = phit_orig_pred_bl_collecting(phit_cluster_orig_pred)

In [ ]:
def rfr_cluster(dataset, dataset_clustering, cluster_num, grid_search):
    def conv_cluster_data_to_sample(dataset, dataset_clustering, cluster_num):
        training_data_tpb8 = dataset.copy(deep=True)
        training_data_tpb8 = training_data_tpb8[[ 'well',
                                                'FORMATION_up',
                                                'X_mean', 'Y_mean',
                                                'phit_avg', 
                                                'vsh_avg',
                                                'top_htst',
                                                'htst',
                                                'bot_htst',
                                                'depth',
                                                ]]
        def clusters_joining(dataset_clustering, training_data_tpb):
            cluster_nums = dataset_clustering[(dataset_clustering.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
            cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
            cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
            training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
            training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
            return training_data_tpb_gm
        training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb8)
        training_data_tpb_gm0 = training_data_tpb_gm[training_data_tpb_gm.cluster.isin(cluster_num)]

        def closest_wells_per_cluster(training_data_tpb_gm0):
            def dist_prop_calc_xy(dataset, dist_formation, dist_cutoff, value):
                """
                dataset have to contain 'X_mean', 'Y_mean', 'TVD_SCS' and 'KHtst', if you assing value as KHtst
                """
                data = dataset[(dataset.FORMATION_up == dist_formation)]
                row_name = data.well.reset_index().drop(['index'], axis=1)
                distance_fm = pd.DataFrame(euclidean_distances(data[['X_mean', 'Y_mean']]), columns=list(data.well))
                distance_fm_well = distance_fm.join(row_name).set_index('well')
                distance_fm_well.reset_index(inplace=True)
                def well_kh_accum(wells, dataset, kh_formation):
                    well_kh_accum = []
                    well_x_accum = []
                    well_y_accum = []
                    for i in wells:
                        well_kh_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)][value].reset_index())    
                        well_x_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['X_mean'].reset_index())
                        well_y_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['Y_mean'].reset_index())
                    well_kh3 = pd.concat(well_kh_accum).T[1:]
                    well_kh3.columns = [value + '_1',value + '_2', value + '_3']
                    well_x3 = pd.concat(well_x_accum).T[1:]
                    well_x3.columns = ['x1','x2','x3']
                    well_y3 = pd.concat(well_y_accum).T[1:]
                    well_y3.columns = ['y1','y2','y3']
                    final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                                        well_x3.reset_index().drop('index',axis=1), 
                                        well_y3.reset_index().drop('index',axis=1)], axis=1)
                    return final
                df_collect = []
                for num, well_name in enumerate(distance_fm_well.well[:]):
                    well_dist3 = distance_fm_well[distance_fm_well.well == well_name].T[1:].sort_values(by=num)
                    well_dist3_s2 = well_dist3[well_dist3[num] > dist_cutoff][:3].reset_index()
                    well_dist3_tuple = tuple(well_dist3_s2['index'])
                    well_dist3_res = well_dist3_s2.T[1:].reset_index().drop('index', axis=1)   
                    well_name3_res = well_dist3_s2.T[:1].reset_index().drop('index', axis=1)
                    well_kh3_res = well_kh_accum(well_dist3_tuple,dataset, dist_formation)
                    well_dist3_res.columns =['dist1', 'dist2', 'dist3']
                    well_name3_res.columns =['well1', 'well2', 'well3']
                    concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
                    result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
                    df_collect.append(result)     
                df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
                df_well_kh_dist['FORMATION_up'] = dist_formation
                return df_well_kh_dist
            tpb8_cluster0 = training_data_tpb_gm0.groupby('well').apply(lambda x: x.iloc[0]).drop('well',axis=1).reset_index()
            list_well_offsets = dist_prop_calc_xy(tpb8_cluster0, 'Balakhany VIII', 0, 'phit_avg')[
                            ['well','FORMATION_up','well1', 'well2', 'well3','dist1', 'dist2', 'dist3']]
            return list_well_offsets
        list_wells_offsets0 = closest_wells_per_cluster(training_data_tpb_gm0)

        def loop_cluster_data_to_sample(dataset, list_wells_offsets_cluster, wellname):
            target_train = dataset[dataset.well == wellname]
            offset_wells = list_wells_offsets_cluster[list_wells_offsets_cluster.well == wellname]

            ftr1 = dataset[dataset.well == offset_wells.well1.iloc[0]]
            ftr2 = dataset[dataset.well == offset_wells.well2.iloc[0]]
            ftr3 = dataset[dataset.well == offset_wells.well3.iloc[0]]

            def conv_cluster_target_to_sample(dataset):
                def w_avg(phit, h):
                    result = phit*h
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg_target': [phit_wavg_cl]})
                return result

            target_wavg = conv_cluster_target_to_sample(target_train)

            def conv_cluster_feature_to_sample(dataset, offset_wells, well_seq, num):
                suff = '_' + str(num)
                dist = offset_wells[well_seq].iloc[0]
                def w_avg(phit, h):
                    result = phit*h
                    return result
                def summ_h(x1,x2):
                    result = x1+x2
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                dataset['vshh'] =  dataset.apply(lambda x: w_avg(x['vsh_avg'], x['htst']), axis=1)
                dataset['sumh'] =  dataset.apply(lambda x: summ_h(x['top_htst'], x['htst']), axis=1)
                vsh_wavg_cl = dataset['vshh'].sum() / dataset['htst'].sum()
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                gross = dataset['sumh'].sum() + dataset['bot_htst'].iloc[-1]
                net = dataset['htst'].sum()
                ntg = net/gross
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg' + suff: [phit_wavg_cl],
                                        'net' + suff:[net], 
                                        'gross' + suff:[gross], 
                                        'ntg' + suff:[ntg], 
                                        'vsh_wavg' + suff:[vsh_wavg_cl], 
                                        'dist' + suff:dist})
                return result

            ftr1_wavg = conv_cluster_feature_to_sample(ftr1, offset_wells, 'dist1', 1)
            ftr2_wavg = conv_cluster_feature_to_sample(ftr2, offset_wells, 'dist2', 2)
            ftr3_wavg = conv_cluster_feature_to_sample(ftr3, offset_wells, 'dist3', 3)

            return target_wavg, ftr1_wavg, ftr2_wavg, ftr3_wavg
        df_lst = []
        for wellname in training_data_tpb_gm0.well.unique():
            t, f1, f2, f3 = loop_cluster_data_to_sample(training_data_tpb_gm0, list_wells_offsets0, wellname)
            result_dataset = pd.concat([t.iloc[:,3:], f1.iloc[:,3:], f2.iloc[:,3:], f3.iloc[:,3:]], axis=1)
            df_lst.append(result_dataset)
        data_Xy = pd.concat(df_lst).reset_index(drop=True)
        return data_Xy
    data_Xy = conv_cluster_data_to_sample(dataset, dataset_clustering, cluster_num)
    X = data_Xy.iloc[:,1:]
    y = data_Xy['phit_wavg_target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

    gs_set = {  'bootstrap': [True],
                'max_depth': [100, 150, 200, 250],
                'min_samples_leaf': [5, 10, 15, 20],
                'min_samples_split': [5, 10, 15, 20],
                'n_estimators': [30, 100, 150, 200]}

    if grid_search == 'grid_search':
        scorer_r2 = make_scorer(r2, greater_is_better=True)
        grid_rfr = RandomForestRegressor(n_jobs=-1, random_state=42)
        grid_calc_rfr = GridSearchCV(estimator = grid_rfr, param_grid = gs_set, scoring=scorer_r2, cv = 5)
        grid_calc_rfr.fit(X_train, y_train)
        gd_sr_setting = grid_calc_rfr.best_params_
        print('Grid_search cluster # {cluster_num}: ', gd_sr_setting)
        rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(**gd_sr_setting, n_jobs=-1, random_state=42))])
    else:
        rfr = RandomForestRegressor(n_jobs=-1, random_state=42)

    rfr.fit(X_train, y_train)
    y_pred_train = rfr.predict(X_train)
    y_pred_test = rfr.predict(X_test)

    train = pd.DataFrame(zip(y_train,y_pred_train), columns=['y_orig', 'y_pred'])
    test = pd.DataFrame(zip(y_test,y_pred_test), columns=['y_orig', 'y_pred'])
    tolerance = 0.05
    def test_processing(tolerance):
        test['up'] = test['y_orig']*(1 + tolerance)
        test['down'] = test['y_orig']*(1 - tolerance)
        test['qc'] = 'out'
        test.loc[(test['y_pred'] <= test.up) & (test['y_pred'] >= test.down), 'qc'] = 'in'
        return test
    test = test_processing(tolerance)
    metric = test.qc.value_counts(normalize=True)['in'].round(2)
    def train_test_display(tolerance, min_phit):
        fig, ax = plt.subplots()
        ax.scatter(train['y_orig'], train['y_pred'], label = 'train', ec='gray', alpha=0.5)
        ax.scatter(test['y_orig'], test['y_pred'], label = 'test', ec='gray', alpha=0.9)
        line = mlines.Line2D([min_phit, 0.28], [min_phit, 0.28], color='red', ls='--')
        line1 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 + tolerance), 0.28*(1 + tolerance)], color='blue', ls='--')
        line2 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 - tolerance), 0.28*((1 - tolerance))], color='blue', ls='--')
        ax.add_line(line), ax.add_line(line1), ax.add_line(line2), ax.grid(), ax.legend(), ax.set_ylabel('pred'), ax.set_xlabel('orig'),
        ax.set_xlim(0.14, 0.28), ax.set_ylim(0.14, 0.28) 
        ax.set_title(f'Cluster # {cluster_num} metric "test in": {metric}')
    train_test_display(tolerance, 0.15)
    return  data_Xy, train, test
data_Xy, train0, test0 = rfr_cluster(top_phi_bot8_xy, ntd_val_kmeans3, [0,1,2], 'dont_gs')
# data_Xy0, train0, test0 = rfr_cluster(top_phi_bot8_xy, ntd_val_kmeans3, [0], 'dont_gs')
# data_Xy1, train1, test1 = rfr_cluster(top_phi_bot8_xy, ntd_val_kmeans3, [1], 'dont_gs')
# data_Xy2, train2, test2 = rfr_cluster(top_phi_bot8_xy, ntd_val_kmeans3, [2], 'dont_gs')

#### Cluster 0

In [106]:
def join_df_prediction(base_dataframe, target_var):
    def columns_reorder(dataset, selected_column):
        new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
        dataset = dataset[new_order]
        return dataset
    def cat_finder(dataset):
        """
        cat_list: categorical columns to drop out
        get_dum_list: categorical columns to run via pd.get_dummies
        """
        cat_list = []
        gm_list = []
        for col in dataset.columns:
            if dataset[col].dtype == 'string' or dataset[col].dtype == 'object':
                cat_list.append(col)
                if col != 'well':
                    gm_list.append(col)
        return cat_list, gm_list
    col_names, gm_list = cat_finder(base_dataframe)
    df_corr = base_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(base_dataframe[gm_list], columns=gm_list, drop_first=True)
    dataframe = pd.concat([base_dataframe, mem_cell], axis=1)
    return df_corr, dataframe
def pairplot_special(dataset, xsize, ysize, flag=1):
    if flag == 1:
        def corrfunc(x, y, **kws):
            r, _ = stats.pearsonr(x, y)
            ax = plt.gca()
            ax.annotate("r = {:.2f}".format(r),
                        xy=(.1, .9), xycoords=ax.transAxes)
        sns.set_context(rc={'axes.labelsize':10, 'lines.linewidth': 0.75})
        g = sns.PairGrid(dataset)
        g.fig.set_size_inches(xsize,ysize)
        g.set(xticklabels=[], yticklabels=[]) 
        g.map_upper(plt.scatter, s=10, alpha=0.5)
        g.map_diag(sns.distplot, kde=False)
        g.map_lower(sns.kdeplot, cmap="Blues_d")
        g.map_lower(corrfunc)
    else:
        pass
training_data_tpb8 = top_phi_bot8_xy.copy(deep=True)
training_data_tpb8 = training_data_tpb8[[ 'well',
                                        'FORMATION_up',
                                        'X_mean', 'Y_mean',
                                        'phit_avg', 
                                        'vsh_avg',
                                        'htst',
                                        'depth',
                                        ]]
def clusters_joining(ntd_val_kmeans3, training_data_tpb):
    cluster_nums = ntd_val_kmeans3[(ntd_val_kmeans3.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
    cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
    cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
    training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
    training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
    return training_data_tpb_gm
training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb8)
training_data_tpb_gm0 = training_data_tpb_gm[training_data_tpb_gm.cluster == 0].drop('cluster', axis=1)
training_data_tpb_corr, training_data_tpb_gm0 = join_df_prediction(training_data_tpb_gm0, 'phit_avg')
training_data_tpb_gm0 = training_data_tpb_gm0.reset_index().drop('index', axis=1)
pairplot_special(training_data_tpb_corr, 7, 7, 0)

In [ ]:
# Run train-split block to get an optimized hyper parameters for 1-to-all function
gs_set = {  'bootstrap': [True, False],
            'max_depth': [10, 30, 100, 150, 200],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [1, 5, 10],
            'n_estimators': [10, 30, 100, 150, 200]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict = run_rfr_train_test_split(training_data_tpb_gm0, gs_set, scorer, 'phit_avg', 
                                     0.05, 0, 'training_data_tpb_gm.txt', 'training_data_tpb_gm_grid_search', 0, 0)

In [98]:
training_data_tpb_gm0 = training_data_tpb_gm0.reset_index().drop('index',axis=1)

In [ ]:
# Run 1-to-all block to to predict target vaiable
phit_pred_cluster_cl0 = run_rfr_1_to_all(training_data_tpb_gm0, hyperdict, 'phit_avg', 
                                         0.05, 0, 'training_data_tpb_gm.txt', 'training_data_tpb_gm_1-to-all', 1, 0.3, 0)
phit_pred_cluster_cl0 =  phit_pred_cluster_cl0.rename(columns={'predict':'phit_cluster0'})[
                                            ['well','FORMATION_up','phit_cluster0','qc']].sort_values(
                                            by=['well','FORMATION_up']).reset_index().drop('index', axis=1)

In [ ]:
def phit_orig_pred_joining(dataset_xy, dataset_training, dataset_phit_pred):
    add_info = dataset_xy[['well','phit_avg','top_htst', 'htst', 'bot_htst']]
    add_info['phit_avg'] = add_info['phit_avg'].round(5)
    phit_pred_cluster_collect = dataset_phit_pred.join(dataset_training.drop(['well','FORMATION_up','htst'],axis=1))
    phit_cluster_orig_pred =    phit_pred_cluster_collect.set_index(['well','phit_avg']).join(
                                add_info.set_index(['well','phit_avg'])).reset_index()
    return phit_cluster_orig_pred
def phit_orig_pred_collecting(dataset, target_column):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        phit_pred_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            phit_pred_lst.append(np.nan)
            phit_pred_lst.append(row[target_column])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

        phit_lst.append(0)
        phit_pred_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, phit_pred_lst ), 
                                            columns=['well','phit_orig', 'htst', 'phit_pred'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
phit_cluster_orig_pred0 = phit_orig_pred_joining(top_phi_bot8_xy, training_data_tpb_gm0, phit_pred_cluster_cl0)
phit_orig_pred_cluster_cl0 = phit_orig_pred_collecting(phit_cluster_orig_pred0, 'phit_cluster0')

def coloring_clusters_matrix_plot(dataset, letters_list, rows, columns):
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit_orig':[0], 
                                           'htst':[0], 'phit_pred':[0], 'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit_orig'], welldata['depth'], drawstyle='steps-post', color='gray', alpha=1, lw=1.5)
                    ax[j,i].plot(welldata['phit_pred'], welldata['depth'], drawstyle='steps-post', color='blue', alpha=1, lw=2)
                    ax[j,i].set_xlim(0.1, 0.3)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)   
                    fig.tight_layout()
                    counter +=1
coloring_clusters_matrix_plot(  phit_orig_pred_cluster_cl0, 
                                ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], 4, 9)

#### Cluster 1

In [102]:
def join_df_prediction(base_dataframe, target_var):
    def columns_reorder(dataset, selected_column):
        new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
        dataset = dataset[new_order]
        return dataset
    def cat_finder(dataset):
        """
        cat_list: categorical columns to drop out
        get_dum_list: categorical columns to run via pd.get_dummies
        """
        cat_list = []
        gm_list = []
        for col in dataset.columns:
            if dataset[col].dtype == 'string' or dataset[col].dtype == 'object':
                cat_list.append(col)
                if col != 'well':
                    gm_list.append(col)
        return cat_list, gm_list
    col_names, gm_list = cat_finder(base_dataframe)
    df_corr = base_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(base_dataframe[gm_list], columns=gm_list, drop_first=True)
    dataframe = pd.concat([base_dataframe, mem_cell], axis=1)
    return df_corr, dataframe
def pairplot_special(dataset, xsize, ysize, flag=1):
    if flag == 1:
        def corrfunc(x, y, **kws):
            r, _ = stats.pearsonr(x, y)
            ax = plt.gca()
            ax.annotate("r = {:.2f}".format(r),
                        xy=(.1, .9), xycoords=ax.transAxes)
        sns.set_context(rc={'axes.labelsize':10, 'lines.linewidth': 0.75})
        g = sns.PairGrid(dataset)
        g.fig.set_size_inches(xsize,ysize)
        g.set(xticklabels=[], yticklabels=[]) 
        g.map_upper(plt.scatter, s=10, alpha=0.5)
        g.map_diag(sns.distplot, kde=False)
        g.map_lower(sns.kdeplot, cmap="Blues_d")
        g.map_lower(corrfunc)
    else:
        pass
training_data_tpb = top_phi_bot8_xy.copy(deep=True)
training_data_tpb = training_data_tpb[[ 'well',
                                        'FORMATION_up',
                                        'X_mean', 'Y_mean',
                                        'phit_avg', 
                                        'vsh_avg',
                                        'htst',
                                        'depth',
                                        ]]
def clusters_joining(ntd_val_kmeans3, training_data_tpb):
    cluster_nums = ntd_val_kmeans3[(ntd_val_kmeans3.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
    cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
    cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
    training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
    training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
    return training_data_tpb_gm
training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb)
training_data_tpb_gm1 = training_data_tpb_gm[training_data_tpb_gm.cluster == 1].drop('cluster', axis=1)
training_data_tpb_corr, training_data_tpb_gm1 = join_df_prediction(training_data_tpb_gm1, 'phit_avg')
training_data_tpb_gm1 = training_data_tpb_gm1.reset_index().drop('index', axis=1)
pairplot_special(training_data_tpb_corr, 7, 7, 0)

In [ ]:
# Run train-split block to get an optimized hyper parameters for 1-to-all function
gs_set = {  'bootstrap': [True, False],
            'max_depth': [10, 30, 100, 150, 200],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [1, 5, 10],
            'n_estimators': [10, 30, 100, 150, 200]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict = run_rfr_train_test_split(training_data_tpb_gm1, gs_set, scorer, 'phit_avg', 
                                     0.05, 0, 'training_data_tpb_gm.txt', 'training_data_tpb_gm_grid_search', 0, 0)

In [104]:
training_data_tpb_gm1 = training_data_tpb_gm1.reset_index().drop('index',axis=1)

In [ ]:
# Run 1-to-all block to to predict target vaiable
phit_pred_cluster_cl1 = run_rfr_1_to_all(training_data_tpb_gm1, hyperdict, 'phit_avg', 
                                         0.05, 0, 'training_data_tpb_gm.txt', 'training_data_tpb_gm_1-to-all', 1, 0.3, 0)
phit_pred_cluster_cl1 =  phit_pred_cluster_cl1.rename(columns={'predict':'phit_cluster1'})[
                                            ['well','FORMATION_up','phit_cluster1','qc']].sort_values(
                                            by=['well','FORMATION_up']).reset_index().drop('index', axis=1)

In [ ]:
def phit_orig_pred_joining(dataset_xy, dataset_training, dataset_phit_pred):
    add_info = dataset_xy[['well','phit_avg','top_htst', 'htst', 'bot_htst']]
    add_info['phit_avg'] = add_info['phit_avg'].round(5)
    phit_pred_cluster_collect = dataset_phit_pred.join(dataset_training.drop(['well','FORMATION_up'],axis=1))
    phit_cluster_orig_pred =    phit_pred_cluster_collect.set_index(['well','phit_avg']).join(
                                add_info.set_index(['well','phit_avg'])).reset_index()
    return phit_cluster_orig_pred
def phit_orig_pred_collecting(dataset, target_column):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        phit_pred_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            phit_pred_lst.append(np.nan)
            phit_pred_lst.append(row[target_column])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

        phit_lst.append(0)
        phit_pred_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, phit_pred_lst ), 
                                            columns=['well','phit_orig', 'htst', 'phit_pred'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
phit_cluster_orig_pred1 = phit_orig_pred_joining(top_phi_bot_xy, training_data_tpb_gm1, phit_pred_cluster_cl1)
phit_orig_pred_cluster_cl1 = phit_orig_pred_collecting(phit_cluster_orig_pred1, 'phit_cluster1')

def coloring_clusters_matrix_plot(dataset, letters_list, rows, columns):
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit_orig':[0], 
                                           'htst':[0], 'phit_pred':[0], 'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit_orig'], welldata['depth'], drawstyle='steps-post', color='gray', alpha=1, lw=1.5)
                    ax[j,i].plot(welldata['phit_pred'], welldata['depth'], drawstyle='steps-post', color='green', alpha=1, lw=2)
                    ax[j,i].set_xlim(0.1, 0.3)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)   
                    fig.tight_layout()
                    counter +=1
coloring_clusters_matrix_plot(  phit_orig_pred_cluster_cl1, 
                                ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], 4, 9)

#### Cluster 2

In [ ]:
def join_df_prediction(base_dataframe, target_var):
    def columns_reorder(dataset, selected_column):
        new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
        dataset = dataset[new_order]
        return dataset
    def cat_finder(dataset):
        """
        cat_list: categorical columns to drop out
        get_dum_list: categorical columns to run via pd.get_dummies
        """
        cat_list = []
        gm_list = []
        for col in dataset.columns:
            if dataset[col].dtype == 'string' or dataset[col].dtype == 'object':
                cat_list.append(col)
                if col != 'well':
                    gm_list.append(col)
        return cat_list, gm_list
    col_names, gm_list = cat_finder(base_dataframe)
    df_corr = base_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(base_dataframe[gm_list], columns=gm_list, drop_first=True)
    dataframe = pd.concat([base_dataframe, mem_cell], axis=1)
    return df_corr, dataframe
def pairplot_special(dataset, xsize, ysize, flag=1):
    if flag == 1:
        def corrfunc(x, y, **kws):
            r, _ = stats.pearsonr(x, y)
            ax = plt.gca()
            ax.annotate("r = {:.2f}".format(r),
                        xy=(.1, .9), xycoords=ax.transAxes)
        sns.set_context(rc={'axes.labelsize':10, 'lines.linewidth': 0.75})
        g = sns.PairGrid(dataset)
        g.fig.set_size_inches(xsize,ysize)
        g.set(xticklabels=[], yticklabels=[]) 
        g.map_upper(plt.scatter, s=10, alpha=0.5)
        g.map_diag(sns.distplot, kde=False)
        g.map_lower(sns.kdeplot, cmap="Blues_d")
        g.map_lower(corrfunc)
    else:
        pass
training_data_tpb = top_phi_bot_xy.copy(deep=True)
training_data_tpb = training_data_tpb[[ 'well',
                                        'FORMATION_up',
                                        'X_mean', 'Y_mean',
                                        'phit_avg', 
                                        'vsh_avg',
                                        # 'htst',
                                        # 'fm_top_tst',
                                        ]]
def clusters_joining(ntd_val_kmeans3, training_data_tpb):
    cluster_nums = ntd_val_kmeans3[(ntd_val_kmeans3.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
    cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
    cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
    training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
    training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
    return training_data_tpb_gm
training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb)
training_data_tpb_gm2 = training_data_tpb_gm[training_data_tpb_gm.cluster == 2].drop('cluster', axis=1)
training_data_tpb_corr, training_data_tpb_gm2 = join_df_prediction(training_data_tpb_gm2, 'phit_avg')
training_data_tpb_gm2 = training_data_tpb_gm2.reset_index().drop('index', axis=1)
pairplot_special(training_data_tpb_corr, 7, 7, 1)

In [ ]:
training_data_tpb_gm2 = training_data_tpb_gm2.reset_index().drop('index',axis=1)
training_data_tpb_gm2

In [ ]:
# Run train-split block to get an optimized hyper parameters for 1-to-all function
gs_set = {  'bootstrap': [True, False],
            'max_depth': [10, 30, 100, 150, 200],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [1, 5, 10],
            'n_estimators': [10, 30, 100, 150, 200]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict = run_rfr_train_test_split(training_data_tpb_gm2, gs_set, scorer, 'phit_avg', 
                                     0.05, 0, 'training_data_tpb_gm.txt', 'training_data_tpb_gm_grid_search', 0, 0)

In [ ]:
# Run 1-to-all block to to predict target vaiable
phit_pred_cluster_cl2 = run_rfr_1_to_all(training_data_tpb_gm2, hyperdict, 'phit_avg', 
                                         0.05, 0, 'training_data_tpb_gm.txt', 'training_data_tpb_gm_1-to-all', 1, 0.3, 0)
phit_pred_cluster_cl2 =  phit_pred_cluster_cl2.rename(columns={'predict':'phit_cluster2'})[
                                            ['well','FORMATION_up','phit_cluster2','qc']].sort_values(
                                            by=['well','FORMATION_up']).reset_index().drop('index', axis=1)

In [ ]:
def phit_orig_pred_joining(dataset_xy, dataset_training, dataset_phit_pred):
    add_info = dataset_xy[['well','phit_avg','top_htst', 'htst', 'bot_htst']]
    add_info['phit_avg'] = add_info['phit_avg'].round(5)
    phit_pred_cluster_collect = dataset_phit_pred.join(dataset_training.drop(['well','FORMATION_up'],axis=1))
    phit_cluster_orig_pred =    phit_pred_cluster_collect.set_index(['well','phit_avg']).join(
                                add_info.set_index(['well','phit_avg'])).reset_index()
    return phit_cluster_orig_pred
def phit_orig_pred_collecting(dataset, target_column):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        phit_pred_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            phit_pred_lst.append(np.nan)
            phit_pred_lst.append(row[target_column])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])

        phit_lst.append(0)
        phit_pred_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)

        well_collect_cluster = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, phit_pred_lst ), 
                                            columns=['well','phit_orig', 'htst', 'phit_pred'])
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    return result
phit_cluster_orig_pred2 = phit_orig_pred_joining(top_phi_bot_xy, training_data_tpb_gm2, phit_pred_cluster_cl2)
phit_orig_pred_cluster_cl2 = phit_orig_pred_collecting(phit_cluster_orig_pred2, 'phit_cluster2')

def coloring_clusters_matrix_plot(dataset, letters_list, rows, columns):
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit_orig':[0], 
                                           'htst':[0], 'phit_pred':[0], 'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit_orig'], welldata['depth'], drawstyle='steps-post', color='gray', alpha=1, lw=1.5)
                    ax[j,i].plot(welldata['phit_pred'], welldata['depth'], drawstyle='steps-post', color='red', alpha=1, lw=2)
                    ax[j,i].set_xlim(0.1, 0.3)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].tick_params(axis='both', which='major', labelsize=10)   
                    fig.tight_layout()
                    counter +=1
coloring_clusters_matrix_plot(  phit_orig_pred_cluster_cl2, 
                                ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], 4, 9)

## Prediction Optuna

In [1]:
def rfr_cluster(dataset, cluster_num, grid_search):
    def conv_cluster_data_to_sample(dataset, cluster_num):
        training_data_tpb8 = dataset.copy(deep=True)
        training_data_tpb8 = training_data_tpb8[[ 'well',
                                                'FORMATION_up',
                                                'X_mean', 'Y_mean',
                                                'phit_avg', 
                                                'vsh_avg',
                                                'top_htst',
                                                'htst',
                                                'bot_htst',
                                                'depth',
                                                ]]
        def clusters_joining(ntd_val_kmeans3, training_data_tpb):
            cluster_nums = ntd_val_kmeans3[(ntd_val_kmeans3.phit != 0)][['well','phit','cluster']].reset_index().drop('index', axis=1)
            cluster_nums = cluster_nums.rename(columns={'phit':'phit_avg'})
            cluster_nums['phit_avg'] = cluster_nums['phit_avg'].round(5)
            training_data_tpb['phit_avg'] = training_data_tpb['phit_avg'].round(5)
            training_data_tpb_gm = training_data_tpb.set_index(['well','phit_avg']).join(cluster_nums.set_index(['well','phit_avg'])).reset_index()
            return training_data_tpb_gm
        training_data_tpb_gm = clusters_joining(ntd_val_kmeans3, training_data_tpb8)
        training_data_tpb_gm0 = training_data_tpb_gm[training_data_tpb_gm.cluster == cluster_num]

        def closest_wells_per_cluster(training_data_tpb_gm0):
            def dist_prop_calc_xy(dataset, dist_formation, dist_cutoff, value):
                """
                dataset have to contain 'X_mean', 'Y_mean', 'TVD_SCS' and 'KHtst', if you assing value as KHtst
                """
                data = dataset[(dataset.FORMATION_up == dist_formation)]
                row_name = data.well.reset_index().drop(['index'], axis=1)
                distance_fm = pd.DataFrame(euclidean_distances(data[['X_mean', 'Y_mean']]), columns=list(data.well))
                distance_fm_well = distance_fm.join(row_name).set_index('well')
                distance_fm_well.reset_index(inplace=True)
                def well_kh_accum(wells, dataset, kh_formation):
                    well_kh_accum = []
                    well_x_accum = []
                    well_y_accum = []
                    for i in wells:
                        well_kh_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)][value].reset_index())    
                        well_x_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['X_mean'].reset_index())
                        well_y_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['Y_mean'].reset_index())
                    well_kh3 = pd.concat(well_kh_accum).T[1:]
                    well_kh3.columns = [value + '_1',value + '_2', value + '_3']
                    well_x3 = pd.concat(well_x_accum).T[1:]
                    well_x3.columns = ['x1','x2','x3']
                    well_y3 = pd.concat(well_y_accum).T[1:]
                    well_y3.columns = ['y1','y2','y3']
                    final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                                        well_x3.reset_index().drop('index',axis=1), 
                                        well_y3.reset_index().drop('index',axis=1)], axis=1)
                    return final
                df_collect = []
                for num, well_name in enumerate(distance_fm_well.well[:]):
                    well_dist3 = distance_fm_well[distance_fm_well.well == well_name].T[1:].sort_values(by=num)
                    well_dist3_s2 = well_dist3[well_dist3[num] > dist_cutoff][:3].reset_index()
                    well_dist3_tuple = tuple(well_dist3_s2['index'])
                    well_dist3_res = well_dist3_s2.T[1:].reset_index().drop('index', axis=1)   
                    well_name3_res = well_dist3_s2.T[:1].reset_index().drop('index', axis=1)
                    well_kh3_res = well_kh_accum(well_dist3_tuple,dataset, dist_formation)
                    well_dist3_res.columns =['dist1', 'dist2', 'dist3']
                    well_name3_res.columns =['well1', 'well2', 'well3']
                    concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
                    result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
                    df_collect.append(result)     
                df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
                df_well_kh_dist['FORMATION_up'] = dist_formation
                return df_well_kh_dist
            tpb8_cluster0 = training_data_tpb_gm0.groupby('well').apply(lambda x: x.iloc[0]).drop('well',axis=1).reset_index()
            list_well_offsets = dist_prop_calc_xy(tpb8_cluster0, 'Balakhany VIII', 0, 'phit_avg')[
                            ['well','FORMATION_up','well1', 'well2', 'well3','dist1', 'dist2', 'dist3']]
            return list_well_offsets
        list_wells_offsets0 = closest_wells_per_cluster(training_data_tpb_gm0)

        def loop_cluster_data_to_sample(dataset, list_wells_offsets_cluster, wellname):
            target_train = dataset[dataset.well == wellname]
            offset_wells = list_wells_offsets_cluster[list_wells_offsets_cluster.well == wellname]

            ftr1 = dataset[dataset.well == offset_wells.well1.iloc[0]]
            ftr2 = dataset[dataset.well == offset_wells.well2.iloc[0]]
            ftr3 = dataset[dataset.well == offset_wells.well3.iloc[0]]

            def conv_cluster_target_to_sample(dataset):
                def w_avg(phit, h):
                    result = phit*h
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg_target': [phit_wavg_cl]})
                return result

            target_wavg = conv_cluster_target_to_sample(target_train)

            def conv_cluster_feature_to_sample(dataset, offset_wells, well_seq, num):
                suff = '_' + str(num)
                dist = offset_wells[well_seq].iloc[0]
                def w_avg(phit, h):
                    result = phit*h
                    return result
                def summ_h(x1,x2):
                    result = x1+x2
                    return result
                dataset['phith'] =  dataset.apply(lambda x: w_avg(x['phit_avg'], x['htst']), axis=1)
                dataset['vshh'] =  dataset.apply(lambda x: w_avg(x['vsh_avg'], x['htst']), axis=1)
                dataset['sumh'] =  dataset.apply(lambda x: summ_h(x['top_htst'], x['htst']), axis=1)
                vsh_wavg_cl = dataset['vshh'].sum() / dataset['htst'].sum()
                phit_wavg_cl = dataset['phith'].sum() / dataset['htst'].sum()
                gross = dataset['sumh'].sum() + dataset['bot_htst'].iloc[-1]
                net = dataset['htst'].sum()
                ntg = net/gross
                well = dataset['well'].iloc[0]
                fm = dataset['FORMATION_up'].iloc[0]
                cluster = dataset['cluster'].iloc[0]
                result = pd.DataFrame({ 'well':[well], 
                                        'FORMATION_up': [fm], 
                                        'cluster': [cluster], 
                                        'phit_wavg' + suff: [phit_wavg_cl],
                                        'net' + suff:[net], 
                                        'gross' + suff:[gross], 
                                        'ntg' + suff:[ntg], 
                                        'vsh_wavg' + suff:[vsh_wavg_cl], 
                                        'dist' + suff:dist})
                return result

            ftr1_wavg = conv_cluster_feature_to_sample(ftr1, offset_wells, 'dist1', 1)
            ftr2_wavg = conv_cluster_feature_to_sample(ftr2, offset_wells, 'dist2', 2)
            ftr3_wavg = conv_cluster_feature_to_sample(ftr3, offset_wells, 'dist3', 3)

            return target_wavg, ftr1_wavg, ftr2_wavg, ftr3_wavg
        df_lst = []
        for wellname in training_data_tpb_gm0.well.unique():
            t, f1, f2, f3 = loop_cluster_data_to_sample(training_data_tpb_gm0, list_wells_offsets0, wellname)
            result_dataset = pd.concat([t.iloc[:,3:], f1.iloc[:,3:], f2.iloc[:,3:], f3.iloc[:,3:]], axis=1)
            df_lst.append(result_dataset)
        data_Xy = pd.concat(df_lst).reset_index(drop=True)
        return data_Xy
    data_Xy = conv_cluster_data_to_sample(dataset, cluster_num)
    X = data_Xy.iloc[:,1:]
    y = data_Xy['phit_wavg_target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

    gs_set = {  'bootstrap': [True],
                'max_depth': [100, 150, 200, 250],
                'min_samples_leaf': [5, 10, 15, 20],
                'min_samples_split': [5, 10, 15, 20],
                'n_estimators': [30, 100, 150, 200]}

    if grid_search == 'grid_search':
        scorer_r2 = make_scorer(r2, greater_is_better=True)
        grid_rfr = RandomForestRegressor(n_jobs=-1, random_state=42)
        grid_calc_rfr = GridSearchCV(estimator = grid_rfr, param_grid = gs_set, scoring=scorer_r2, cv = 5)
        grid_calc_rfr.fit(X_train, y_train)
        gd_sr_setting = grid_calc_rfr.best_params_
        print('Grid_search cluster # {cluster_num}: ', gd_sr_setting)
        rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(**gd_sr_setting, n_jobs=-1, random_state=42))])
    else:
        rfr = RandomForestRegressor(n_jobs=-1, random_state=42)

    rfr.fit(X_train, y_train)
    y_pred_train = rfr.predict(X_train)
    y_pred_test = rfr.predict(X_test)

    train = pd.DataFrame(zip(y_train,y_pred_train), columns=['y_orig', 'y_pred'])
    test = pd.DataFrame(zip(y_test,y_pred_test), columns=['y_orig', 'y_pred'])
    tolerance = 0.05
    def test_processing(tolerance):
        test['up'] = test['y_orig']*(1 + tolerance)
        test['down'] = test['y_orig']*(1 - tolerance)
        test['qc'] = 'out'
        test.loc[(test['y_pred'] <= test.up) & (test['y_pred'] >= test.down), 'qc'] = 'in'
        return test
    test = test_processing(tolerance)
    metric = test.qc.value_counts(normalize=True)['in'].round(2)
    def train_test_display(tolerance, min_phit):
        fig, ax = plt.subplots()
        ax.scatter(train['y_orig'], train['y_pred'], label = 'train', ec='gray', alpha=0.5)
        ax.scatter(test['y_orig'], test['y_pred'], label = 'test', ec='gray', alpha=0.9)
        line = mlines.Line2D([min_phit, 0.28], [min_phit, 0.28], color='red', ls='--')
        line1 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 + tolerance), 0.28*(1 + tolerance)], color='blue', ls='--')
        line2 = mlines.Line2D([min_phit, 0.28], [min_phit*(1 - tolerance), 0.28*((1 - tolerance))], color='blue', ls='--')
        ax.add_line(line), ax.add_line(line1), ax.add_line(line2), ax.grid(), ax.legend(), ax.set_ylabel('pred'), ax.set_xlabel('orig'),
        ax.set_xlim(0.14, 0.28), ax.set_ylim(0.14, 0.28) 
        ax.set_title(f'Cluster # {cluster_num} metric "test in": {metric}')
    train_test_display(tolerance, 0.15)
    return  data_Xy, train, test

In [ ]:
from sklearn.metrics import r2_score
X = data_Xy.iloc[:,1:]
y = data_Xy['phit_wavg_target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=False, test_size=0.3)
X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, shuffle=True,  random_state=42)
param = {
    'tree_method':'approx',  # this parameter means using the GPU when training our model to speedup the training process
    'sampling_method': 'uniform',
    # 'lambda': trial.suggest_loguniform('lambda', 7.0, 17.0),
    # 'alpha': trial.suggest_loguniform('alpha', 7.0, 17.0),
    'eta': trial.suggest_categorical('eta', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
    # 'gamma': trial.suggest_categorical('gamma', [18, 19, 20, 21, 22, 23, 24, 25]),
    'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
    'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
    # 'colsample_bynode': trial.suggest_categorical('colsample_bynode', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
    'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
    # 'min_child_weight': trial.suggest_int('min_child_weight', 8, 600),  
    # 'max_depth': trial.suggest_categorical('max_depth', [3, 4, 5, 6, 7]),  
    # 'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
    'random_state': 42
}
model = XGBRegressor(**param)
# model = XGBRegressor()      
model.fit(X_trainval, y_trainval, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
predict = model.predict(X_valid)
r_2 = r2_score(predict, y_valid)
test_optuna = pd.DataFrame(zip(y_valid,predict), columns=['y_orig', 'y_pred'])
plt.scatter(test_optuna['y_orig'], test_optuna['y_pred'])
plt.xlim(0.15,0.28), plt.ylim(0.15,0.28)
plt.grid()
r_2

In [ ]:
import optuna
from sklearn.metrics import r2_score

X = data_Xy.iloc[:,1:]
y = data_Xy['phit_wavg_target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.3)
X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, shuffle=True,  random_state=42)
def objective(trial):
    param = {
        'tree_method':'approx',  # this parameter means using the GPU when training our model to speedup the training process
        'sampling_method': 'uniform',
        # 'lambda': trial.suggest_loguniform('lambda', 7.0, 17.0),
        # 'alpha': trial.suggest_loguniform('alpha', 7.0, 17.0),
        'eta': trial.suggest_categorical('eta', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        # 'gamma': trial.suggest_categorical('gamma', [18, 19, 20, 21, 22, 23, 24, 25]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        # 'colsample_bynode': trial.suggest_categorical('colsample_bynode', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        # 'min_child_weight': trial.suggest_int('min_child_weight', 8, 600),  
        # 'max_depth': trial.suggest_categorical('max_depth', [3, 4, 5, 6, 7]),  
        # 'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
        'random_state': 42
    }
    model = XGBRegressor(**param)    
    model.fit(X_trainval, y_trainval, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predict = model.predict(X_valid)
    r_2 = r2_score(predict, y_valid)
    return r_2

study = optuna.create_study()
study.optimize(objective, n_trials=50,  timeout=600)
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
fig = optuna.visualization.plot_slice(study)
fig.update_layout(title='<b>Slice Plot', title_x=0.2)

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(title='<b>Optimization History Plot', title_x=0.5)

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.update_layout(title='<b>Hyperparameter Importances', title_x=0.5)

In [ ]:
best_model = XGBRegressor(**study.best_params)
best_model.fit(X_train, y_train)

In [ ]:
print(f'Best model result in Test: {r2_score(y_test, best_model.predict(X_test))}')
print(f'Best model result in Train: {r2_score(y_train, best_model.predict(X_train))}')

In [ ]:
test_optuna2 = pd.DataFrame(zip(y_test, best_model.predict(X_test)), columns=['y_orig', 'y_pred'])
plt.scatter(test_optuna2['y_orig'], test_optuna2['y_pred'])
plt.xlim(0.15,0.28), plt.ylim(0.15,0.28)
plt.grid()
r2_score(y_test, best_model.predict(X_test))

## Geopandas Zone

In [ ]:
surf1 = np.loadtxt('C:\jupyter\SPP\input\surfaces\PW_H10_Dec22_CACI_5176_M400000_QLSKPrSDM_SCF_balVIIIs_ismat4')
X = surf1[:,0] 
Y = surf1[:,1] 
Z = surf1[:,2]
plt.scatter(X,Y, c=Z)
plt.colorbar(label='depth')

In [37]:
bal8_1510_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_3.shp')
bal8_1510_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_1.shp')
bal8_1510_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_0.shp')
bal8_20_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_3.shp')
bal8_1510_3 = bal8_1510_3.set_crs('EPSG:2499')
bal8_1510_1 = bal8_1510_1.set_crs('EPSG:2499')
bal8_1510_0 = bal8_1510_0.set_crs('EPSG:2499')
bal8_20_3 = bal8_20_3.set_crs('EPSG:2499')

In [ ]:
bdl8_xy = df_bal_net2_kh[df_bal_net2_kh.FORMATION_up == 'Balakhany VIII'][['well','X_mean','Y_mean']]
bdl8_xy = bdl8_xy.groupby('well').mean().reset_index()
geometry = [Point(xy) for xy in zip(bdl8_xy['X_mean'], bdl8_xy['Y_mean'])]
bdl8_xy_gpd = gpd.GeoDataFrame(bdl8_xy, geometry=geometry).drop(['X_mean','Y_mean'], axis=1)
buffers = bdl8_xy_gpd.buffer(250)
buffers = gpd.GeoDataFrame(geometry=buffers)
bdl8_xy_gpd = bdl8_xy_gpd.join(buffers, rsuffix='_plgn')
bdl8_xy_gpd = gpd.GeoDataFrame(bdl8_xy_gpd, geometry='geometry_plgn').set_crs('EPSG:2499')
bdl8_xy_gpd

In [ ]:
cross_section = gpd.overlay(bal8_1510_3, bdl8_xy_gpd, how='intersection')
cross_section

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
bal8_1510_3.plot(ax=ax, color='red', label='bal8_1510_3')
# bal8_1510_1.plot(ax=ax, color='green', label='bal8_1510_1')
# bal8_1510_0.plot(ax=ax, color='blue', label='bal8_1510_0')
bal8_20_3.plot(ax=ax, color='orange', label='bal8_20_3')
bdl8_xy_gpd.plot(ax=ax, markersize = 3, color='green', label='wells', alpha=0.5)
# ax.grid()
ax.legend()
ax.set_title('Polygons of Balakhany VIII 20 15 10 body #3 & wells (buffer 250m)')